<a href="https://colab.research.google.com/github/bbanzai88/Artificial_Influencers/blob/main/SynbioCrowV003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This system is an automated end-to-end bio-retrosynthesis and design platform that integrates multiple computational tools into a unified LangGraph-based workflow. It identifies, evaluates, and simulates novel enzymatic pathways for the biosynthesis of target molecules, combining retrosynthetic search, enzyme selection, thermodynamic filtering, and metabolic simulation.

Key Capabilities:

*   Automated Retrosynthesis (RetroBioCat2)
*   Generates de novo biochemical pathways from a target molecule’s SMILES.
*   Performs Monte Carlo Tree Search (MCTS) over reaction rules to explore feasible routes.
*   Pathway Extraction & Annotation
*   Converts retrosynthesis outputs into structured reaction tables (substrates, products, scores).
*   Integrates similarity metrics, prior precedents, and enzyme reaction type metadata.
*   Thermodynamic & Ranking Modules
*   Uses dGPredictor / eQuilibrator to estimate Gibbs free energies (ΔG′).
*   Ranks pathways based on combined heuristic and thermodynamic feasibility scores.
*   Enzyme Identification (Selenzyme)
*   Predicts candidate enzymes for each step, optionally scraping Selenzyme for activity and organism data.
*   Metabolic Simulation (COBRApy / novoStoic)
*   Constructs toy stoichiometric models for top-ranked pathways.
*   Runs flux balance analysis (FBA) to estimate steady-state feasibility and pathway yield.
*   Sequence Ranking + Design of Experiments (DoE)
*   Build / Approval / Export Workflow
*   Produces pathway briefs, annotated CSVs/JSONs, simulation flux tables, and an export manifest for downstream integration or lab validation.
*   List item

Technical Architecture

*   LangGraph orchestration: modular “nodes” for each stage (retrosynthesis, extract, thermo, rank, simulate, selenzyme, seqrank, doe, build, approve, export).
*   Micromamba-based environment isolation ensures reproducible dependency management per run.
*   Streaming execution & checkpointed state allow resuming or extending pipelines dynamically.
*   Output artifacts (CSV, JSON, Markdown) capture every stage for transparency and reproducibility.

In [ ]:
from typing import List, Dict, Optional, TypedDict

from typing_extensions import TypedDict, Annotated
from operator import add

class DBTLState(TypedDict, total=False):
    # constants (never “updated” again)
    run_id: Annotated[str, "const"]
    workdir: Annotated[str, "const"]
    target_smiles: Annotated[str, "const"]
    host: Annotated[str, "const"]

    # normal fields (LastValue by default)
    constraints: dict
    status: str
    last_node: str
    approved: bool
    signals: dict

    # append-only logs
    logs: Annotated[list[str], add]

    # your artifacts…
    retro_out_dir: str
    pathways_all_csv: str
    pathways_solved_csv: str
    pathways_all_json: str
    pathways_solved_json: str
    pathway_json_map: dict
    steps_plan_csv: str
    ranked_csv: str
    # etc…


In [ ]:
import io, os, stat, tarfile, urllib.request, subprocess, sys, pathlib, textwrap

WORKDIR = pathlib.Path("/content/runs/run_001")
BIN_DIR = WORKDIR / "bin"
MAMBA_ROOT = WORKDIR / "micromamba"
ENV_PREFIX = MAMBA_ROOT / "envs" / "retrobiocat"
MM_BIN = BIN_DIR / "micromamba"

BIN_DIR.mkdir(parents=True, exist_ok=True)
MAMBA_ROOT.mkdir(parents=True, exist_ok=True)
os.environ["MAMBA_ROOT_PREFIX"] = str(MAMBA_ROOT)

def sh(cmd, **kw):
    """Run a command, print it, and return CompletedProcess (raises on error by default)."""
    kw.setdefault("check", True)
    print("$", " ".join(cmd))
    return subprocess.run(cmd, **kw)

# 1) micromamba binary (download & extract once; auto-detect compression)
if not MM_BIN.exists():
    url = "https://micro.mamba.pm/api/micromamba/linux-64/latest"
    print("↓ downloading micromamba…", url)
    with urllib.request.urlopen(url) as r:
        data = r.read()
    print("→ extracting micromamba (auto-detect compression)…")
    with tarfile.open(fileobj=io.BytesIO(data), mode="r:*") as tf:
        member = None
        for m in tf.getmembers():
            name = m.name.replace("\\", "/")
            if name.endswith("/micromamba") and "/bin/" in name:
                member = m
                break
        if member is None:
            for m in tf.getmembers():
                if m.name.split("/")[-1] == "micromamba":
                    member = m
                    break
        if member is None:
            raise RuntimeError("micromamba binary not found in archive")
        with tf.extractfile(member) as src, open(MM_BIN, "wb") as dst:
            dst.write(src.read())
    MM_BIN.chmod(MM_BIN.stat().st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)
print("✅ micromamba:", MM_BIN)

# 2) create env (idempotent)
if not ENV_PREFIX.exists():
    sh([str(MM_BIN), "create", "-y", "-p", str(ENV_PREFIX), "-c", "conda-forge", "python=3.10", "pip"])

# 3) base pins (numpy/pandas/pint)
sh([str(MM_BIN), "run", "-p", str(ENV_PREFIX), "python", "-m", "pip", "install", "-q", "--upgrade",
    "pip", "setuptools", "wheel", "numpy==1.26.4", "pandas==2.2.2", "pint==0.22"])

# 4) equilibrator
sh([str(MM_BIN), "run", "-p", str(ENV_PREFIX), "python", "-m", "pip", "install", "-q",
    "equilibrator-api==0.6.0", "equilibrator-cache==0.6.0"])

# 5) cobra stack
sh([str(MM_BIN), "run", "-p", str(ENV_PREFIX), "python", "-m", "pip", "install", "-q",
    "cobra==0.29.0", "optlang>=1.8.0", "swiglpk>=5.0.10"])

# 6) RDKit via conda-forge
sh([str(MM_BIN), "install", "-y", "-p", str(ENV_PREFIX), "-c", "conda-forge", "rdkit=2022.09.5"])

# 7) RetroBioCat-2 (zip first, fallback to git)
try:
    sh([str(MM_BIN), "run", "-p", str(ENV_PREFIX), "python", "-m", "pip", "install", "-q",
        "https://github.com/willfinnigan/RetroBioCat-2/archive/refs/heads/main.zip"])
except subprocess.CalledProcessError:
    sh([str(MM_BIN), "run", "-p", str(ENV_PREFIX), "python", "-m", "pip", "install", "-q",
        "git+https://github.com/willfinnigan/RetroBioCat-2.git"])

# 8) light sanity (feed a tiny script via stdin)
mini = textwrap.dedent("""
import sys, importlib
print("python:", sys.executable)
mods = ("numpy","pandas","equilibrator_api","equilibrator_cache","cobra","rdkit")
for m in mods:
    try:
        mod = importlib.import_module(m)
        print(f"{m}: {getattr(mod,'__version__','?')}")
    except Exception as e:
        print(f"{m}: NOT FOUND ({type(e).__name__}: {e})")
""").encode()

print("\n→ versions (light)")
sh([str(MM_BIN), "run", "-p", str(ENV_PREFIX), "python", "-"], input=mini)
print("✔ bootstrap done.")



↓ downloading micromamba… https://micro.mamba.pm/api/micromamba/linux-64/latest
→ extracting micromamba (auto-detect compression)…
✅ micromamba: /content/runs/run_001/bin/micromamba
$ /content/runs/run_001/bin/micromamba create -y -p /content/runs/run_001/micromamba/envs/retrobiocat -c conda-forge python=3.10 pip
$ /content/runs/run_001/bin/micromamba run -p /content/runs/run_001/micromamba/envs/retrobiocat python -m pip install -q --upgrade pip setuptools wheel numpy==1.26.4 pandas==2.2.2 pint==0.22
$ /content/runs/run_001/bin/micromamba run -p /content/runs/run_001/micromamba/envs/retrobiocat python -m pip install -q equilibrator-api==0.6.0 equilibrator-cache==0.6.0
$ /content/runs/run_001/bin/micromamba run -p /content/runs/run_001/micromamba/envs/retrobiocat python -m pip install -q cobra==0.29.0 optlang>=1.8.0 swiglpk>=5.0.10
$ /content/runs/run_001/bin/micromamba install -y -p /content/runs/run_001/micromamba/envs/retrobiocat -c conda-forge rdkit=2022.09.5
$ /content/runs/run_001

In [ ]:
import subprocess, pathlib, os
ENV_PREFIX = pathlib.Path("/content/runs/run_001/micromamba/envs/retrobiocat")
MM_BIN = pathlib.Path("/content/runs/run_001/bin/micromamba")
print(subprocess.check_output([str(MM_BIN), "run", "-p", str(ENV_PREFIX), "python", "-c",
                               "from rdkit import Chem; print(bool(Chem.MolFromSmiles('CCO')))"]).decode())

True



In [ ]:
# =========================
# Micromamba bootstrap + nodes (clean, consolidated) -- 1
# =========================

import io, os, stat, tarfile, urllib.request, textwrap, subprocess, json, math, time
from pathlib import Path
import pandas as pd

# ------------------------
# Path helpers (per-state)
# ------------------------
def _paths(workdir: Path):
    workdir = Path(workdir)
    mm_root = workdir / "micromamba"
    mm_bin  = mm_root / "bin" / "micromamba"
    env_prefix = mm_root / "envs" / "retrobiocat"
    return {
        "WORKDIR": workdir,
        "MAMBA_ROOT": mm_root,
        "MAMBA_BIN": mm_bin,
        "ENV_PREFIX": env_prefix,
        "BIN_DIR": workdir / "bin",
        "RETRO_OUT": workdir / "retro_out",
        "FINISH": workdir / "retro_finish_out",
    }

# ---------------------------------
# Micromamba download (safe, clean)
# ---------------------------------


# ---------------------------------
# Run a command inside the env
# ---------------------------------
def _run_in_env(workdir: Path, argv, input_text=None, extra_env=None):
    p = _paths(workdir)
    mm = p["MAMBA_BIN"]
    envp = p["ENV_PREFIX"]
    cmd = [str(mm), "run", "-p", str(envp)] + list(argv)
    env = os.environ.copy()
    if extra_env:
        env.update(extra_env)
    proc = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE if input_text is not None else None,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        env=env,
    )
    out, err = proc.communicate(input=input_text)
    return proc.returncode, out, err

# ----------------------------------------------------------
# Node: thermo_node
#   - builds/repairs metabolite_map.csv using RDKit + matcher
#   - tries eQuilibrator for dG′°
# ----------------------------------------------------------
# ---- helper: make sure RDKit is available in the *notebook kernel* (outer) ----
def _ensure_rdkit_outer():
    try:
        import rdkit  # noqa: F401
        return
    except Exception:
        import subprocess, sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rdkit-pypi"])
        import rdkit  # noqa: F401


# ------------------------------- thermo_node --------------------------------
# ---------- helpers (safe to paste multiple times) ----------

def _run_in_env(env_prefix, argv, input_text=None, extra_env=None):
    """
    Run a command inside the micromamba env at env_prefix.
    Returns (rc, stdout, stderr).
    """
    import os, subprocess
    from pathlib import Path
    mm = Path(env_prefix).parent.parent / "bin" / "micromamba"
    cmd = [str(mm), "run", "-p", str(env_prefix)] + list(argv)
    env = os.environ.copy()
    if extra_env:
        env.update(extra_env)
    p = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE if input_text is not None else None,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        env=env,
    )
    out, err = p.communicate(input=input_text)
    return p.returncode, out, err


def _ensure_equilibrator(env_prefix, state=None):
    """Best-effort install of equilibrator stack with safe pins."""
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """
    Install RDKit quickly via wheels (rdkit-pypi).
    This gives us Chem.MolFromSmiles and MolToInchiKey for the compound map.
    """
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import rdkit; print('rdkit_ok')"])
    if rc == 0 and "rdkit_ok" in (out or ""):
        return
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","rdkit-pypi"])
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: RDKit install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


# ------------------------------- thermo_node --------------------------------

# ---------------- common helpers ----------------

from pathlib import Path
import subprocess, textwrap, os, io, tarfile, stat, time, math
import pandas as pd

WORKDIR      = Path("/content/runs/run_001")
MAMBA_ROOT   = WORKDIR / "micromamba"
ENV_PREFIX   = MAMBA_ROOT / "envs" / "retrobiocat"
MAMBA_BIN    = WORKDIR / "bin" / "micromamba"

def _sh(cmd, input_text=None, check=True, env=None):
    p = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE if input_text is not None else None,
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        text=True, env=env
    )
    out, err = p.communicate(input=input_text)
    if check and p.returncode != 0:
        raise subprocess.CalledProcessError(p.returncode, cmd, out, err)
    return p.returncode, out, err

def _ensure_micromamba():
    MAMBA_BIN.parent.mkdir(parents=True, exist_ok=True)
    if not MAMBA_BIN.exists():
        # download micromamba tar (auto-detected by tarfile "r:*")
        url = "https://micro.mamba.pm/api/micromamba/linux-64/latest"
        import urllib.request
        data = urllib.request.urlopen(url).read()
        with tarfile.open(fileobj=io.BytesIO(data), mode="r:*") as tf:
            member = None
            for m in tf.getmembers():
                name = m.name.replace("\\","/")
                if name.endswith("/micromamba") and "/bin/" in name:
                    member = m; break
            if member is None:
                for m in tf.getmembers():
                    if m.name.split("/")[-1] == "micromamba":
                        member = m; break
            if member is None:
                raise RuntimeError("micromamba binary not found in archive")
            with tf.extractfile(member) as src, open(MAMBA_BIN, "wb") as dst:
                dst.write(src.read())
        MAMBA_BIN.chmod(MAMBA_BIN.stat().st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)
    os.environ["MAMBA_ROOT_PREFIX"] = str(MAMBA_ROOT)

def _mm_run(argv, input_text=None):
    return _sh([str(MAMBA_BIN), "run", "-p", str(ENV_PREFIX)] + list(argv), input_text=input_text, check=False)

def _ensure_env_and_pins(state=None):
    _ensure_micromamba()
    # 1) create env if missing
    if not ENV_PREFIX.exists():
        _sh([str(MAMBA_BIN), "create", "-y", "-p", str(ENV_PREFIX), "-c", "conda-forge", "python=3.10", "pip"])
    # 2) pins (fast wheels)
    _mm_run(["python","-m","pip","install","-q","--upgrade",
             "pip","setuptools","wheel",
             "numpy==1.26.4","pandas==2.2.2","pint==0.22"])
    # 3) equilibrator stack
    _mm_run(["python","-m","pip","install","-q",
             "equilibrator-api==0.6.0","equilibrator-cache==0.6.0"])
    # 4) RDKit (pip build is fine here)
    _mm_run(["python","-m","pip","install","-q","rdkit-pypi"])
    # 5) cobra stack (modern)
    _mm_run(["python","-m","pip","install","-q","cobra==0.29.0","optlang>=1.8.0","swiglpk>=5.0.10"])
    # quick sanity (optional)
    rc, out, _ = _mm_run(["python","- <<'PY'"], input_text=textwrap.dedent("""
        import importlib, sys
        mods = ("equilibrator_api","equilibrator_cache","rdkit","cobra","pandas","numpy")
        for m in mods:
            try:
                mod=importlib.import_module(m)
                print(m, "OK", getattr(mod,"__version__","?"))
            except Exception as e:
                print(m, "MISS", type(e).__name__)
    PY
    """))
    if state is not None and out:
        for ln in out.splitlines():
            if ln.strip():
                state.setdefault("logs", []).append("env: " + ln.strip())

# ---------------- thermo_node ----------------
def _run_in_env(env_prefix, argv, input_text=None, extra_env=None):
    """Run argv inside the micromamba env at env_prefix. Return (rc, out, err)."""
    import os, subprocess
    from pathlib import Path
    mm = Path(env_prefix).parent.parent / "bin" / "micromamba"
    cmd = [str(mm), "run", "-p", str(env_prefix)] + list(argv)
    env = os.environ.copy()
    if extra_env:
        env.update(extra_env)
    p = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE if input_text is not None else None,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        env=env,
    )
    out, err = p.communicate(input=input_text)
    return p.returncode, out, err


def _ensure_eq_and_rdkit(env_prefix, state=None):
    """Idempotent install of equilibrator stack and RDKit wheels."""
    pins = [
        "pip", "setuptools", "wheel",
        "numpy==1.26.4", "pandas==2.2.2", "pint==0.22",
        "equilibrator-api==0.6.0", "equilibrator-cache==0.6.0",
        "rdkit-pypi==2022.9.5",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install had warnings (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])



# ----------------------------------------------------------
# Node: simulate_gem_node -new
#   - runs cobra FBA in env; writes CSV; tolerates missing cobra
# ----------------------------------------------------------

def simulate_gem_node(state):
    """
    Run a simple GEM simulation (FBA) inside the micromamba env.
    - Tries to load the SBML model and run an FBA.
    - Writes retro_finish_out/sim_fba_summary.csv  (always).
    - Gracefully degrades if cobra isn't available or model fails to load.

    Output CSV columns (best effort):
      pathway_tag, status, objective_value, note
    """
    from pathlib import Path
    import time
    import pandas as pd

    def log(msg):
        print(msg)
        state.setdefault("logs", []).append(msg)

    t0 = time.time()
    log("▶ start simulate_gem")

    workdir = Path(state["workdir"])
    finish  = workdir / "retro_finish_out"
    finish.mkdir(parents=True, exist_ok=True)

    sbml_path = Path(state.get("sbml_model_path", "/content/models/iML1515.xml"))
    steps_csv = finish / "steps_annotated.csv"
    if not steps_csv.exists():
        # fall back to enzyme-plan if thermo didn’t run
        alt = finish / "steps_enzyme_plan.csv"
        if alt.exists():
            steps_csv = alt

    # ensure the env (no workdir arg!)
    _ensure_env_and_pins(state=state)

    # small helper to choose a pathway (purely cosmetic; not required for FBA)
    pathway = "P001"
    try:
        dfp = pd.read_csv(steps_csv)
        if "pathway_tag" in dfp.columns and len(dfp):
            pathway = str(dfp["pathway_tag"].iloc[0])
    except Exception:
        pass

    # Write a stub result quickly in case runner fails; we’ll overwrite on success.
    summary_path = finish / "sim_fba_summary.csv"
    pd.DataFrame([{
        "pathway_tag": pathway,
        "status": "not_run",
        "objective_value": float("nan"),
        "note": "pre-run stub"
    }]).to_csv(summary_path, index=False)

    # COBRA runner (isolated)
    runner = r"""
import pandas as pd
from pathlib import Path

SBML   = Path("__SBML__")
OUT    = Path("__OUT__")
PATHWY = "__PATHWAY__"

def write(rows):
    pd.DataFrame(rows, columns=["pathway_tag","status","objective_value","note"]).to_csv(OUT, index=False)

try:
    import cobra
    from cobra.io import read_sbml_model
except Exception as e:
    write([{"pathway_tag": PATHWY, "status": "cobra_not_available",
            "objective_value": float("nan"), "note": f"{type(e).__name__}: {e}"}])
    raise SystemExit(0)

# Try to load the model
try:
    model = read_sbml_model(str(SBML))
except Exception as e:
    write([{"pathway_tag": PATHWY, "status": "model_load_failed",
            "objective_value": float("nan"), "note": f"{type(e).__name__}: {e}"}])
    raise SystemExit(0)

# Choose an objective
# Common biomass rxn for iML1515; fallback to existing objective or first reaction
obj = None
for cand in [
    "BIOMASS_Ec_iML1515_core_75p37M",
    "BIOMASS_Ec_iML1515_WT_75p37M",
    "BIOMASS_Ecoli_core_w_GAM"
]:
    if cand in model.reactions:
        obj = model.reactions.get_by_id(cand)
        break
if obj is None:
    try:
        obj = model.objective.expression.free_symbols.pop()._reaction  # best-effort
    except Exception:
        obj = next(iter(model.reactions)) if len(model.reactions) else None

if obj is None:
    write([{"pathway_tag": PATHWY, "status": "no_objective",
            "objective_value": float("nan"), "note": "No reactions/objective in model"}])
    raise SystemExit(0)

# Set objective and optimize
try:
    model.objective = obj
    sol = model.optimize()  # default maximize
    val = float(sol.objective_value) if sol and sol.status in ("optimal","time_limit") else float("nan")
    status = getattr(sol, "status", "unknown") or "unknown"
    note = f"objective={obj.id}"
    write([{"pathway_tag": PATHWY, "status": status, "objective_value": val, "note": note}])
except Exception as e:
    write([{"pathway_tag": PATHWY, "status": "opt_failed",
            "objective_value": float("nan"), "note": f"{type(e).__name__}: {e}"}])
"""

    runner = runner.replace("__SBML__", str(sbml_path).replace("\\","\\\\"))
    runner = runner.replace("__OUT__", str(summary_path).replace("\\","\\\\"))
    runner = runner.replace("__PATHWAY__", pathway)

    rc, out, err = _mm_run(["python","-"], input_text=runner)

    if rc != 0 and err:
        # keep the stub file but add an informative line
        state.setdefault("logs", []).append("simulate_gem: runner failed → stub preserved")
        state["logs"].append((err or "").strip().splitlines()[-1])

    # log a compact result summary
    try:
        dfres = pd.read_csv(summary_path)
        if not dfres.empty:
            msg = f"simulate_gem: results={summary_path}"
            # if note is long, don’t flood the logs
            state.setdefault("logs", []).append(msg)
            print(msg)
    except Exception:
        pass

    print(f"✔ done simulate_gem ({time.time()-t0:.1f}s)")

    # publish artifact
    state.setdefault("artifacts", []).append(str(summary_path))
    return state



In [ ]:
# ---------------------- helpers used by thermo_node ----------------------2

def _run_in_env(env_prefix, argv, input_text=None, extra_env=None):
    """
    Run a command inside the micromamba env at env_prefix.
    Returns (rc, stdout, stderr).
    """
    import os, subprocess
    from pathlib import Path
    mm = Path(env_prefix).parent.parent / "bin" / "micromamba"
    cmd = [str(mm), "run", "-p", str(env_prefix)] + list(argv)
    env = os.environ.copy()
    if extra_env:
        env.update(extra_env)
    p = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE if input_text is not None else None,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        env=env,
    )
    out, err = p.communicate(input=input_text)
    return p.returncode, out, err


def _ensure_equilibrator(env_prefix, state=None):
    """Best-effort, idempotent install of equilibrator stack with safe pins."""
    pins = [
        "pip", "setuptools", "wheel",
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """
    Install RDKit into the same micromamba env (conda-forge build).
    """
    # Quick ping
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import rdkit; print('rdkit_ok')"])
    if rc == 0 and "rdkit_ok" in (out or ""):
        return
    # Install from conda-forge (works well in micromamba env)
    rc, out, err = _run_in_env(
        env_prefix,
        ["micromamba","install","-y","-p",str(env_prefix),"-c","conda-forge","rdkit=2022.09.5"]
    )
    if rc != 0:
        # Fallback to rdkit-pypi if conda install is unavailable
        rc2, out2, err2 = _run_in_env(
            env_prefix,
            ["python","-m","pip","install","-q","rdkit-pypi"]
        )
        if state is not None and rc2 != 0:
            state.setdefault("logs", []).append("thermo: RDKit install warning (continuing)")
            if err2:
                state["logs"].append(err2.strip().splitlines()[-1])

def _canon_smiles(smi):
    """Return a canonical isomeric SMILES for stable PubChem lookups."""
    try:
        from rdkit import Chem
        m = Chem.MolFromSmiles(smi)
        if m is None:
            return ""
        return Chem.MolToSmiles(m, isomericSmiles=True, canonical=True)
    except Exception:
        return ""

def _pubchem_cid_from_smiles(smi, session=None):
    """SMILES -> CID via PUG REST (tolerant)."""
    smi = _canon_smiles(smi) or smi
    if not smi:
        return None
    import urllib.parse
    enc = urllib.parse.quote(smi, safe="")
    js = _pc_json(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{enc}/cids/JSON",
                  session=session)
    try:
        cids = js.get("IdentifierList", {}).get("CID", []) if js else []
        if cids:
            return int(cids[0])
    except Exception:
        pass
    return None


def thermo_node(state):
    """
    Compute thermodynamics and write:
      - retro_finish_out/steps_annotated.csv
      - retro_finish_out/pathways_ranked_final.csv
      - retro_finish_out/metabolite_map.csv
      - retro_finish_out/_thermo_metabolite_map.json

    Mapping strategy per token (first success wins):
      1) explicit IDs in token (KEGG Cxxxxx or CHEBI:nnn) via CompoundMatcher
      2) literal InChIKey inside token via CompoundMatcher
      3) RDKit -> InChIKey, then CompoundMatcher
      4) RDKit -> InChIKey only (record IK even if no CID)
      5) PubChem: IK -> CID -> KEGG
      6) PubChem: SMILES -> CID -> KEGG     <-- NEW
      7) PubChem: CAS -> CID -> KEGG        <-- NEW
      8) name match via CompoundMatcher
      9) unmapped

    Also:
      - Prefer tokens from `substrates`/`products` columns; fallback to reaction_smiles.
      - Drop tiny/query-like fragments (<=3 heavy atoms) before attempting resolution.
    """
    import time, math, pandas as pd
    from pathlib import Path

    def _log(msg):
        print(msg)
        state.setdefault("logs", []).append(msg)

    t0 = time.time()
    _log("▶ start thermo")

    workdir   = Path(state["workdir"])
    finish    = workdir / "retro_finish_out"
    retro_out = workdir / "retro_out"
    finish.mkdir(parents=True, exist_ok=True)

    env_prefix = workdir / "micromamba" / "envs" / "retrobiocat"

    # Ensure deps in env (idempotent)
    _run_in_env(env_prefix, [
        "python","-m","pip","install","-q","--upgrade",
        "pip","setuptools","wheel",
        "equilibrator-api==0.6.0","equilibrator-cache==0.6.0",
        "numpy==1.26.4","pandas==2.2.2","pint==0.22",
        "requests>=2.31.0"
    ])
    # RDKit via conda if missing
    _run_in_env(env_prefix, [
        "bash","-lc",
        f'set -e; python -c "import rdkit" 2>/dev/null || micromamba install -y -p "{env_prefix}" -c conda-forge rdkit=2022.09.5 >/dev/null'
    ])

    # choose steps table (prefer extractor output that already has selenzyme_url)
    steps_csv = finish / "steps_enzyme_plan.csv"
    if not steps_csv.exists():
        steps_csv = retro_out / "steps.csv"

    if not steps_csv.exists():
        _log("thermo: no steps table found; creating empty placeholders")
        (finish / "steps_annotated.csv").write_text("")
        (finish / "pathways_ranked_final.csv").write_text("")
        _log(f"✔ done thermo ({time.time()-t0:.1f}s)")
        return state

    # Embedded runner executed inside the env
    runner = r"""
import os, re, json, math, time
import pandas as pd
from pathlib import Path

STEPS_CSV = Path("__STEPS__")
FINISH    = Path("__FINISH__")

OUT_STEPS    = FINISH / "steps_annotated.csv"
OUT_RANK     = FINISH / "pathways_ranked_final.csv"
OUT_MAP_CSV  = FINISH / "metabolite_map.csv"
OUT_MAP_JSON = FINISH / "_thermo_metabolite_map.json"

# Load steps
df = pd.read_csv(STEPS_CSV)
for col in ["pathway_tag","step_idx","reaction_smiles","substrates","products",
            "rbc2_score","precedent_best_similarity","selenzyme_url"]:
    if col not in df.columns:
        if col in ("rbc2_score","precedent_best_similarity"):
            df[col] = 0.0
        else:
            df[col] = ""

# --- Tokenization helpers ---
def _split_rxn(r):
    if not isinstance(r, str) or ">>" not in r:
        return [], []
    L, R = r.split(">>", 1)
    subs  = [s.strip() for s in L.split(".") if s.strip()]
    prods = [p.strip() for p in R.split(".") if p.strip()]
    return subs, prods

def _split_field(val):
    # supports dot '.' or semicolon ';' separation
    if not isinstance(val, str) or not val.strip():
        return []
    parts = []
    for sep in (".",";"):
        if sep in val:
            parts = [p.strip() for p in val.split(sep)]
            break
    if not parts:
        parts = [val.strip()]
    return [p for p in parts if p]

# Prefer substrates/products columns if present and non-empty;
# fallback to reaction_smiles tokenization.
raw_tokens = []
if (df["substrates"].astype(str).str.strip().ne("").any() or
    df["products"].astype(str).str.strip().ne("").any()):
    for a,b in zip(df.get("substrates","").astype(str), df.get("products","").astype(str)):
        raw_tokens.extend(_split_field(a))
        raw_tokens.extend(_split_field(b))
else:
    for rxn in df.get("reaction_smiles", pd.Series([""]*len(df))).astype(str).fillna(""):
        L, R = _split_rxn(rxn)
        raw_tokens.extend(L); raw_tokens.extend(R)

# --- RDKit helpers ---
def _smiles_to_inchikey(smi):
    try:
        from rdkit import Chem
        m = Chem.MolFromSmiles(smi)
        if m is None:
            return ""
        return Chem.MolToInchiKey(m) or ""
    except Exception:
        return ""

def _heavy_atoms(smi):
    try:
        from rdkit import Chem
        m = Chem.MolFromSmiles(smi)
        return int(m.GetNumHeavyAtoms()) if m is not None else 0
    except Exception:
        return 0

# Filter out tiny/query-like fragments (≤3 heavy atoms)
tokens = sorted({t for t in raw_tokens if _heavy_atoms(t) > 3})

# Try to import CompoundMatcher (optional)
matcher = None
try:
    try:
        from equilibrator_cache.match import CompoundMatcher
    except Exception:
        from equilibrator_cache.matching import CompoundMatcher
    try:
        matcher = CompoundMatcher()
    except Exception:
        matcher = None
except Exception:
    matcher = None

# --- PubChem helpers (tolerant) ---
def _pc_get_json(url, timeout=6.0, session=None):
    try:
        import requests
        s = session or requests.Session()
        s.headers.update({"User-Agent": "dbtl-thermo/0.3"})
        r = s.get(url, timeout=timeout)
        if r.status_code == 200:
            return r.json()
    except Exception:
        return None
    return None

def _pubchem_kegg_from_cid(cid, timeout=6.0, session=None):
    js = _pc_get_json(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/xrefs/KEGG/JSON", timeout, session)
    if js:
        info = js.get("InformationList", {}).get("Information", [])
        if info:
            klist = info[0].get("KEGG") or []
            for k in klist:
                if isinstance(k, str) and re.fullmatch(r"C\\d{5}", k):
                    return k
    js = _pc_get_json(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/synonyms/JSON", timeout, session)
    if js:
        info = js.get("InformationList", {}).get("Information", [])
        if info:
            syns = info[0].get("Synonym") or []
            for s in syns:
                if isinstance(s, str) and re.fullmatch(r"C\\d{5}", s):
                    return s
    return ""

def _pubchem_kegg_from_inchikey(ik, timeout=6.0, session=None):
    js = _pc_get_json(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{ik}/cids/JSON", timeout, session)
    cid = None
    try:
        cid_list = js.get("IdentifierList", {}).get("CID", []) if js else []
        if cid_list:
            cid = int(cid_list[0])
    except Exception:
        cid = None
    if cid is None:
        js = _pc_get_json(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{ik}/cids/JSON?match=exact", timeout, session)
        try:
            cid_list = js.get("IdentifierList", {}).get("CID", []) if js else []
            if cid_list:
                cid = int(cid_list[0])
        except Exception:
            cid = None
    if cid is None:
        return ""
    return _pubchem_kegg_from_cid(cid, timeout, session)

def _pubchem_kegg_from_smiles(smi, timeout=6.0, session=None):
    # SMILES → CID → KEGG  (identity search)
    js = _pc_get_json(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{smi}/cids/JSON?search=identity",
                      timeout, session)
    cid = None
    try:
        cid_list = js.get("IdentifierList", {}).get("CID", []) if js else []
        if cid_list:
            cid = int(cid_list[0])
    except Exception:
        cid = None
    if cid is None:
        return ""
    return _pubchem_kegg_from_cid(cid, timeout, session)

def _pubchem_kegg_from_cas(cas_rn, timeout=6.0, session=None):
    js = _pc_get_json(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/xref/RN/{cas_rn}/cids/JSON", timeout, session)
    cid = None
    try:
        cid_list = js.get("IdentifierList", {}).get("CID", []) if js else []
        if cid_list:
            cid = int(cid_list[0])
    except Exception:
        cid = None
    if cid is None:
        return ""
    return _pubchem_kegg_from_cid(cid, timeout, session)

_cas_re = re.compile(r"\\b\\d{2,7}-\\d{2}-\\d\\b")

route_counts = {
    "via_explicit_id": 0,
    "via_inchikey_literal": 0,
    "via_rdkit_inchikey": 0,
    "via_rdkit_inchikey_only": 0,
    "via_pubchem_kegg": 0,
    "via_pubchem_smiles": 0,
    "via_pubchem_cas": 0,
    "via_match_smiles": 0,
    "via_name": 0,
    "unmapped": 0,
}

def _best_match(tok):
    # Returns: dict(token, cid, name, inchikey)
    # 1) explicit KEGG/CHEBI in token
    try:
        if matcher is not None:
            m = re.search(r"(CHEBI:\\d+|C\\d{5})", tok, flags=re.I)
            if m:
                key = m.group(1)
                try:
                    if key.upper().startswith("CHEBI:"):
                        res = matcher.match_chebi_id(key.upper())
                    else:
                        res = matcher.match_kegg_id(key.upper())
                    if res:
                        route_counts["via_explicit_id"] += 1
                        return {"token": tok,
                                "cid": getattr(res, "cid", "") or "",
                                "name": getattr(res, "name", "") or "",
                                "inchikey": getattr(res, "inchi_key", "") or ""}
                except Exception:
                    pass
    except Exception:
        pass

    # 2) literal IK in token
    try:
        if matcher is not None:
            m2 = re.search(r"[A-Z]{14}-[A-Z]{10}-[A-Z]", tok)
            if m2:
                res = matcher.match_inchi_key(m2.group(0))
                if res:
                    route_counts["via_inchikey_literal"] += 1
                    return {"token": tok,
                            "cid": getattr(res, "cid", "") or "",
                            "name": getattr(res, "name", "") or "",
                            "inchikey": getattr(res, "inchi_key", "") or ""}
    except Exception:
        pass

    # 3) RDKit IK then matcher
    ik = ""
    try:
        ik = _smiles_to_inchikey(tok)
        if ik and matcher is not None:
            res = matcher.match_inchi_key(ik)
            if res:
                route_counts["via_rdkit_inchikey"] += 1
                return {"token": tok,
                        "cid": getattr(res, "cid", "") or "",
                        "name": getattr(res, "name", "") or "",
                        "inchikey": getattr(res, "inchi_key", "") or ik}
    except Exception:
        pass

    # 4/5) RDKit IK only + PubChem IK->KEGG
    if ik:
        route_counts["via_rdkit_inchikey_only"] += 1
        kegg = _pubchem_kegg_from_inchikey(ik)
        if kegg:
            route_counts["via_pubchem_kegg"] += 1
            return {"token": tok, "cid": kegg, "name": "", "inchikey": ik}

    # 6) PubChem SMILES → KEGG (identity)
    try:
        kegg = _pubchem_kegg_from_smiles(tok)
        if kegg:
            route_counts["via_pubchem_smiles"] += 1
            return {"token": tok, "cid": kegg, "name": "", "inchikey": ik or _smiles_to_inchikey(tok)}
    except Exception:
        pass

    # 7) CAS in token → KEGG
    try:
        m3 = _cas_re.search(tok)
        if m3:
            cas = m3.group(0)
            kegg = _pubchem_kegg_from_cas(cas)
            if kegg:
                route_counts["via_pubchem_cas"] += 1
                return {"token": tok, "cid": kegg, "name": "", "inchikey": ik or ""}
    except Exception:
        pass

    # 8) name match (very last resort)
    try:
        if matcher is not None:
            res = matcher.match_name(tok)
            if res:
                route_counts["via_name"] += 1
                return {"token": tok,
                        "cid": getattr(res, "cid", "") or "",
                        "name": getattr(res, "name", "") or "",
                        "inchikey": getattr(res, "inchi_key", "") or ""}
    except Exception:
        pass

    route_counts["unmapped"] += 1
    return {"token": tok, "cid": "", "name": "", "inchikey": ik or ""}

map_rows = [_best_match(t) for t in tokens]

if map_rows:
    map_df = pd.DataFrame(map_rows, columns=["token","inchikey","cid","name"])
    map_df.to_csv(OUT_MAP_CSV, index=False)
    with open(OUT_MAP_JSON, "w", encoding="utf-8") as f:
        json.dump({"generated_from": str(STEPS_CSV),
                   "routes": route_counts,
                   "rows": map_rows}, f, indent=2)

# Equilibrator
have_eq = False
try:
    from equilibrator_api import ComponentContribution, Reaction
    cc = ComponentContribution()
    have_eq = True
except Exception:
    cc = None

cid_by_token = {r["token"]: r.get("cid","") for r in map_rows}

def _stoich_from_rxn(rxn):
    if not isinstance(rxn, str) or ">>" not in rxn:
        return None
    L, R = _split_rxn(rxn)
    st = {}
    for s in L:
        cid = cid_by_token.get(s, "")
        if not cid: return None
        st[cid] = st.get(cid, 0) - 1
    for p in R:
        cid = cid_by_token.get(p, "")
        if not cid: return None
        st[cid] = st.get(cid, 0) + 1
    return st

pH, I, pMg, tempK, margin = 7.5, 0.25, 3.0, 298.15, 0.0
dGs, dGsig, passf, form = [], [], [], []
matched = 0

for rxn in df.get("reaction_smiles", pd.Series([""]*len(df))).astype(str).fillna(""):
    st = _stoich_from_rxn(rxn)
    if have_eq and st:
        try:
            r = Reaction(st)
            dG0, dG0_unc = cc.standard_dg_prime(r, pH=pH, ionic_strength=I, pMg=pMg, temperature=tempK)
            dGs.append(float(dG0)); dGsig.append(float(dG0_unc))
            passf.append(bool((float(dG0) + margin) <= 0.0))
            form.append("eq")
            matched += 1
        except Exception:
            dGs.append(math.nan); dGsig.append(math.nan); passf.append(False); form.append("")
    else:
        dGs.append(math.nan); dGsig.append(math.nan); passf.append(False); form.append("")

df["dGprime_kJ_per_mol"]   = dGs
df["uncert_kJ_per_mol"]    = dGsig
df["thermo_pass"]          = passf
df["equilibrator_formula"] = form
df.to_csv(OUT_STEPS, index=False)

# Rank (tolerant)
tmp = df.copy()
tmp["rbc2_score"] = pd.to_numeric(tmp.get("rbc2_score", 0.0), errors="coerce").fillna(0.0)
tmp["precedent_best_similarity"] = pd.to_numeric(tmp.get("precedent_best_similarity", 0.0), errors="coerce").fillna(0.0)
tmp["thermo_pass"] = tmp.get("thermo_pass", False)

agg = tmp.groupby("pathway_tag").agg(
    steps_count=("step_idx","max"),
    sum_rbc2=("rbc2_score","sum"),
    sum_prec_sim=("precedent_best_similarity","sum"),
    n_thermo_pass=("thermo_pass","sum"),
).reset_index()

agg["rank_score"] = (
    agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] + 0.5*agg["n_thermo_pass"] - 0.1*agg["steps_count"]
)

ranked = agg.sort_values(
    ["rank_score","sum_rbc2","sum_prec_sim","n_thermo_pass","steps_count"],
    ascending=[False, False, False, False, True]
).reset_index(drop=True)
ranked.to_csv(OUT_RANK, index=False)

# Logging
cid_avail = sum(1 for r in map_rows if (r.get("cid","") or "").startswith("C"))
ik_avail  = sum(1 for r in map_rows if r.get("inchikey",""))
print(f"thermo: token IDs available for {cid_avail} / {len(tokens)} tokens")
print(f"thermo: id routes: {route_counts}")

# Show examples if IK only and no KEGG
if route_counts.get("via_rdkit_inchikey_only",0) > 0 and (
    route_counts.get("via_pubchem_kegg",0) + route_counts.get("via_pubchem_smiles",0) + route_counts.get("via_pubchem_cas",0)
) == 0:
    ex = [r["token"] for r in map_rows if r.get("inchikey","") and not (r.get("cid","") or "").startswith("C")][:5]
    if ex:
        print("thermo: NOTE – RDKit produced InChIKeys but none resolved to KEGG via PubChem.")
        print(f"thermo: example tokens with IK but no KEGG (first 5): {ex}")

print("thermo: equilibrator ran in env" if have_eq else "thermo: equilibrator not available")
print("thermo: steps evaluable %d / %d" % (matched, len(df)))
""".lstrip("\n")

    # inject paths
    runner = runner.replace("__STEPS__", str(steps_csv).replace("\\", "\\\\"))
    runner = runner.replace("__FINISH__", str(finish).replace("\\", "\\\\"))

    rc, out, err = _run_in_env(env_prefix, ["python","-"], input_text=runner)

    if rc != 0:
        _log("thermo: runner failed, writing tolerant placeholders")
        try:
            df = pd.read_csv(steps_csv)
        except Exception:
            df = pd.DataFrame(columns=[
                "pathway_tag","step_idx","reaction_smiles","substrates","products",
                "rbc2_score","precedent_best_similarity"
            ])
        for col in ["dGprime_kJ_per_mol","uncert_kJ_per_mol","thermo_pass","equilibrator_formula"]:
            if col not in df.columns:
                df[col] = (False if col=="thermo_pass" else ("" if col=="equilibrator_formula" else math.nan))
        df.to_csv(finish / "steps_annotated.csv", index=False)

        tmp = df.copy()
        tmp["rbc2_score"] = pd.to_numeric(tmp.get("rbc2_score",0.0), errors="coerce").fillna(0.0)
        tmp["precedent_best_similarity"] = pd.to_numeric(tmp.get("precedent_best_similarity",0.0), errors="coerce").fillna(0.0)
        tmp["thermo_pass"] = tmp.get("thermo_pass", False)
        agg = tmp.groupby("pathway_tag").agg(
            steps_count=("step_idx","max"),
            sum_rbc2=("rbc2_score","sum"),
            sum_prec_sim=("precedent_best_similarity","sum"),
            n_thermo_pass=("thermo_pass","sum"),
        ).reset_index()
        agg["rank_score"] = agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] + 0.5*agg["n_thermo_pass"] - 0.1*agg["steps_count"]
        agg.sort_values(
            ["rank_score","sum_rbc2","sum_prec_sim","n_thermo_pass","steps_count"],
            ascending=[False,False,False,False,True]
        ).reset_index(drop=True).to_csv(finish / "pathways_ranked_final.csv", index=False)

        if err:
            _log(err.strip().splitlines()[-1])
        state["thermo_method"] = "matchonly"
    else:
        for line in (out or "").splitlines():
            if line.strip():
                _log(line.strip())
        state["thermo_method"] = "equilibrator"

    # advertise artifacts
    state.setdefault("artifacts", []).extend([
        str(finish / "steps_annotated.csv"),
        str(finish / "pathways_ranked_final.csv"),
        str(finish / "metabolite_map.csv"),
        str(finish / "_thermo_metabolite_map.json"),
    ])

    _log(f"✔ done thermo ({time.time()-t0:.1f}s)")
    return state



def smarts_smirks_node(state):
    """
    Annotate tokens with SMARTS hits and check SMIRKS applicability.
    Outputs:
      - retro_finish_out/smarts_hits.csv
      - retro_finish_out/smirks_applicability.csv
    """
    import time, pandas as pd
    from pathlib import Path

    def _log(msg):
        print(msg)
        state.setdefault("logs", []).append(msg)

    t0 = time.time()
    _log("▶ start smarts_smirks")

    workdir   = Path(state["workdir"])
    finish    = workdir / "retro_finish_out"
    retro_out = workdir / "retro_out"
    finish.mkdir(parents=True, exist_ok=True)

    env_prefix = workdir / "micromamba" / "envs" / "retrobiocat"

    # Ensure RDKit
    _run_in_env(env_prefix, [
        "bash","-lc",
        f'set -e; python -c "import rdkit" 2>/dev/null || micromamba install -y -p "{env_prefix}" -c conda-forge rdkit=2022.09.5 >/dev/null'
    ])

    # Prefer steps_annotated (after thermo), else steps_enzyme_plan, else steps.csv
    steps_csv = finish / "steps_annotated.csv"
    if not steps_csv.exists():
        steps_csv = finish / "steps_enzyme_plan.csv"
    if not steps_csv.exists():
        steps_csv = retro_out / "steps.csv"

    # supply default libs if absent
    smarts_lib = state.get("smarts_library") or {
        "carbonyl": "[CX3]=[OX1]",
        "primary_alcohol": "[CX4;H2][OX2H]",
        "secondary_alcohol": "[CX4;H1][OX2H]",
        "aromatic_phenol": "c[OH]",
        "nitrile": "[CX2]#N",
    }
    smirks_lib = state.get("smirks_library") or {
        # toy examples
        "reduce_carbonyl_to_alcohol": "[CX3:1]=[OX1:2]>>[CX3H1:1][OX2H1:2]",
        "hydrolyze_nitrile": "[CX2:1]#N>>[CX3:1](=O)N",
    }

    # Runner that does RDKit work inside env
    runner = r"""
import pandas as pd
from pathlib import Path

STEPS_CSV = Path("__STEPS__")
FINISH    = Path("__FINISH__")

OUT_SMARTS  = FINISH / "smarts_hits.csv"
OUT_SMIRKS  = FINISH / "smirks_applicability.csv"

SMARTS_LIB = __SMARTS_JSON__
SMIRKS_LIB = __SMIRKS_JSON__

from rdkit import Chem
from rdkit.Chem import rdChemReactions as Reactions

# Load steps
df = pd.read_csv(STEPS_CSV)

def _split_rxn(r):
    if not isinstance(r, str) or ">>" not in r:
        return [], []
    L, R = r.split(">>", 1)
    subs  = [s.strip() for s in L.split(".") if s.strip()]
    prods = [p.strip() for p in R.split(".") if p.strip()]
    return subs, prods

# Collect tokens (substrates + products) with step indices
rows = []
for i, rxn in enumerate(df.get("reaction_smiles", pd.Series([""]*len(df))).astype(str).fillna("")):
    subs, prods = _split_rxn(rxn)
    for smi in subs + prods:
        rows.append({"step_idx": int(df.get("step_idx", pd.Series(range(1,len(df)+1))).iloc[i] if "step_idx" in df.columns else i+1),
                     "token": smi})
tokdf = pd.DataFrame(rows).drop_duplicates()

# SMARTS matching (long format)
smart_rows = []
compiled = {}
for name, patt in SMARTS_LIB.items():
    try:
        compiled[name] = Chem.MolFromSmarts(patt)
    except Exception:
        compiled[name] = None

for _, r in tokdf.iterrows():
    smi = r["token"]
    m = Chem.MolFromSmiles(smi)
    if m is None:
        continue
    for name, q in compiled.items():
        hit = False
        if q is not None:
            try:
                hit = (m.HasSubstructMatch(q))
            except Exception:
                hit = False
        smart_rows.append({"token": smi, "smarts_name": name, "hit": bool(hit)})

pd.DataFrame(smart_rows).to_csv(OUT_SMARTS, index=False)

# SMIRKS applicability (per step, per rule; count applicable substrates)
app_rows = []
compiled_rxn = {}
for name, smks in SMIRKS_LIB.items():
    try:
        rxn = Reactions.ReactionFromSmarts(smks, useSmiles=True)
        compiled_rxn[name] = rxn
    except Exception:
        compiled_rxn[name] = None

for i, rxn in enumerate(df.get("reaction_smiles", pd.Series([""]*len(df))).astype(str).fillna("")):
    step_i = int(df.get("step_idx", pd.Series(range(1,len(df)+1))).iloc[i] if "step_idx" in df.columns else i+1)
    subs, prods = _split_rxn(rxn)
    subs_mols = []
    for smi in subs:
        m = Chem.MolFromSmiles(smi)
        if m is not None:
            subs_mols.append(m)
    for name, rxnobj in compiled_rxn.items():
        applicable = 0
        if rxnobj is not None and subs_mols:
            try:
                # try each substrate alone as a reactant tuple of length 1
                # (multi-substrate SMIRKS will need a library-specific adapter)
                for m in subs_mols:
                    out = rxnobj.RunReactants((m,))
                    if out and len(out) > 0:
                        applicable += 1
            except Exception:
                applicable = 0
        app_rows.append({"step_idx": step_i, "smirks_name": name, "n_applicable_substrates": int(applicable)})

pd.DataFrame(app_rows).to_csv(OUT_SMIRKS, index=False)

print("smarts_smirks: tokens", len(tokdf))
print("smarts_smirks: smarts rules", len(SMARTS_LIB))
print("smarts_smirks: smirks rules", len(SMIRKS_LIB))
""".lstrip("\n")

    import json
    runner = runner.replace("__STEPS__", str(steps_csv).replace("\\", "\\\\"))
    runner = runner.replace("__FINISH__", str(finish).replace("\\", "\\\\"))
    runner = runner.replace("__SMARTS_JSON__", json.dumps(smarts_lib))
    runner = runner.replace("__SMIRKS_JSON__", json.dumps(smirks_lib))

    rc, out, err = _run_in_env(env_prefix, ["python","-"], input_text=runner)
    if rc != 0:
        _log("smarts_smirks: runner failed")
        if err:
            _log(err.strip().splitlines()[-1])
    else:
        for line in (out or "").splitlines():
            if line.strip():
                _log(line.strip())

    state.setdefault("artifacts", []).extend([
        str(finish / "smarts_hits.csv"),
        str(finish / "smirks_applicability.csv"),
    ])
    _log(f"✔ done smarts_smirks ({time.time()-t0:.1f}s)")
    return state



In [ ]:
! /content/runs/run_001/bin/micromamba run -p /content/runs/run_001/micromamba/envs/retrobiocat \
  python -m equilibrator_cache.cli download --to /content/runs/run_001/equilibrator_cache


/content/runs/run_001/micromamba/envs/retrobiocat/bin/python: No module named equilibrator_cache.cli


In [ ]:
#sequuence matching
# --- sequence matching ---
def sequence_fetch_node(state, per_step=10, reviewed_first=False, timeout=12.0):
    """
    Fetches sequence candidates from UniProt based on Selenzyme links, EC numbers,
    and keywords. Writes retro_finish_out/steps_sequence_candidates.csv and
    sequence_fetch_debug.json.
    """
    # --- GUARDS ---
    if state is None:
        state = {}

    import json, re, requests, pandas as pd, time
    from pathlib import Path

    def _log(msg):
        print(msg)
        state.setdefault("logs", []).append(msg)

    t0 = time.time()
    _log("▶ start sequence_fetch")

    workdir = Path(state.get("workdir") or state.get("run_dir") or ".")
    state["workdir"] = str(workdir)  # ensure for downstream
    finish = workdir / "retro_finish_out"
    finish.mkdir(parents=True, exist_ok=True)

    save_debug = True

    # ----- load steps (choice > plan) -----
    steps_csv = finish / "steps_enzyme_choice.csv"
    if not steps_csv.exists():
        steps_csv = finish / "steps_enzyme_plan.csv"

    if not steps_csv.exists():
        _log("sequence_fetch: no steps file found; wrote empty candidates")
        (finish / "steps_sequence_candidates.csv").write_text("")
        if save_debug:
            (finish / "sequence_fetch_debug.json").write_text("[]")
        _log(f"✔ done sequence_fetch ({time.time()-t0:.1f}s)")
        return state

    try:
        df = pd.read_csv(steps_csv).fillna("")
    except Exception as e:
        _log(f"sequence_fetch: ERROR loading steps CSV: {type(e).__name__}: {e}")
        (finish / "steps_sequence_candidates.csv").write_text("")
        if save_debug:
            (finish / "sequence_fetch_debug.json").write_text("[]")
        _log(f"✔ done sequence_fetch ({time.time()-t0:.1f}s)")
        return state

    # Normalize columns we rely on
    for col in ["pathway_tag","step_idx","rxn_type","name","selenzyme_url","seed_uniprots","reaction_smiles"]:
        if col not in df.columns:
            df[col] = ""
    df["step_idx"] = pd.to_numeric(df["step_idx"], errors="coerce").fillna(0).astype(int)

    # ----- UniProt helpers -----
    U = requests.Session()
    U.headers.update({"User-Agent": "dbtl-seq/0.4"})
    uniprot_base = "https://rest.uniprot.org/uniprotkb/search"
    fasta_base   = "https://rest.uniprot.org/uniprotkb/"  # {ACC}.fasta

    def _uniprot_search(query, size):
        fields = ",".join([
            "accession","id","organism_name","reviewed","length","protein_name",
            "genes","ec","cc_function","sequence"
        ])
        params = dict(query=query, fields=fields, format="json", size=int(max(1,size)))
        meta = {"status": None, "url": None, "query": query}
        try:
            r = U.get(uniprot_base, params=params, timeout=timeout)
            meta["status"] = r.status_code; meta["url"] = str(r.url)
            if r.status_code != 200:
                return [], meta
            js = r.json()
            return (js.get("results", []) or []), meta
        except Exception as e:
            meta["status"] = f"ERR:{type(e).__name__}"
            return [], meta

    def _fetch_fasta(acc):
        try:
            r = U.get(f"{fasta_base}{acc}.fasta", timeout=timeout)
            if r.status_code != 200 or not r.text or not r.text.startswith(">"):
                return ""
            lines = [ln.strip() for ln in r.text.splitlines() if ln.strip()]
            return "".join(ln for ln in lines if not ln.startswith(">"))
        except Exception:
            return ""

    # Selenzyme page scraper for UniProt IDs
    def _scrape_uniprot_from_selenzyme(url, session=None, timeout=8.0):
        if not url or not isinstance(url, str):
            return set()
        s = session or requests.Session()
        s.headers.update({"User-Agent": "dbtl-scraper/0.1"})
        try:
            r = s.get(url, timeout=timeout)
            if r.status_code != 200:
                return set()
            html = r.text
        except Exception:
            return set()
        hits = set()
        # plain accessions
        for m in re.finditer(r"\b([OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]{5})\b", html):
            hits.add(m.group(1))
        # uniprot links
        for m in re.finditer(r'uniprot(?:kb)?/(?:entry/)?([A-Z0-9]{6,10})', html, re.I):
            hits.add(m.group(1))
        return hits

    def _ec_from_row(row):
        # full or dashed EC patterns
        m = re.search(r"(?:EC\s*)?(\d+\.\d+\.\d+\.\d+|\d+\.\d+\.\d+\.-|\d+\.\d+\.-\.-|\d+\.-\.-\.-)",
                      str(row.get("rxn_type","")) + " " + str(row.get("name","")))
        return m.group(1) if m else ""

    def _guess_keywords(row):
        kws = []
        for src in (row.get("retrobiocat_reaction",""), row.get("name","")):
            if isinstance(src, str) and src.strip():
                kws.extend(re.split(r"\W+", src.strip()))
        return [k.lower() for k in kws if len(k) > 2]

    RE_EC = re.compile(r'\d+\.\d+\.\d+\.(?:\d+|-)')

    def _rows_from_hits(step_row, hits, source="uniprot_rest"):
        out = []
        for h in hits:
            acc   = h.get("primaryAccession","")
            org   = (h.get("organism",{}) or {}).get("scientificName","")
            rev   = bool(h.get("entryType","") == "Swiss-Prot" or h.get("reviewed", False))
            plen  = int((h.get("sequence",{}) or {}).get("length") or h.get("length") or 0)
            seq   = (h.get("sequence",{}) or {}).get("value","")
            pdsc  = h.get("proteinDescription",{}) or {}
            pname = (
                ((pdsc.get("recommendedName",{}) or {}).get("fullName",{}) or {}).get("value","")
                or (pdsc.get("submissionNames",[{}]) or [{}])[0].get("fullName",{}).get("value","")
                or h.get("proteinName","") or h.get("uniProtkbId","")
            )
            ec_text = json.dumps(h, ensure_ascii=False)
            ecs = sorted(set(RE_EC.findall(ec_text)))
            out.append({
                "pathway_tag": step_row["pathway_tag"],
                "step_idx":    int(step_row["step_idx"]),
                "reaction_smiles": step_row.get("reaction_smiles",""),
                "rxn_type":    step_row.get("rxn_type",""),
                "name":        step_row.get("name","") or step_row.get("enzyme",""),
                "selenzyme_url": step_row.get("selenzyme_url",""),
                "uniprot_acc": acc,
                "protein_name": pname,
                "organism":     org,
                "reviewed":     rev,
                "sequence_len": plen,
                "ec_numbers":   ";".join(ecs),
                "aa_sequence":  seq,
                "source":       source
            })
        return out

    debug_log, out_rows = [], []

    for _, row in df.iterrows():
        tag  = row.get("pathway_tag","")
        i    = int(row.get("step_idx",0) or 0)
        per_this = 0
        step_dbg = {"pathway_tag": tag, "step_idx": i, "tried": []}

        # A) seeds (selenzyme + prefilled)
        seed = set()
        if row.get("seed_uniprots",""):
            # accept both ; and , just in case
            for a in re.split(r"[;,]+", str(row["seed_uniprots"])):
                a = a.strip()
                if a:
                    seed.add(a)
        seed |= _scrape_uniprot_from_selenzyme(row.get("selenzyme_url",""))

        if seed:
            acc_q = " OR ".join(f"accession:{a}" for a in list(seed)[:per_step])
            hits, meta = _uniprot_search(acc_q, size=per_step)
            step_dbg["tried"].append({"mode":"acc_batch", **meta, "n_hits":len(hits)})
            rows = _rows_from_hits(row, hits, source="uniprot_by_acc")
            out_rows.extend(rows); per_this += len(rows)

            if per_this < per_step:
                for a in list(seed)[:per_step]:
                    if per_this >= per_step:
                        break
                    if any(r["uniprot_acc"] == a and r["step_idx"]==i and r["pathway_tag"]==tag for r in out_rows):
                        continue
                    seq = _fetch_fasta(a)
                    step_dbg["tried"].append({"mode":"acc_fasta", "acc":a, "status":"ok" if seq else "miss"})
                    if seq:
                        out_rows.append({
                            "pathway_tag": tag, "step_idx": i,
                            "reaction_smiles": row.get("reaction_smiles",""),
                            "rxn_type": row.get("rxn_type",""),
                            "name": row.get("name","") or row.get("enzyme",""),
                            "selenzyme_url": row.get("selenzyme_url",""),
                            "uniprot_acc": a,
                            "protein_name": "",
                            "organism":     "",
                            "reviewed":     False,
                            "sequence_len": len(seq),
                            "ec_numbers":   "",
                            "aa_sequence":  seq,
                            "source":       "uniprot_fasta_fallback"
                        })
                        per_this += 1

        # B) EC-driven search
        if per_this < per_step:
            ec = _ec_from_row(row)
            if ec:
                q = f"ec:{ec}"
                if reviewed_first:
                    q += " AND reviewed:true"
                hits, meta = _uniprot_search(q, size=per_step-per_this)
                step_dbg["tried"].append({"mode":"ec", **meta, "n_hits":len(hits)})
                rows = _rows_from_hits(row, hits)
                out_rows.extend(rows); per_this += len(rows)

        # C) keywords (looser)
        if per_this < per_step:
            kws = _guess_keywords(row)
            if kws:
                q = " AND ".join(f'("{k}")' for k in kws)
                q += " AND annotation_score:[2 TO 5]"
                host = (state.get("host") or "").strip()
                if host:
                    q += f' AND (organism_name:"{host}" OR taxonomy_id:562)'
                hits, meta = _uniprot_search(q, size=per_step-per_this)
                step_dbg["tried"].append({"mode":"keywords", **meta, "n_hits":len(hits)})
                rows = _rows_from_hits(row, hits)
                out_rows.extend(rows); per_this += len(rows)

        debug_log.append(step_dbg)

    import pandas as pd
    cand = pd.DataFrame(out_rows)
    out = finish / "steps_sequence_candidates.csv"
    cand.to_csv(out, index=False)

    if save_debug:
        with open(finish / "sequence_fetch_debug.json","w") as f:
            json.dump(debug_log, f, indent=2)

    _log(f"sequence_fetch: wrote {len(cand)} candidates → {out}")
    _log(f"✔ done sequence_fetch ({time.time()-t0:.1f}s)")
    return state


In [ ]:
#sequence ranking

# --- shared tiny helper
def _get_workdir(st):
    try:
        return Path(st.get("workdir") or st.get("run_dir") or "/content/runs/run_001")
    except Exception:
        return Path("/content/runs/run_001")

# ========= sequence_fetch_node (ensure state, workdir, always returns) =========
def sequence_fetch_node(state, per_step=10, reviewed_first=False, timeout=12.0):
    # --- GUARDS ---
    if state is None:
        state = {}
    from pathlib import Path
    import time, json, re, requests, pandas as pd

    def log(msg):
        print(msg)
        state.setdefault("logs", []).append(msg)

    t0 = time.time()
    log("▶ start sequence_fetch (v4)")
    workdir = _get_workdir(state)
    state["workdir"] = str(workdir)  # ensure it exists for downstream nodes
    finish  = workdir / "retro_finish_out"
    finish.mkdir(parents=True, exist_ok=True)

    try:
        steps = finish / "steps_enzyme_choice.csv"
        if not steps.exists():
            # fallback to the plan (should have seed_uniprots after selenzyme_rank_node)
            steps = finish / "steps_enzyme_plan.csv"
        if not steps.exists():
            log("sequence_fetch: no steps file found; wrote empty candidates")
            (finish / "steps_sequence_candidates.csv").write_text("")
            log(f"✔ done sequence_fetch ({time.time()-t0:.1f}s)")
            return state

        df = pd.read_csv(steps).fillna("")
        # robust seed cleaner (accepts semicolons & commas)
        def _clean_seed_list(val):
            if not isinstance(val, str) or not val.strip():
                return []
            toks = re.split(r"[,\s;|]+", val.strip())
            return [t.upper() for t in toks if re.fullmatch(r"[A-Z0-9]{6,10}", t)]

        # Pull seeds if present
        seeds_col = "seed_uniprots" if "seed_uniprots" in df.columns else None
        df["seed_uniprots"] = df[seeds_col].astype(str) if seeds_col else ""

        # UniProt fetcher (with EC wildcard fix)
        S = requests.Session()
        S.headers.update({"User-Agent":"dbtl-seq/0.4"})
        base = "https://rest.uniprot.org/uniprotkb/search"

        def _ec_to_uniprot_pattern(ec_raw: str) -> str:
            if not isinstance(ec_raw, str) or not ec_raw.strip():
                return ""
            parts = ec_raw.strip().split(".")
            # trim trailing '-' and replace with wildcard
            while parts and parts[-1] == "-":
                parts.pop()
            if not parts:
                return ""
            return ".".join(parts) + ".*" if len(parts) < 4 else ".".join(parts)

        rows = []
        for _, r in df.iterrows():
            tag = str(r.get("pathway_tag",""))
            idx = int(pd.to_numeric(r.get("step_idx",0), errors="coerce") or 0)
            rxn = str(r.get("reaction_smiles",""))
            name= str(r.get("name",""))
            rxnt= str(r.get("rxn_type",""))
            seeds = _clean_seed_list(r.get("seed_uniprots",""))

            # (A) seed accessions path
            seed_hits = []
            if seeds:
                for acc in seeds:
                    params = dict(query=f"accession:{acc}", fields="accession,id,organism_name,reviewed,length,sequence,ec", format="json", size=1)
                    try:
                        resp = S.get(base, params=params, timeout=timeout)
                        if resp.status_code == 200:
                            js = resp.json()
                            for h in js.get("results", []):
                                seed_hits.append(h)
                    except Exception:
                        pass

            # (B) EC-based search (fallback)
            # try to derive EC from rxn_type/name
            ec = ""
            for src in (rxnt, name):
                m = re.search(r"(?:EC\s*)?(\d+\.\d+\.\d+\.\d+|\d+\.\d+\.\d+\.-|\d+\.\d+\.-\.-|\d+\.-\.-\.-)", str(src))
                if m:
                    ec = m.group(1); break
            ec_pat = _ec_to_uniprot_pattern(ec)

            hits = []
            if not seed_hits:
                qparts = []
                if ec_pat:
                    qparts.append(f"ec:{ec_pat}")
                elif name:
                    qparts.append(f'({name})')
                if reviewed_first:
                    qparts.append("reviewed:true")
                params = dict(query=" AND ".join(qparts) if qparts else "reviewed:true",
                              fields="accession,id,organism_name,reviewed,length,sequence,ec",
                              format="json", size=per_step)
                try:
                    resp = S.get(base, params=params, timeout=timeout)
                    if resp.status_code == 200:
                        hits = resp.json().get("results", [])
                except Exception:
                    hits = []
            else:
                hits = seed_hits

            for h in hits:
                rows.append({
                    "pathway_tag": tag,
                    "step_idx": idx,
                    "reaction_smiles": rxn,
                    "uniprot_acc": h.get("primaryAccession",""),
                    "organism": (h.get("organism",{}) or {}).get("scientificName",""),
                    "reviewed": bool(h.get("entryType","") == "Swiss-Prot" or h.get("reviewed", False)),
                    "sequence_len": int(h.get("sequence",{}).get("length",0) or h.get("length",0) or 0),
                    "aa_sequence": (h.get("sequence",{}) or {}).get("value",""),
                    "ec_numbers": ";".join(h.get("ec",[]) or []),
                    "source": "seed" if seed_hits else "search",
                })

        out = finish / "steps_sequence_candidates.csv"
        pd.DataFrame(rows).to_csv(out, index=False)
        log(f"sequence_fetch: wrote {len(rows)} candidates → {out}")
        log(f"✔ done sequence_fetch ({time.time()-t0:.1f}s)")
        return state

    except Exception as e:
        # never return None; log and continue
        log(f"sequence_fetch: ERROR {type(e).__name__}: {e}")
        out = finish / "steps_sequence_candidates.csv"
        if not out.exists():
            out.write_text("")  # ensure file exists so next node tolerates it
        log(f"✔ done sequence_fetch ({time.time()-t0:.1f}s)")
        return state


# ========= sequence_rank_node (guard state/workdir, always returns) =========
def sequence_rank_node(state):
    import time
    from pathlib import Path
    import pandas as pd

    t0 = time.time()
    workdir = Path(state.get("workdir") or state.get("run_dir") or ".")
    finish  = workdir / "retro_finish_out"
    finish.mkdir(parents=True, exist_ok=True)

    in_csv  = finish / "steps_sequence_candidates.csv"
    out_csv = finish / "steps_sequence_plan.csv"
    cols = ["pathway_tag","step_idx","enzyme","uniprot_acc","score","source",
            "organism","sequence_len","reviewed","aa_sequence","protein_name"]

    def log(m): print(m); state.setdefault("logs", []).append(m)
    log("▶ start sequence_rank")

    if (not in_csv.exists()) or in_csv.stat().st_size == 0:
        pd.DataFrame(columns=cols).to_csv(out_csv, index=False)
        log("sequence_rank: no or empty candidates → wrote empty plan (headers only)")
        log(f"✔ done sequence_rank ({time.time()-t0:.1f}s)")
        return state

    try:
        cand = pd.read_csv(in_csv)
    except Exception:
        pd.DataFrame(columns=cols).to_csv(out_csv, index=False)
        log("sequence_rank: read error → wrote empty plan (headers only)")
        log(f"✔ done sequence_rank ({time.time()-t0:.1f}s)")
        return state

    if cand.empty:
        pd.DataFrame(columns=cols).to_csv(out_csv, index=False)
        log("sequence_rank: 0 rows → wrote empty plan (headers only)")
        log(f"✔ done sequence_rank ({time.time()-t0:.1f}s)")
        return state

    # simple scoring
    cand["reviewed"] = cand.get("reviewed", False).astype(str).str.lower().isin(["true","1","yes","reviewed"]).astype(int)
    cand["sequence_len"] = pd.to_numeric(cand.get("sequence_len", 0), errors="coerce").fillna(0).astype(int)
    cand["len_pen"] = ((cand["sequence_len"] < 100) | (cand["sequence_len"] > 1000)).astype(int)
    cand["score"] = 1.0*cand["reviewed"] - 0.2*cand["len_pen"]

    # pick 1 per step
    cand["step_idx"] = pd.to_numeric(cand.get("step_idx",0), errors="coerce").fillna(0).astype(int)
    plan = (cand.sort_values(["pathway_tag","step_idx","score"], ascending=[True,True,False])
                 .groupby(["pathway_tag","step_idx"], as_index=False)
                 .head(1)
                 .reset_index(drop=True))

    keep_cols = ["pathway_tag","step_idx","uniprot_acc","score","source",
                 "organism","sequence_len","reviewed","aa_sequence"]
    plan = plan.reindex(columns=[c for c in keep_cols if c in plan.columns])
    plan.to_csv(out_csv, index=False)
    log(f"sequence_rank: wrote {len(plan)} picks → {out_csv}")
    log(f"✔ done sequence_rank ({time.time()-t0:.1f}s)")
    return state






In [ ]:
def _ec_to_uniprot_pattern(ec_raw: str) -> str:
    # '1.1.1.-' → '1.1.1.*', '1.1.-.-' → '1.1.*', '1.-.-.-' → '1.*'
    import re
    if not isinstance(ec_raw, str):
        return ""
    parts = ec_raw.strip().split(".")
    # replace trailing '-' segments with '*', keep leading defined parts
    while parts and parts[-1] == "-":
        parts.pop()
    if not parts:
        return ""
    return ".".join(parts) + ".*" if len(parts) < 4 else ".".join(parts)  # already full EC


In [ ]:
#build node

def build_node(state, overhang5="AATG", overhang3="GCTT", enzyme="BsaI"):
    """
    Turn steps_sequence_plan.csv into codon-optimized FASTA + simple Golden Gate plan.
    Writes:
      - retro_finish_out/sequences_codon_opt.fasta
      - retro_finish_out/cloning_plan.csv
    """
    import re, time, pandas as pd
    from pandas.errors import EmptyDataError
    from pathlib import Path

    def _log(m):
        print(m); state.setdefault("logs", []).append(m)

    _log("▶ start build (codon-opt + Golden Gate plan)")
    t0 = time.time()
    workdir = Path(state.get("workdir") or state.get("run_dir") or ".")
    finish  = workdir / "retro_finish_out"
    finish.mkdir(parents=True, exist_ok=True)

    plan_csv = finish / "steps_sequence_plan.csv"

    def _emit_empty_and_exit(note: str):
        # always create empty artifacts so downstream never crashes
        (finish / "sequences_codon_opt.fasta").write_text("")
        import pandas as pd
        pd.DataFrame(columns=[
            "pathway_tag","step_idx","uniprot_acc","protein_name","organism",
            "aa_len","cds_len","enzyme","site_forward","site_reverse","overhang_5","overhang_3","fasta_id"
        ]).to_csv(finish / "cloning_plan.csv", index=False)
        _log(f"build: {note}; wrote empty FASTA and cloning_plan.csv; skipping")
        _log(f"✔ done build ({time.time()-t0:.1f}s)")
        return state

    # Guard: file missing or zero-bytes
    if (not plan_csv.exists()) or plan_csv.stat().st_size == 0:
        return _emit_empty_and_exit("plan missing or empty")

    # Read plan safely
    try:
        df = pd.read_csv(plan_csv).fillna("")
    except EmptyDataError:
        return _emit_empty_and_exit("plan CSV had no columns (EmptyDataError)")

    if df.empty:
        return _emit_empty_and_exit("plan has 0 rows")

    # ---------- keep your existing codon optimization + Golden Gate code below ----------
    # Preferred E. coli codons (… your table …)
    pref = {
        'A':'GCT','R':'CGT','N':'AAT','D':'GAT','C':'TGT','Q':'CAA','E':'GAA','G':'GGT',
        'H':'CAT','I':'ATT','L':'CTG','K':'AAA','M':'ATG','F':'TTT','P':'CCT','S':'TCT',
        'T':'ACT','W':'TGG','Y':'TAT','V':'GTG','*':'TAA'
    }
    alt = {
        'A':['GCT','GCC','GCA','GCG'], 'R':['CGT','CGC','CGA','CGG','AGA','AGG'],
        'N':['AAT','AAC'], 'D':['GAT','GAC'], 'C':['TGT','TGC'], 'Q':['CAA','CAG'],
        'E':['GAA','GAG'], 'G':['GGT','GGC','GGA','GGG'], 'H':['CAT','CAC'],
        'I':['ATT','ATC','ATA'], 'L':['CTG','CTC','CTA','CTT','TTA','TTG'], 'K':['AAA','AAG'],
        'M':['ATG'], 'F':['TTT','TTC'], 'P':['CCT','CCC','CCA','CCG'],
        'S':['TCT','TCC','TCA','TCG','AGC','AGT'], 'T':['ACT','ACC','ACA','ACG'],
        'W':['TGG'], 'Y':['TAT','TAC'], 'V':['GTG','GTC','GTA','GTT'], '*':['TAA','TAG','TGA']
    }

    enzyme = enzyme.upper()
    if enzyme == "BSAI":
        site_f, site_r = "GGTCTC", "GAGACC"
    elif enzyme == "BSMBI":
        site_f, site_r = "CGTCTC", "GAGACG"
    else:
        site_f, site_r = "GGTCTC", "GAGACC"

    forbid = [site_f, site_r]

    def _remove_sites(dna, aa):
        def has_site(s):
            S = s.upper()
            if any(f in S for f in forbid): return True
            comp = str.maketrans("ACGT","TGCA")
            rc = S.translate(comp)[::-1]
            return any(f in rc for f in forbid)
        if not has_site(dna): return dna
        codons = [dna[i:i+3] for i in range(0, len(dna), 3)]
        AAs    = list(aa)
        for i in range(len(codons)):
            aa_i = AAs[i] if i < len(AAs) else None
            if aa_i not in alt: continue
            for c in alt[aa_i]:
                if c == codons[i]: continue
                trial = "".join(codons[:i] + [c] + codons[i+1:])
                if not has_site(trial): return trial
        return dna

    def aa_to_dna(aa):
        aa = aa.strip().upper().replace("U","C")
        return "".join(pref.get(a, "NNN") for a in aa)

    def decorate(dna):
        return f"{site_f}{overhang5}{dna}{overhang3}{site_r}"

    fasta_lines, build_rows = [], []
    keep = (df.sort_values(["pathway_tag","step_idx","score"], ascending=[True,True,False])
              .groupby(["pathway_tag","step_idx"], as_index=False).head(1))

    for _, r in keep.iterrows():
        tag  = str(r.get("pathway_tag",""))
        idx  = int(r.get("step_idx",0) or 0)
        acc  = str(r.get("uniprot_acc","") or r.get("uniprot_id",""))
        org  = str(r.get("organism",""))
        pname= str(r.get("protein_name",""))
        aa   = str(r.get("aa_sequence","")).strip().upper()
        if not aa: continue

        cds  = aa_to_dna(aa)
        cds2 = _remove_sites(cds, aa)
        full = decorate(cds2)

        name = f"{tag}_step{idx}_{acc}".replace(" ","_")
        fasta_lines.append(f">{name} | {pname} | {org}\n{full}")

        build_rows.append({
            "pathway_tag": tag, "step_idx": idx, "uniprot_acc": acc,
            "protein_name": pname, "organism": org,
            "aa_len": len(aa), "cds_len": len(cds2), "enzyme": enzyme,
            "site_forward": site_f, "site_reverse": site_r,
            "overhang_5": overhang5, "overhang_3": overhang3, "fasta_id": name
        })

    (finish / "sequences_codon_opt.fasta").write_text("\n".join(fasta_lines) + ("\n" if fasta_lines else ""))
    import pandas as pd
    pd.DataFrame(build_rows).to_csv(finish / "cloning_plan.csv", index=False)
    _log(f"build: wrote {len(build_rows)} constructs -> sequences_codon_opt.fasta, cloning_plan.csv")
    _log(f"✔ done build ({time.time()-t0:.1f}s)")
    return state



In [ ]:
import re, time
from typing import List, Optional

_UNIPROT_RE = re.compile(
    r"\b(?:[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9][A-Z0-9]{3}[0-9])\b"  # swissprot/trembl patterns
)

def _uniprot_hints_from_selenzyme(url: str, session=None, timeout: float = 8.0, max_ids: int = 6) -> List[str]:
    """
    Fetch a Selenzyme result page and extract UniProt accessions if present.
    - Looks for explicit links to uniprot.org and also scans the page text.
    - Returns a de-duplicated (uppercased) list, up to max_ids.
    """
    if not url or not isinstance(url, str):
        return []

    try:
        import requests
        from bs4 import BeautifulSoup  # optional but preferred
    except Exception:
        requests = None
        BeautifulSoup = None

    if requests is None:
        return []

    s = session or requests.Session()
    s.headers.update({"User-Agent": "dbtl-selenzyme-scraper/0.1"})

    try:
        r = s.get(url, timeout=timeout, allow_redirects=True)
        if r.status_code != 200 or "text/html" not in (r.headers.get("Content-Type","")):
            return []
        html = r.text or ""
    except Exception:
        return []

    ids = set()

    # 1) Parse anchors to uniprot
    if BeautifulSoup is not None:
        try:
            soup = BeautifulSoup(html, "html.parser")
            for a in soup.find_all("a", href=True):
                href = a["href"]
                if "uniprot.org" in href.lower():
                    # common patterns: /uniprotkb/<ACC>, /uniprot/<ACC>, query params, etc.
                    m = re.search(r"/uniprot(?:kb)?/([A-Z0-9]{6,10})", href, flags=re.I)
                    if m:
                        ids.add(m.group(1).upper())
                # also check the anchor text
                txt = (a.get_text() or "").strip()
                for m in _UNIPROT_RE.findall(txt.upper()):
                    ids.add(m)
        except Exception:
            pass

    # 2) Regex scan of the whole page fallback
    for m in _UNIPROT_RE.findall(html.upper()):
        ids.add(m)

    # Light politeness if this is called in a loop
    if session is None:
        time.sleep(0.2)

    # Defensive filter: accessions are 6–10 chars, keep the common 6-char ones first
    ordered = sorted(ids, key=lambda x: (len(x) != 6, x))  # prefer 6-char, then lexicographic
    return ordered[:max_ids]




In [ ]:
def _scrape_uniprot_from_selenzyme(url, session=None, timeout=8.0):
    """Return a set of UniProt accessions found on a Selenzyme result page."""
    import re, requests
    s = session or requests.Session()
    s.headers.update({"User-Agent":"dbtl-scraper/0.1"})
    try:
        r = s.get(url, timeout=timeout)
        if r.status_code != 200:
            return set()
        html = r.text
    except Exception:
        return set()
    hits = set()
    # plain accession tokens
    for m in re.finditer(r"\b([OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]{5})\b", html):
        hits.add(m.group(1))
    # UniProt links
    for m in re.finditer(r'uniprot(?:kb)?/(?:entry/)?([A-Z0-9]{6,10})', html, re.I):
        hits.add(m.group(1))
    return hits


In [ ]:
# --- Test Node (evaluate predicted constructs) ---
def test_node(state):
    """
    Simulates or imports wet-lab results for built constructs.
    """
    import pandas as pd, numpy as np
    from pathlib import Path

    finish = Path(state.get("workdir") or ".") / "retro_finish_out"
    builds = pd.read_csv(finish / "cloning_plan.csv")
    # fake activity measurements
    builds["measured_activity"] = np.random.rand(len(builds)) * 100
    builds["status"] = np.where(builds["measured_activity"] > 50, "active", "inactive")

    out = finish / "test_results.csv"
    builds.to_csv(out, index=False)
    print(f"✔ done test_node ({len(builds)} constructs tested) → {out}")
    state["test_results"] = str(out)
    return state


# --- Learn Node (update scoring models) ---
def learn_node(state):
    """
    Updates sequence ranking model using test data (simple reweighting stub).
    """
    import pandas as pd
    from pathlib import Path

    finish = Path(state.get("workdir") or ".") / "retro_finish_out"
    tests = pd.read_csv(state.get("test_results"))
    avg = tests.groupby("status")["measured_activity"].mean().to_dict()
    print(f"learn_node: mean activities by status = {avg}")

    # write placeholder updated model params
    (finish / "learn_update.json").write_text(json.dumps(avg, indent=2))
    print(f"✔ done learn_node → learn_update.json")
    return state


In [ ]:
# If needed in a fresh notebook:
!pip -q install langgraph langchain_core

from __future__ import annotations

import os, sys, re, json, time, math, glob, shutil, textwrap, datetime, subprocess
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from typing_extensions import Annotated, TypedDict
from datetime import datetime

import pandas as pd
from langgraph.graph import END, StateGraph
from langchain_core.runnables import RunnableLambda

from pathlib import Path
Path("/content/runs/run_001/micromamba/bin").mkdir(parents=True, exist_ok=True)
Path("/content/runs/run_001/micromamba/envs/retrobiocat").mkdir(parents=True, exist_ok=True)


# =========================
# State schema (fixes LastValue error by marking "run_id" static)
# =========================
class DBTLState(TypedDict, total=False):
    # Static keys (won’t change across steps; avoids InvalidUpdateError)
    run_id: Annotated[str, "static"]
    workdir: Annotated[str, "static"]

    # Inputs
    target_smiles: str
    host: str
    constraints: Dict[str, Any]

    # Bookkeeping
    status: str
    last_node: str
    logs: List[str]
    error: str
    signals: Dict[str, Any]
    approved: bool

    # Retro out
    retro_out_dir: str
    pathways_all_csv: Optional[str]
    pathways_solved_csv: Optional[str]
    pathways_all_json: Optional[str]
    pathways_solved_json: Optional[str]
    pathway_json_map: Dict[str, str]

    # Extractor / Thermo / Rank
    steps_plan_csv: Optional[str]
    steps_annotated_csv: Optional[str]
    ranked_csv: Optional[str]
    ranked_final_csv: Optional[str]
    ranking_method: Optional[str]
    thermo_method: Optional[str]

    # Selenzyme + sequences
    selenzyme_rows: int
    scraped_sequences_csv: Optional[str]
    sequences_ranked_csv: Optional[str]
    sequences_shortlist_csv: Optional[str]
    sequence_rank_method: Optional[str]
    sequence_rank_counts: Dict[str, int]

    # DoE
    screening_sheet_csv: Optional[str]
    pathway_brief_md: Optional[str]
    doe_pathway_tag: Optional[str]
    doe_steps: Optional[int]
    doe_sequences_merged: Optional[int]

    # Build
    build_out_dir: Optional[str]
    sequence_csv: Optional[str]

    # Export
    export_manifest: Optional[str]
    export_dir: Optional[str]
    human_gate_md: Optional[str]


def log(state: DBTLState, msg: str) -> Dict[str, List[str]]:
    return {"logs": (state.get("logs", []) + [msg])}


# ============== Helpers ==============

def wrap_node(name: str, fn):
    """Add start/finish logs + timing to any node."""
    def _wrapped(state: DBTLState) -> DBTLState:
        t0 = time.time()
        pre = {**state,
               "status": f"running:{name}",
               "last_node": name,
               "logs": [*state.get("logs", []), f"▶ start {name}"]}
        out = fn(pre)
        dt = time.time() - t0
        out_logs = [*out.get("logs", []), f"✔ done {name} ({dt:.1f}s)"]
        return {**out, "status": f"idle:{name}", "logs": out_logs, "last_node": name}
    return RunnableLambda(_wrapped)


# ============== Nodes ==============

import json, time, subprocess, shlex, os, pandas as pd
from pathlib import Path

def _log(state, msg):
    state.setdefault("logs", []).append(msg)

def retrosynthesis_node(state):
    t0 = time.time()
    _log(state, "▶ start retrosynthesis")
    _log(state, "retrosynthesis_node: starting")

    workdir = Path(state["workdir"])
    retro_out = workdir / "retro_out"
    retro_out.mkdir(parents=True, exist_ok=True)

    target_smi = state["target_smiles"]
    expanders = state.get("constraints", {}).get("expanders",
                    ["RetroBioCat","EnzymeMap","BKMS","RetroRules"])

    # --- Try native RBC2 first ---
    try:
        from rbc2 import MCTS, get_expanders
        _log(state, "[rbc2] native import ok; running inside current env")

        ex = get_expanders([e.lower() for e in expanders])
        mcts = MCTS(target_smi, ex)
        mcts.config.max_search_time = int(state.get("constraints", {}).get("max_search_time_s", 15))
        mcts.run()

        all_paths    = mcts.get_all_pathways()
        solved_paths = mcts.get_solved_pathways()

        def _rows(pwy, tag):
            rows = []
            for i, rxn in enumerate(pwy.reactions, start=1):
                rows.append({
                    "pathway_tag": tag, "step_idx": i,
                    "reaction_smiles": rxn.reaction_smiles(),
                    "substrates": " . ".join(rxn.substrates),
                    "products": rxn.product,
                    "rxn_type": rxn.rxn_type, "name": rxn.name,
                    "rbc2_score": rxn.score
                })
            return rows

        all_rows = []
        for k, pwy in enumerate(all_paths, start=1):
            tag = f"P{k:03d}"
            with open(retro_out / f"{tag}.json", "w", encoding="utf-8") as f:
                json.dump(pwy.save(), f, indent=2)
            all_rows.extend(_rows(pwy, tag))

        if all_rows:
            df = pd.DataFrame(all_rows)
            df.to_csv(retro_out / "steps.csv", index=False)

        # Minimal stubs for downstream (mirror your file names)
        df_steps = pd.read_csv(retro_out / "steps.csv")
        df_steps.to_csv(retro_out / "pathways.csv", index=False)
        df_steps.to_csv(retro_out / "pathways_solved.csv", index=False)

        _log(state, f"[rbc2] Total pathways: {len(all_paths)} | Solved: {len(solved_paths)}")

    except Exception as native_err:
        # --- Fallback: spawn micromamba env run (the recipe you posted) ---
        _log(state, "[rbc2] native import failed or missing; attempting in-place install & run")
        mm_bin   = workdir / "bin" / "micromamba"
        env_path = workdir / "micromamba" / "envs" / "retrobiocat"
        retro_out.mkdir(parents=True, exist_ok=True)

        try:
            # 1) micromamba bootstrap (idempotent)
            if not mm_bin.exists():
                cmd = f"curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest -o {workdir}/mm.tar.bz2"
                subprocess.run(shlex.split(cmd), check=True)
                subprocess.run(shlex.split(f"tar -xvjf {workdir}/mm.tar.bz2 -C {workdir} bin/micromamba"), check=True)

            # 2) env create
            if not env_path.exists():
                subprocess.run([str(mm_bin), "create", "-y", "-p", str(env_path), "-c", "conda-forge", "python=3.10", "pip"], check=True)

            # 3) install RBC2 and pins (your exact combo)
            def mm(cmd):
                return [str(mm_bin), "run", "-p", str(env_path)] + cmd

            subprocess.run(mm(["python","-m","pip","install","--upgrade","pip","setuptools","wheel"]), check=True)
            # Prefer zipball, fallback to git (as in your log)
            rc = subprocess.run(mm(["python","-m","pip","install","https://github.com/willfinnigan/RetroBioCat-2/archive/refs/heads/main.zip"]))
            if rc.returncode != 0:
                subprocess.run(mm(["python","-m","pip","install","git+https://github.com/willfinnigan/RetroBioCat-2.git"]), check=True)

            # Pins needed by your earlier runs
            subprocess.run(mm(["python","-m","pip","install","pydantic<2","networkx<3","tqdm","requests"]), check=True)

            # 4) run MCTS inside that env (writes into retro_out)
            pycode = f"""
import json, pandas as pd
from pathlib import Path
from rbc2 import MCTS, get_expanders

retro_out = Path(r"{retro_out}")
retro_out.mkdir(parents=True, exist_ok=True)

target_smi = r"{target_smi}"
expanders  = get_expanders({[e.lower() for e in expanders]})

mcts = MCTS(target_smi, expanders)
mcts.config.max_search_time = int({int(state.get("constraints", {}).get("max_search_time_s", 15))})
mcts.run()

def rows(pwy, tag):
    out = []
    for i, rxn in enumerate(pwy.reactions, start=1):
        out.append({{
            "pathway_tag": tag, "step_idx": i,
            "reaction_smiles": rxn.reaction_smiles(),
            "substrates": " . ".join(rxn.substrates),
            "products": rxn.product,
            "rxn_type": rxn.rxn_type, "name": rxn.name,
            "rbc2_score": rxn.score
        }})
    return out

all_rows = []
for k, pwy in enumerate(mcts.get_all_pathways(), start=1):
    tag = f"P{{k:03d}}"
    with open(retro_out / f"{{tag}}.json","w",encoding="utf-8") as f:
        json.dump(pwy.save(), f, indent=2)
    all_rows.extend(rows(pwy, tag))

if all_rows:
    df = pd.DataFrame(all_rows)
    df.to_csv(retro_out / "steps.csv", index=False)
    df.to_csv(retro_out / "pathways.csv", index=False)
    df.to_csv(retro_out / "pathways_solved.csv", index=False)

print("[rbc2] write:", retro_out)
"""
            subprocess.run(mm(["python","-c", pycode]), check=True)
            _log(state, "[rbc2] run ok")

        except subprocess.CalledProcessError as e2:
            _log(state, "[rbc2] run_failed")
            # Emit empty stub so downstream nodes don’t die
            (retro_out / "steps.csv").write_text("")
            (retro_out / "pathways.csv").write_text("")
            (retro_out / "pathways_solved.csv").write_text("")

    # Hand off common outputs into state
    state["retro_out"] = str(retro_out)
    state.setdefault("artifacts", []).extend([
        str(retro_out / "steps.csv"),
        str(retro_out / "pathways.csv"),
        str(retro_out / "pathways_solved.csv"),
    ])
    _log(state, f"✔ done retrosynthesis ({time.time()-t0:.1f}s)")
    return state


def extract_pathways_node(state: DBTLState) -> DBTLState:
    """
    Build enzyme-planning artifacts from RetroBioCat-2 outputs:
      - steps_enzyme_plan.csv (per-step details incl. enzyme classes, precedents, Selenzyme links)
      - pathways_ranked_no_thermo.csv (RBC2 + precedent-based ranking; no thermodynamics)
    """
    workdir = Path(state["workdir"])
    retro_out = workdir / "retro_out"
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # pick CSV (solved preferred, else all)
    csv_solved = retro_out / "pathways_solved_steps.csv"
    csv_all    = retro_out / "pathways_all_steps.csv"
    csv_in = csv_solved if csv_solved.exists() else csv_all
    if not csv_in.exists():
        raise FileNotFoundError(f"No pathways CSV found under {retro_out}")

    steps_df = pd.read_csv(csv_in)

    # pathway -> json map
    json_map: Dict[str,str] = dict(state.get("pathway_json_map") or {})
    if not json_map:
        for p in glob.glob(str(retro_out / "P*.json")):
            tag = Path(p).stem  # P001
            json_map[tag] = p

    from urllib.parse import quote_plus

    def selenzyme_url_from_rxn(rxn_smiles: str, org: str) -> str:
        return (
            "https://selenzyme.synbiochem.co.uk/selenzy/selenzy?reaction_smiles="
            + quote_plus(rxn_smiles)
            + "&organism="
            + quote_plus(org)
        )

    host_org = state.get("host") or "Escherichia coli"

    # expand per-step details from Pxxx.json
    records = []
    for _, r in steps_df.iterrows():
        tag   = str(r.get("pathway_tag"))
        idx   = int(r.get("step_idx"))
        rxn   = str(r.get("reaction_smiles",""))
        prod  = str(r.get("products", r.get("product","")))
        subs  = str(r.get("substrates",""))

        rxn_type = r.get("rxn_type","")
        name     = r.get("name","")
        rbc2_score = r.get("score", float("nan"))

        possible_enzymes = []
        enzyme_choices = []
        selected_enzyme = ""
        precedents = []

        pj = json_map.get(tag)
        if pj and os.path.exists(pj):
            try:
                data = json.load(open(pj, "r"))
                if isinstance(data, list) and 1 <= idx <= len(data):
                    d = data[idx - 1]
                    rxn_type = d.get("rxn_type", rxn_type)
                    name     = d.get("name", name)
                    tm = d.get("template_metadata", {}) or {}
                    if tm:
                        tkey = next(iter(tm.keys()))
                        tmeta = tm.get(tkey, {}) or {}
                        possible_enzymes = (tmeta.get("possible_enzymes") or [])
                        choices = (tmeta.get("enzyme_choices") or [])
                        enzyme_choices = [
                            " / ".join(x) if isinstance(x, (list, tuple)) else str(x)
                            for x in choices
                        ]
                        selected_enzyme = tmeta.get("selected_enzyme") or ""
                    precedents = d.get("precedents") or []
            except Exception:
                pass

        prec_sorted = sorted(precedents, key=lambda x: x.get("similarity", 0), reverse=True)[:3]
        prec_summ, best_sim = [], float("nan")
        for p in prec_sorted:
            sim = p.get("similarity", float("nan"))
            dat = p.get("data", {}) or {}
            doi = dat.get("html_doi", "") or dat.get("doi", "")
            enz = dat.get("enzyme_name", "") or p.get("name", "")
            cite = dat.get("short_citation", "")
            try:
                prec_summ.append(f"{enz} | sim={float(sim):.3f} | {cite} | {doi}")
            except Exception:
                prec_summ.append(f"{enz} | sim={sim} | {cite} | {doi}")
        if prec_sorted:
            best_sim = prec_sorted[0].get("similarity", float("nan"))

        records.append({
            "pathway_tag": tag,
            "step_idx": idx,
            "rxn_type": rxn_type,
            "retrobiocat_reaction": name,
            "reaction_smiles": rxn,
            "substrates": subs,
            "products": prod,
            "rbc2_score": rbc2_score,
            "selected_enzyme": selected_enzyme,
            "possible_enzymes": " ; ".join(possible_enzymes),
            "enzyme_choices": " ; ".join(enzyme_choices),
            "precedent_top3": " || ".join(prec_summ),
            "precedent_best_similarity": best_sim,
            "selenzyme_url": selenzyme_url_from_rxn(rxn, host_org),
        })

    plan = pd.DataFrame(records).sort_values(["pathway_tag","step_idx"]).reset_index(drop=True)

    # pathway ranking (no thermo)
    tmp = plan.copy()
    tmp["rbc2_score"] = pd.to_numeric(tmp["rbc2_score"], errors="coerce").fillna(0.0)
    tmp["precedent_best_similarity"] = pd.to_numeric(tmp["precedent_best_similarity"], errors="coerce").fillna(0.0)

    agg = tmp.groupby("pathway_tag").agg(
        steps_count=("step_idx","max"),
        sum_rbc2=("rbc2_score","sum"),
        sum_prec_sim=("precedent_best_similarity","sum"),
    ).reset_index()
    agg["rank_score"] = agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] - 0.1*agg["steps_count"]
    ranked = agg.sort_values(
        ["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
        ascending=[False, False, False, True]
    ).reset_index(drop=True)

    steps_plan_csv = str(finish_out / "steps_enzyme_plan.csv")
    ranked_csv     = str(finish_out / "pathways_ranked_no_thermo.csv")
    plan.to_csv(steps_plan_csv, index=False)
    ranked.to_csv(ranked_csv, index=False)

    new_state = {
        **state,
        "steps_plan_csv": steps_plan_csv,
        "ranked_csv": ranked_csv,
        "thermo_method": "none",
    }
    new_state["logs"] = state.get("logs", []) + [
        f"extracted pathways ({len(plan)} step rows, {len(ranked)} pathways)",
        f"steps_plan={steps_plan_csv}",
        f"ranked_no_thermo={ranked_csv}",
    ]
    return new_state


def thermo_score_node(state: DBTLState) -> DBTLState:
    """
    Lightweight 'match-only' thermo pass (no ChemAxon / ΔG):
      - Emits steps_annotated.csv and pathways_ranked.csv (same heuristic as extractor).
    """
    workdir    = Path(state["workdir"])
    mm_root    = workdir / "micromamba"
    env_prefix = mm_root / "envs" / "retrobiocat"  # same env
    retro_out  = workdir / "retro_out"
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    steps_plan_csv = state.get("steps_plan_csv")
    if not steps_plan_csv or not Path(steps_plan_csv).exists():
        csv_solved = retro_out / "pathways_solved_steps.csv"
        csv_all    = retro_out / "pathways_all_steps.csv"
        steps_plan_csv = str(csv_solved if csv_solved.exists() else csv_all)

    steps_out = str(finish_out / "steps_annotated.csv")
    rank_out  = str(finish_out / "pathways_ranked.csv")

    py = f"""\
import pandas as pd
from pathlib import Path

IN  = Path(r"{steps_plan_csv}")
OUT_STEPS = Path(r"{steps_out}")
OUT_RANK  = Path(r"{rank_out}")

if not IN.exists():
    raise FileNotFoundError(f"Input steps table not found: {{IN}}")

df = pd.read_csv(IN)

# Ensure required columns exist
for col in ["pathway_tag","step_idx","reaction_smiles","substrates","products",
            "rbc2_score","precedent_best_similarity","selenzyme_url"]:
    if col not in df.columns:
        df[col] = "" if col not in ["rbc2_score","precedent_best_similarity"] else 0.0

# Add placeholders for thermo
df["dGprime_kJ_per_mol"] = float("nan")
df["uncert_kJ_per_mol"]  = float("nan")
df["thermo_pass"]        = False
df["equilibrator_formula"] = ""

# Save annotated
df.to_csv(OUT_STEPS, index=False)

# Ranking (same as extractor)
tmp = df.copy()
tmp["rbc2_score"] = pd.to_numeric(tmp["rbc2_score"], errors="coerce").fillna(0.0)
tmp["precedent_best_similarity"] = pd.to_numeric(tmp["precedent_best_similarity"], errors="coerce").fillna(0.0)

agg = tmp.groupby("pathway_tag").agg(
    steps_count=("step_idx","max"),
    sum_rbc2=("rbc2_score","sum"),
    sum_prec_sim=("precedent_best_similarity","sum"),
).reset_index()

agg["rank_score"] = agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] - 0.1*agg["steps_count"]
ranked = agg.sort_values(["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
                         ascending=[False,False,False,True]).reset_index(drop=True)
ranked.to_csv(OUT_RANK, index=False)

print("Saved:")
print(" ", OUT_STEPS)
print(" ", OUT_RANK)
"""
    micromamba = workdir / "bin" / "micromamba"
    if not micromamba.exists():
        micromamba = "micromamba"

    env = os.environ.copy()
    env["MAMBA_ROOT_PREFIX"] = str(mm_root)

    try:
        _run(f"""{micromamba} run -p "{env_prefix}" python - <<'PY'\n{py}\nPY""", env=env)
    except subprocess.CalledProcessError:
        _run(f"python - <<'PY'\n{py}\nPY")

    coverage = 0
    method = "matchonly"

    new_state = {
        **state,
        "thermo_method": method,
        "steps_annotated_csv": steps_out,
        "ranked_csv": rank_out,
    }
    new_state["logs"] = state.get("logs", []) + [
        f"thermo: {method}, coverage={coverage}",
        f"steps_annotated={steps_out}",
        f"ranked={rank_out}",
    ]
    return new_state


def rank_node(state: DBTLState) -> DBTLState:
    """
    Merge thermo info (if available) into pathway ranking; else keep RBC2+precedent ranking.
    Emits pathways_ranked_final.csv
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    steps_csv = state.get("steps_annotated_csv") or state.get("steps_plan_csv")
    if not steps_csv or not Path(steps_csv).exists():
        raise FileNotFoundError("rank_node: steps CSV not found in state")

    df = pd.read_csv(steps_csv)
    df.columns = [c.strip() for c in df.columns]

    has_thermo = any(c in df.columns for c in ["dGprime_kJ_per_mol", "thermo_pass"])
    thermo_mode = bool(has_thermo and df["thermo_pass"].any())  # likely False in match-only

    df["rbc2_score"] = pd.to_numeric(df.get("rbc2_score", 0.0), errors="coerce").fillna(0.0)
    df["precedent_best_similarity"] = pd.to_numeric(df.get("precedent_best_similarity", 0.0),
                                                    errors="coerce").fillna(0.0)
    if "dGprime_kJ_per_mol" in df:
        df["dGprime_kJ_per_mol"] = pd.to_numeric(df["dGprime_kJ_per_mol"], errors="coerce")

    group = df.groupby("pathway_tag", dropna=True)

    if thermo_mode:
        ranked = (
            group.agg(
                steps_count=("step_idx","max"),
                sum_rbc2=("rbc2_score","sum"),
                sum_prec_sim=("precedent_best_similarity","sum"),
                thermo_pass_steps=("thermo_pass","sum"),
                sum_dGprime=("dGprime_kJ_per_mol","sum"),
            )
            .reset_index()
        )
        ranked["all_steps_pass"] = ranked["thermo_pass_steps"] == ranked["steps_count"]
        ranked["rank_score"] = (
            ranked["sum_rbc2"]
            + 0.2 * ranked["sum_prec_sim"]
            - 0.1 * ranked["steps_count"]
            - 0.001 * ranked["sum_dGprime"].fillna(0.0)
            + ranked["all_steps_pass"].astype(float) * 0.5
        )
        method = "rbc2+precedent+thermo"
    else:
        ranked = (
            group.agg(
                steps_count=("step_idx","max"),
                sum_rbc2=("rbc2_score","sum"),
                sum_prec_sim=("precedent_best_similarity","sum"),
            )
            .reset_index()
        )
        ranked["rank_score"] = (
            ranked["sum_rbc2"] + 0.2 * ranked["sum_prec_sim"] - 0.1 * ranked["steps_count"]
        )
        method = "rbc2+precedent"

    ranked = ranked.sort_values(
        ["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
        ascending=[False, False, False, True],
    ).reset_index(drop=True)

    out_csv = str(finish_out / "pathways_ranked_final.csv")
    ranked.to_csv(out_csv, index=False)

    new_state = {
        **state,
        "ranked_final_csv": out_csv,
        "ranking_method": method,
    }
    new_state["logs"] = state.get("logs", []) + [
        f"ranked pathways ({method}, {len(ranked)} entries)",
        f"ranked_final={out_csv}",
    ]
    return new_state


def selenzyme_node(state: DBTLState) -> DBTLState:
    """
    Collect Selenzyme links and try to scrape top sequences (polite, tolerant).
    Falls back to counting links if blocked. Emits selenzyme_scrape.csv if rows found.
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    steps_csv = state.get("steps_annotated_csv") or state.get("steps_plan_csv")
    if not steps_csv or not Path(steps_csv).exists():
        raise FileNotFoundError("selenzyme_node: no steps CSV found in state")

    steps = pd.read_csv(steps_csv)
    if "selenzyme_url" not in steps.columns:
        new_state = {
            **state,
            "selenzyme_rows": 0,
            "scraped_sequences_csv": None,
        }
        new_state["logs"] = state.get("logs", []) + ["selenzyme: no 'selenzyme_url' column; skipped"]
        return new_state

    url_map: Dict[str, List[Tuple[str,int]]] = {}
    for _, r in steps.iterrows():
        url = str(r.get("selenzyme_url") or "").strip()
        if not url:
            continue
        tag = str(r.get("pathway_tag"))
        idx = int(r.get("step_idx", 0))
        url_map.setdefault(url, []).append((tag, idx))

    unique_urls = list(url_map.keys())

    cfg = (state.get("constraints", {}) or {}).get("selenzyme", {}) or {}
    do_scrape: bool   = bool(cfg.get("scrape", True))
    max_urls: int     = int(cfg.get("max_urls", 40))
    timeout_s: int    = int(cfg.get("timeout_s", 15))
    sleep_s: float    = float(cfg.get("sleep_s", 1.0))

    urls_to_hit = unique_urls[:max_urls]
    scraped_rows: List[Dict] = []
    blocked_reason: Optional[str] = None

    def _try_scrape(url: str) -> List[Dict]:
        nonlocal blocked_reason
        try:
            import requests
            from bs4 import BeautifulSoup  # type: ignore

            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "Connection": "keep-alive",
            }
            resp = requests.get(url, headers=headers, timeout=timeout_s)
            if resp.status_code in (403, 429):
                raise RuntimeError(f"HTTP {resp.status_code}")
            if not resp.ok or not resp.text:
                return []
            html = resp.text
            soup = BeautifulSoup(html, "html.parser")

            table = None
            for t in soup.find_all("table"):
                ths = [th.get_text(strip=True).lower() for th in t.find_all("th")]
                joined = " ".join(ths)
                if any(k in joined for k in ["uniprot", "organism", "score", "ec", "enzyme"]):
                    table = t
                    break

            rows_local: List[Dict] = []
            if table:
                headers = [th.get_text(strip=True) for th in table.find_all("th")]
                idx_map = {h.lower(): i for i, h in enumerate(headers)}

                def _get(td_list, key_variants):
                    for k in key_variants:
                        i = idx_map.get(k)
                        if i is not None and i < len(td_list):
                            return td_list[i].get_text(strip=True)
                    return ""

                for tr in table.find_all("tr"):
                    tds = tr.find_all("td")
                    if not tds:
                        continue
                    acc = _get(tds, ["uniprot", "accession", "uniprot id"])
                    org = _get(tds, ["organism", "source", "species"])
                    scr = _get(tds, ["score", "similarity", "sim"])
                    ec  = _get(tds, ["ec", "ec number"])
                    enz = _get(tds, ["enzyme", "name", "protein"])

                    if not acc:
                        a = tds[0].find("a")
                        if a and a.get_text(strip=True):
                            acc = a.get_text(strip=True)
                    if not acc:
                        m = re.search(r"[A-NR-Z0-9]{{6,10}}", tr.get_text(" ", strip=True))
                        if m:
                            acc = m.group(0)

                    try:
                        scr_val = float(re.sub("[^0-9.+-eE]", "", scr)) if scr else None
                    except Exception:
                        scr_val = None

                    rows_local.append({
                        "accession": acc,
                        "organism": org,
                        "score": scr_val if scr_val is not None else scr,
                        "ec": ec,
                        "enzyme_name": enz,
                    })

            if not rows_local:
                tokens = re.findall(r"\b[OPQ][0-9][A-Z0-9]{{3}}[0-9]\b|\b[A-NR-Z0-9]{{6}}\b", html)
                rows_local = [{"accession": t, "organism": "", "score": "", "ec": "", "enzyme_name": ""} for t in dict.fromkeys(tokens)]

            return rows_local

        except Exception as e:
            blocked_reason = str(e)
            return []

    if do_scrape and len(urls_to_hit) > 0:
        for i, url in enumerate(urls_to_hit, 1):
            rows_local = _try_scrape(url)
            for (tag, idx) in url_map.get(url, []):
                for r in rows_local:
                    scraped_rows.append({
                        "pathway_tag": tag,
                        "step_idx": idx,
                        "selenzyme_url": url,
                        **r
                    })
            if i < len(urls_to_hit):
                try:
                    time.sleep(sleep_s)
                except Exception:
                    pass

    scraped_csv_path: Optional[str] = None
    if scraped_rows:
        out_csv = finish_out / "selenzyme_scrape.csv"
        pd.DataFrame(scraped_rows, columns=[
            "pathway_tag","step_idx","selenzyme_url",
            "accession","organism","score","ec","enzyme_name"
        ]).to_csv(out_csv, index=False)
        scraped_csv_path = str(out_csv)

    total_links = len(unique_urls)
    rows = len(scraped_rows)

    log_lines = [
        f"selenzyme: links={total_links}, scraped_rows={rows}, mode={'scrape' if do_scrape else 'count-only'}"
    ]
    if blocked_reason and rows == 0 and do_scrape:
        log_lines.append(f"selenzyme: scrape blocked or failed ({blocked_reason}); fell back to count-only")

    new_state = {
        **state,
        "selenzyme_rows": rows,
        "scraped_sequences_csv": scraped_csv_path,
    }
    new_state["logs"] = state.get("logs", []) + log_lines
    return new_state


def selenzyme_rank_node(state):
    """
    Reads steps_enzyme_plan.csv (or choice) and writes steps_enzyme_choice.csv with:
      - selenzyme_url intact
      - seed_uniprots column ALWAYS present ('' when none)
      - delimiters normalized to ';' (so downstream cleaner sees them)
    """
    import re, time
    from pathlib import Path
    import pandas as pd

    t0 = time.time()
    workdir = Path(state.get("workdir") or state.get("run_dir") or ".")
    finish  = workdir / "retro_finish_out"
    finish.mkdir(parents=True, exist_ok=True)

    # input can be steps_enzyme_plan.csv or already-ranked file; prefer plan if present
    plan = finish / "steps_enzyme_plan.csv"
    out  = finish / "steps_enzyme_choice.csv"

    if not plan.exists():
        raise FileNotFoundError(f"missing: {plan}")

    df = pd.read_csv(plan)

    # --- ensure column exists ---
    if "seed_uniprots" not in df.columns:
        df["seed_uniprots"] = ""

    # --- normalize anything we *do* have to canonical ';' delimiter ---
    # accept commas, pipes, spaces, semicolons; keep only UniProt-like tokens
    def _norm(s: str) -> str:
        if not isinstance(s, str) or not s.strip():
            return ""
        toks = re.split(r"[,\s;|]+", s.strip())
        toks = [t.upper() for t in toks if re.fullmatch(r"[A-Z0-9]{6,10}", t)]
        return ";".join(sorted(set(toks)))

    df["seed_uniprots"] = df["seed_uniprots"].map(_norm)

    # (optional) if you scraped anything temporary like 'seed_scraped' merge & normalize here:
    if "seed_scraped" in df.columns:
        merged = []
        for a, b in zip(df["seed_uniprots"], df["seed_scraped"]):
            aa = set(_norm(a).split(";")) if a else set()
            bb = set(_norm(b).split(";")) if isinstance(b, str) else set()
            tok = ";".join(sorted((aa | bb) - {""}))
            merged.append(tok)
        df["seed_uniprots"] = merged

    df.to_csv(out, index=False)
    n_nonempty = int((df["seed_uniprots"].fillna("")!="").sum())
    state.setdefault("logs", []).append(
        f"selenzyme_rank: seed_uniprots populated for {n_nonempty} / {len(df)} steps"
    )
    state["selenzyme_choice_csv"] = str(out)
    state["workdir"] = str(workdir)  # keep around for downstream nodes
    return state


def doe_node(state: DBTLState) -> DBTLState:
    """
    Build a DoE screening sheet + markdown brief for the top-ranked pathway.
    Also merges the top-ranked sequence per step (if available).
    """
    from urllib.parse import quote_plus

    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    rank_csv  = state.get("ranked_csv") or state.get("ranked_final_csv")
    steps_csv = state.get("steps_plan_csv") or state.get("steps_annotated_csv") or str(finish_out / "steps_enzyme_plan.csv")
    if not (rank_csv and Path(rank_csv).exists()):
        raise FileNotFoundError("doe_node: ranked_csv missing or not found.")
    if not (steps_csv and Path(steps_csv).exists()):
        raise FileNotFoundError("doe_node: steps CSV missing or not found.")

    ranked = pd.read_csv(rank_csv)
    steps  = pd.read_csv(steps_csv)
    ranked.columns = [c.strip() for c in ranked.columns]
    steps.columns  = [c.strip() for c in steps.columns]

    best_tag = str(ranked.iloc[0]["pathway_tag"])
    psteps = (
        steps[steps["pathway_tag"] == best_tag]
        .sort_values("step_idx")
        .reset_index(drop=True)
    )

    # bring in top-ranked sequence per step (optional)
    seq_csv = state.get("sequences_ranked_csv")
    merged_n = 0
    top_seq_by_step = None
    if seq_csv and Path(seq_csv).exists():
        seq = pd.read_csv(seq_csv)
        seq.columns = [c.strip() for c in seq.columns]
        if "pathway_tag" in seq.columns and "rank_score" in seq.columns:
            seq_best = seq[seq["pathway_tag"] == best_tag].copy()
            if not seq_best.empty:
                seq_best["rank_score"] = pd.to_numeric(seq_best["rank_score"], errors="coerce").fillna(0.0)
                seq_best = (
                    seq_best.sort_values(["step_idx","rank_score"], ascending=[True, False])
                            .groupby("step_idx", as_index=False)
                            .first()
                )
                keep_cols = {
                    "step_idx":"step_idx",
                    "accession":"sequence_accession",
                    "organism":"source_organism",
                    "uniprot_url":"uniprot_url"
                }
                for k in keep_cols.keys():
                    if k not in seq_best.columns:
                        seq_best[k] = ""
                top_seq_by_step = seq_best[list(keep_cols.keys())].rename(columns=keep_cols)

    cols = [
        "pathway_tag","step_idx","retrobiocat_reaction","reaction_smiles",
        "selected_enzyme","possible_enzymes","selenzyme_url",
        "sequence_accession","source_organism","uniprot_url",
        "assay_buffer","pH","temp_C","cofactor","cofactor_recycle",
        "substrate_conc_mM","enzyme_loading_mg_per_mL","time_h",
        "conversion_%","notes"
    ]
    rows = []
    for _, r in psteps.iterrows():
        query = quote_plus(str(r.get("selected_enzyme") or r.get("retrobiocat_reaction")))
        uniprot_search = f"https://www.uniprot.org/uniprotkb?query={query}"
        rows.append({
            "pathway_tag": best_tag,
            "step_idx": int(r.get("step_idx", 0)),
            "retrobiocat_reaction": r.get("retrobiocat_reaction", ""),
            "reaction_smiles": r.get("reaction_smiles", ""),
            "selected_enzyme": r.get("selected_enzyme", ""),
            "possible_enzymes": r.get("possible_enzymes", ""),
            "selenzyme_url": r.get("selenzyme_url", ""),
            "sequence_accession": "",
            "source_organism": "",
            "uniprot_url": uniprot_search,
            "assay_buffer": "",
            "pH": "",
            "temp_C": "",
            "cofactor": "",
            "cofactor_recycle": "",
            "substrate_conc_mM": "",
            "enzyme_loading_mg_per_mL": "",
            "time_h": "",
            "conversion_%": "",
            "notes": "",
        })
    df_screen = pd.DataFrame(rows, columns=cols)

    if top_seq_by_step is not None and not top_seq_by_step.empty:
        before_na = df_screen["sequence_accession"].isna().sum() + (df_screen["sequence_accession"] == "").sum()
        df_screen = df_screen.merge(top_seq_by_step, on="step_idx", how="left", suffixes=("","_best"))
        for col in ["sequence_accession","source_organism","uniprot_url"]:
            best_col = f"{col}_best"
            if best_col in df_screen.columns:
                df_screen[col] = df_screen[best_col].where(df_screen[best_col].notna() & (df_screen[best_col]!=""), df_screen[col])
                df_screen.drop(columns=[best_col], inplace=True)
        after_na = df_screen["sequence_accession"].isna().sum() + (df_screen["sequence_accession"] == "").sum()
        merged_n = max(0, before_na - after_na)

    screen_csv = str(finish_out / "enzyme_screening_sheet.csv")
    df_screen.to_csv(screen_csv, index=False)

    md_path = str(finish_out / "pathway_brief.md")
    lines = []
    lines.append(f"# Pathway {best_tag} — Enzyme Screening Summary\n")
    lines.append(f"**Total Steps:** {len(psteps)}\n")
    lines.append(f"**Ranking File:** `{Path(rank_csv).name}`\n")

    def bullet(txt): return f"- {txt}"

    for _, r in psteps.iterrows():
        lines.append(f"\n## Step {int(r['step_idx'])}: {r.get('retrobiocat_reaction','')}\n")
        lines.append(bullet(f"Reaction SMILES: `{r.get('reaction_smiles','')}`"))
        lines.append(bullet(f"Selected enzyme: {r.get('selected_enzyme','-')}"))
        lines.append(bullet(f"Possible enzymes: {r.get('possible_enzymes','-')}"))
        if "precedent_best_similarity" in psteps.columns and pd.notna(r.get("precedent_best_similarity")):
            lines.append(bullet(f"Precedent similarity: {r.get('precedent_best_similarity')}"))
        if r.get("selenzyme_url"):
            lines.append(bullet(f"[Selenzyme link]({r.get('selenzyme_url')})"))
        if merged_n and "sequence_accession" in df_screen.columns:
            acc = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"sequence_accession"].values[0]
            org = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"source_organism"].values[0]
            up  = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"uniprot_url"].values[0]
            if acc:
                lines.append(bullet(f"Top sequence: [{acc}]({up}) ({org})"))
        else:
            if r.get("selected_enzyme"):
                q = quote_plus(r.get("selected_enzyme"))
                lines.append(bullet(f"[UniProt search](https://www.uniprot.org/uniprotkb?query={q})"))

    with open(md_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    new_state = {
        **state,
        "screening_sheet_csv": screen_csv,
        "pathway_brief_md": md_path,
        "doe_pathway_tag": best_tag,
        "doe_steps": len(psteps),
        "doe_sequences_merged": int(merged_n),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"DoE sheet + brief ready for {best_tag}",
        f"screening={screen_csv}",
        f"brief={md_path}",
        f"sequences merged into DoE: {merged_n}",
    ]
    return new_state


def human_gate_node(state: DBTLState) -> DBTLState:
    """
    Pause-and-review checkpoint node. Writes a small summary file and
    checks for an external 'signals.human_approved' flag.
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    summary = {
        "top_pathway": state.get("doe_pathway_tag"),
        "steps": state.get("doe_steps"),
        "thermo_method": state.get("thermo_method"),
        "sequence_rank_counts": state.get("sequence_rank_counts", {}),
        "screening_sheet": state.get("screening_sheet_csv"),
        "brief": state.get("pathway_brief_md"),
    }

    gate_path = finish_out / "_HUMAN_GATE.md"
    with open(gate_path, "w", encoding="utf-8") as f:
        f.write("# Human Approval Gate\n\n")
        f.write("Please review the current design-build-test-learn outputs:\n\n")
        for k, v in summary.items():
            f.write(f"- **{k}**: {v}\n")
        f.write("\nMark `approved=True` in the next LangGraph signal to continue.\n")

    signals = state.get("signals", {})
    approved = bool(signals.get("human_approved")) or state.get("approved", False)

    new_state = {
        **state,
        "approved": approved,
        "human_gate_md": str(gate_path),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"human_gate: approval file ready at {gate_path}",
        f"human_gate: approved={approved}",
    ]
    return new_state




def export_node(state: DBTLState) -> DBTLState:
    """
    Export all key DBTL artifacts and write a manifest.json.
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    export_dir = workdir / "retro_export"
    export_dir.mkdir(parents=True, exist_ok=True)

    artifacts = []
    for k, v in state.items():
        if isinstance(v, str) and any(v.endswith(ext) for ext in [".csv",".md",".json",".xlsx"]):
            if os.path.exists(v):
                dst = export_dir / Path(v).name
                try:
                    shutil.copy2(v, dst)
                    artifacts.append(str(dst))
                except Exception as e:
                    state.setdefault("logs", []).append(f"export_node: failed to copy {v}: {e}")

    manifest = {
        "timestamp": datetime.now().isoformat(),
        "export_dir": str(export_dir),
        "artifacts": artifacts,
        "meta": {
            "pathway_tag": state.get("doe_pathway_tag"),
            "approved": state.get("approved"),
            "thermo_method": state.get("thermo_method"),
        },
    }
    manifest_path = export_dir / "manifest.json"
    with open(manifest_path, "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2)

    new_state = {
        **state,
        "export_manifest": str(manifest_path),
        "export_dir": str(export_dir),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"export_node: exported {len(artifacts)} artifacts",
        f"export_node: manifest={manifest_path}",
    ]
    return new_state


def simulate_node(state: "DBTLState") -> "DBTLState":
    """
    Simulate pathway feasibility (toy COBRApy flux demo + mass-balance check).
    Writes:
      - retro_finish_out/simulation_fluxes.csv
      - retro_finish_out/simulation_log.txt
    Falls back gracefully if COBRApy is unavailable.
    """
    import os, subprocess, textwrap, json, shutil, time
    from pathlib import Path
    from string import Template

    def _log(s, msg):
        s = {**s}
        s["logs"] = [*s.get("logs", []), msg]
        return s

    workdir = Path(state["workdir"]).expanduser().resolve()
    retro_out = workdir / "retro_out"
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # Inputs
    rank_csv  = state.get("ranked_final_csv") or state.get("ranked_csv")
    steps_csv = state.get("steps_annotated_csv") or state.get("steps_plan_csv")
    if not rank_csv or not Path(rank_csv).exists():
        return _log(state, "simulate: no ranking CSV; skipped")
    if not steps_csv or not Path(steps_csv).exists():
        return _log(state, "simulate: no steps CSV; skipped")

    # Outputs
    out_flux = finish_out / "simulation_fluxes.csv"
    log_path = finish_out / "simulation_log.txt"

    # Use the same micromamba env as RBC2
    mm_root    = workdir / "micromamba"
    env_prefix = mm_root / "envs" / "retrobiocat"
    micromamba = workdir / "bin" / "micromamba"
    if not micromamba.exists():
        micromamba = "micromamba"

    env = os.environ.copy()
    env["MAMBA_ROOT_PREFIX"] = str(mm_root)

    # Ensure cobra is present (best-effort)
    try:
        subprocess.run(
            f"""{micromamba} run -p "{env_prefix}" python -c "import cobra" """,
            shell=True, env=env, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
        )
        cobra_ok = True
    except subprocess.CalledProcessError:
        cobra_ok = False
        try:
            subprocess.run(
                f"""{micromamba} run -p "{env_prefix}" python -m pip install cobra""",
                shell=True, env=env, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
            )
            cobra_ok = True
        except subprocess.CalledProcessError:
            cobra_ok = False

    # ---- Embedded script (Template with $placeholders to avoid {} issues) ----
    tpl = Template(r"""
import pandas as pd, math, csv, sys
from pathlib import Path

RANK_CSV = Path("$rank_csv")
STEPS_CSV = Path("$steps_csv")
OUT_FLUX = Path("$out_flux")
LOG_PATH = Path("$log_path")

log_lines = []

def log(x):
    log_lines.append(str(x))

ranked = pd.read_csv(RANK_CSV)
steps  = pd.read_csv(STEPS_CSV)
best_tag = str(ranked.iloc[0]["pathway_tag"])

psteps = (steps[steps["pathway_tag"] == best_tag]
          .sort_values("step_idx")
          .reset_index(drop=True))

# Basic mass-balance parse: count participants LHS/RHS for sanity
def split_rxn(r):
    if not isinstance(r, str) or ">>" not in r: return [], []
    L,R = r.split(">>",1)
    subs=[s for s in L.split(".") if s]; prods=[p for p in R.split(".") if p]
    return subs, prods

imbalances = []
for _, row in psteps.iterrows():
    subs, prods = split_rxn(str(row.get("reaction_smiles","")))
    imbalances.append(abs(len(subs) - len(prods)))
mass_ok = (sum(imbalances) == 0)
log(f"mass_balance_ok={mass_ok} (total_imbalance={sum(imbalances)})")

# If COBRApy is available, create a toy model and assign unit flux per step
try:
    import cobra
    model = cobra.Model("toy_pathway")
    # Make exchange for overall substrate mix and product pool
    exch_in  = cobra.Reaction("EX_subs")
    exch_out = cobra.Reaction("EX_prod")
    model.add_reactions([exch_in, exch_out])
    exch_in.lower_bound  = -1000.0; exch_in.upper_bound  = 0.0
    exch_out.lower_bound = 0.0;     exch_out.upper_bound = 1000.0

    # create a metabolite per unique token just to wire reactions
    tokens = set()
    for rxn in psteps["reaction_smiles"].astype(str):
        L,R = split_rxn(rxn)
        tokens.update(L); tokens.update(R)
    # Minimal placeholders; we don't use stoichiometries beyond 1
    mets = {}
    for t in tokens:
        m_id = ("m_" + "".join([c if c.isalnum() else "_" for c in t]))[:60]
        mets[t] = cobra.Metabolite(id=m_id, name=t, compartment="c")

    # Add step reactions (each consumes its LHS tokens and produces RHS tokens)
    step_flux = []
    for _, row in psteps.iterrows():
        sid = f"R_step_{int(row['step_idx'])}"
        rxn = cobra.Reaction(sid)
        L,R = split_rxn(str(row.get("reaction_smiles","")))
        rxn.lower_bound = 0.0; rxn.upper_bound = 1000.0
        rxn.add_metabolites({ mets[s]: -1.0 for s in L if s in mets })
        rxn.add_metabolites({ mets[p]:  1.0 for p in R if p in mets })
        model.add_reactions([rxn])
        step_flux.append((sid, 1.0))  # target steady-state flux guess

    # Objective: push flux through the last step
    if step_flux:
        last_sid = step_flux[-1][0]
        model.objective = last_sid

    sol = model.optimize()
    fluxes = []
    if sol.status == "optimal":
        for sid, _ in step_flux:
            fluxes.append({"pathway_tag": best_tag,
                           "step_id": sid,
                           "flux": float(sol.fluxes.get(sid, 0.0))})
        flux_ok = True
    else:
        flux_ok = False
        for sid, _ in step_flux:
            fluxes.append({"pathway_tag": best_tag, "step_id": sid, "flux": float("nan")})

    pd.DataFrame(fluxes).to_csv(OUT_FLUX, index=False)
    log(f"cobra_status={sol.status}, flux_ok={flux_ok}, rows={len(fluxes)}")

except Exception as e:
    # COBRA missing or failed — emit a stub CSV with zeros
    fluxes = []
    for _, row in psteps.iterrows():
        fluxes.append({"pathway_tag": best_tag,
                       "step_id": f"R_step_{int(row['step_idx'])}",
                       "flux": 0.0})
    pd.DataFrame(fluxes).to_csv(OUT_FLUX, index=False)
    log(f"cobra_unavailable_or_failed: {e}; wrote zero-flux stub")

with open(LOG_PATH, "w", encoding="utf-8") as f:
    f.write("\n".join(log_lines))
print("simulate: wrote", OUT_FLUX, "and", LOG_PATH)
""")

    py = textwrap.dedent(tpl.substitute(
        rank_csv=str(rank_csv),
        steps_csv=str(steps_csv),
        out_flux=str(out_flux),
        log_path=str(log_path),
    ))

    # run inside env
    try:
        subprocess.run(
            f"""{micromamba} run -p "{env_prefix}" python - <<'PY'\n{py}\nPY""",
            shell=True, env=env, check=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT
        )
    except subprocess.CalledProcessError as e:
        # if even that fails, record and continue
        with open(log_path, "a", encoding="utf-8") as f:
            f.write("\nsubprocess failed:\n" + (e.stdout or ""))

    new_state = {
        **state,
        "simulation_flux_csv": str(out_flux) if out_flux.exists() else None,
        "simulation_log": str(log_path) if log_path.exists() else None,
    }
    new_state["logs"] = [*state.get("logs", []), f"simulate: fluxes={out_flux.exists()}, log={log_path.exists()}"]
    return new_state

import time, math, pandas as pd
from pathlib import Path

# --- helpers ---------------------------------------------------------------

import os, json, time, math, subprocess, datetime
from pathlib import Path
import pandas as pd

def _log(state, msg):
    print(msg)
    state.setdefault("logs", []).append(msg)


def ensure_equilibrator(env_prefix: Path):
    """
    Make sure equilibrator (api + cache) is installed in the env with pins that
    are compatible with your rbc2 pins (numpy==1.26.4, pandas>=2).
    Safe to re-run; will no-op if already satisfied.
    """
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if rc != 0:
        _log(state, f"thermo: pip install warning (continuing):\n{err.strip()}")

def ensure_cobra(env_prefix, state=None):
    """Install a cobra stack compatible with numpy>=1.26 / pandas>=2.x."""
    rc, out, err = _run_in_env(
        env_prefix,
        ["python","-c","import cobra; print('cobra_ok')"]
    )
    if rc == 0 and "cobra_ok" in (out or ""):
        return
    rc, out, err = _run_in_env(
        env_prefix,
        ["python","-m","pip","install","-q","--upgrade","cobra==0.29.0","optlang>=1.8.0","swiglpk>=5.0.10"]
    )
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("simulate_gem: cobra install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])

# --- THERMO NODE (patched) ------------------------------------------------



from pathlib import Path
import json, pandas as pd, os

# ---------- helpers used by thermo_node ----------

# -------- helpers used by thermo_node (safe to paste multiple times) --------



# ----------- PATCH: robust micromamba discovery + runner -------------

def _find_micromamba(env_prefix):
    """
    Return a usable micromamba executable path.
    Search common locations relative to the env, else bootstrap it to workdir/bin.
    """
    import os, stat, tarfile, urllib.request
    from pathlib import Path

    p = Path(env_prefix).resolve()
    # env_prefix = <workdir>/micromamba/envs/retrobiocat
    # workdir is two levels up from 'envs/retrobiocat'
    try:
        workdir = p.parents[2]
    except Exception:
        workdir = Path("/content/runs/run_001")

    candidates = [
        workdir / "bin" / "micromamba",                 # /content/runs/run_001/bin/micromamba
        workdir / "micromamba" / "bin" / "micromamba",  # /content/runs/run_001/micromamba/bin/micromamba
        Path("micromamba"),                             # from PATH, if any
    ]
    for c in candidates:
        if str(c) == "micromamba":
            # If a PATH micromamba exists, just return the string; Popen will resolve it
            return "micromamba"
        if c.exists():
            try:
                mode = c.stat().st_mode
                if not (mode & stat.S_IXUSR):
                    c.chmod(mode | stat.S_IXUSR)
            except Exception:
                pass
            return str(c)

    # --- Bootstrap: download and extract micromamba into <workdir>/bin ---
    workdir.mkdir(parents=True, exist_ok=True)
    (workdir / "bin").mkdir(parents=True, exist_ok=True)
    url = "https://micro.mamba.pm/api/micromamba/linux-64/latest"
    tar_path = workdir / "mm.tar.bz2"
    try:
        urllib.request.urlretrieve(url, tar_path.as_posix())
        with tarfile.open(tar_path.as_posix(), mode="r:bz2") as tf:
            # Extract only 'bin/micromamba' if present; else extract all and hope path matches
            members = [m for m in tf.getmembers() if m.name.endswith("/micromamba") or m.name == "micromamba" or m.name.endswith("bin/micromamba")]
            if members:
                for m in members:
                    tf.extract(m, path=workdir)
            else:
                tf.extractall(path=workdir)
        # Move to workdir/bin if needed
        possible = [
            workdir / "bin" / "micromamba",
            workdir / "micromamba" / "bin" / "micromamba",
            workdir / "micromamba.exe",  # just in case
        ]
        for c in possible:
            if c.exists():
                final = workdir / "bin" / "micromamba"
                if c != final:
                    try:
                        final.write_bytes(c.read_bytes())
                    except Exception:
                        pass
                try:
                    final.chmod(final.stat().st_mode | stat.S_IXUSR)
                except Exception:
                    pass
                return str(final)
    except Exception:
        # last resort: let shell resolve it if available
        return "micromamba"

    return "micromamba"




#########Thermo_node######################################################################

# ---------- helpers used by thermo_node (safe to paste more than once) ----------


def _ensure_equilibrator(env_prefix, state=None):
    """Idempotent install of equilibrator stack + compatible numpy/pandas/pint."""
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """
    Best-effort RDKit so we can turn SMILES → InChIKey and improve matching.
    Tries the light wheel (rdkit-pypi). If it fails, we continue without RDKit.
    """
    # Check first
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import rdkit; print('ok')"])
    if rc == 0 and "ok" in (out or ""):
        return
    # Try install (quiet)
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","rdkit-pypi==2022.9.5"])
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: RDKit install skipped (will continue without)")


# ------------------------------- thermo_node --------------------------------
# -------- helpers used by thermo_node (safe to paste multiple times) --------


def _ensure_equilibrator(env_prefix, state=None):
    """Best-effort, idempotent install of equilibrator stack with safe pins."""
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """
    Install RDKit into the same micromamba env (used to convert SMILES→InChIKey).
    Using conda-forge build for compatibility.
    """
    rc, out, err = _run_in_env(
        env_prefix,
        ["python","-c","import rdkit; print('rdkit_ok')"]
    )
    if rc == 0 and "rdkit_ok" in (out or ""):
        return
    # Install via mamba/conda (micromamba run is fine)
    rc, out, err = _run_in_env(
        env_prefix,
        ["python","-m","pip","install","-q","rdkit-pypi"]
    )
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: RDKit install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


# ------------------------------- thermo_node --------------------------------

# ---------- helpers used by thermo_node (safe to paste multiple times) ----------


def _ensure_equilibrator(env_prefix, state=None):
    """Best-effort, idempotent install of equilibrator stack with safe pins."""
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """Install RDKit (used for SMILES→InChIKey) into the same env."""
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import rdkit; print('rdkit_ok')"])
    if rc == 0 and "rdkit_ok" in (out or ""):
        return
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","rdkit-pypi"])
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: RDKit install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


# -------------------------------- thermo_node ----------------------------------
# ----------------------------- shared helpers ------------------------------


def _ensure_equilibrator(env_prefix, state=None):
    """Best-effort, idempotent install of equilibrator stack with safe pins."""
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """
    Install RDKit into the same micromamba env (used to convert SMILES→InChIKey).
    We use the PyPI wheel for portability.
    """
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import rdkit; print('rdkit_ok')"])
    if rc == 0 and "rdkit_ok" in (out or ""):
        return
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","rdkit-pypi"])
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: RDKit install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_cobra(env_prefix, state=None):
    """
    Install a cobra stack compatible with numpy>=1.26 / pandas>=2.0.
    """
    rc, out, err = _run_in_env(
        env_prefix,
        ["python","-m","pip","install","-q","--upgrade","cobra==0.29.0","optlang>=1.8.0","swiglpk>=5.0.10"]
    )
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("simulate_gem: cobra install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


# ------------------------------- thermo_node --------------------------------

# ---------------- helpers used by thermo/sim nodes (safe to redefine) ----------------

# -------- robust micromamba resolver + runner --------
# ---------- robust micromamba resolver + runner (drop-in patch) ----------

def _resolve_micromamba(env_prefix):
    """
    Find a usable micromamba binary.
    Tries the 'expected' location plus common fallbacks and $MICROMAMBA_EXE.
    If it finds a fallback, it mirrors it into the expected path for future calls.
    """
    import os, shutil
    from pathlib import Path

    expected = Path(env_prefix).parent.parent / "bin" / "micromamba"
    fallbacks = [
        expected,
        Path("/content/runs/run_001/bin/micromamba"),
        Path("/content/bin/micromamba"),
        Path(os.environ.get("MICROMAMBA_EXE", "")).expanduser(),
    ]

    for p in fallbacks:
        if p and str(p) != "" and Path(p).exists():
            # Mirror into the expected path if needed
            if p != expected and not expected.exists():
                try:
                    expected.parent.mkdir(parents=True, exist_ok=True)
                    try:
                        expected.symlink_to(p)
                    except Exception:
                        shutil.copy2(p, expected)
                        expected.chmod(expected.stat().st_mode | 0o111)
                except Exception:
                    # best-effort; still return the working fallback path
                    pass
            return str(p)

    # If we got here, nothing was found
    raise FileNotFoundError(
        "micromamba not found. Checked:\n"
        f" - {expected}\n"
        " - /content/runs/run_001/bin/micromamba\n"
        " - /content/bin/micromamba\n"
        " - $MICROMAMBA_EXE"
    )


def _run_in_env(env_prefix, argv, input_text=None, extra_env=None):
    """
    Run argv inside the micromamba env at env_prefix. Return (rc, stdout, stderr).
    Uses the robust micromamba resolver above.
    """
    import os, subprocess
    mm = _resolve_micromamba(env_prefix)
    cmd = [mm, "run", "-p", str(env_prefix)] + list(argv)

    env = os.environ.copy()
    if extra_env:
        env.update(extra_env)

    p = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE if input_text is not None else None,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        env=env,
    )
    out, err = p.communicate(input=input_text)
    return p.returncode, out, err


def _ensure_equilibrator(env_prefix, state=None):
    """Best-effort, idempotent install of equilibrator stack with safe pins."""
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """Install RDKit (pip wheel) into the same env; used for SMILES normalization."""
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import rdkit; print('rdkit_ok')"])
    if rc == 0 and "rdkit_ok" in (out or ""):
        return
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","rdkit-pypi"])
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: RDKit install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_cobra(env_prefix, state=None):
    """Make sure cobra & friends are present in env (modern, pandas>=2 compatible)."""
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import cobra; print('cobra_ok')"])
    if rc == 0 and "cobra_ok" in (out or ""):
        return
    pkgs = ["cobra==0.29.0", "optlang>=1.8.0", "swiglpk>=5.0.10"]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pkgs)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("simulate: cobra install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


# --------------------------------- THERMO NODE ---------------------------------

# -------- helpers used by thermo_node (safe to paste multiple times) --------



def _ensure_equilibrator(env_prefix, state=None):
    """Best-effort, idempotent install of equilibrator stack with safe pins."""
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """
    Install RDKit into the same micromamba env (used to convert SMILES→InChIKey).
    Uses rdkit-pypi (wheel). If that fails we continue gracefully.
    """
    rc, out, err = _run_in_env(
        env_prefix,
        ["python","-c","import rdkit; print('rdkit_ok')"]
    )
    if rc == 0 and (out or "").strip().endswith("rdkit_ok"):
        return
    rc, out, err = _run_in_env(
        env_prefix,
        ["python","-m","pip","install","-q","rdkit-pypi"]
    )
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: RDKit install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


# ------------------------------- thermo_node --------------------------------

# ---------- helpers used by thermo_node (safe to paste multiple times) ----------

def _run_in_env(env_prefix, argv, input_text=None, extra_env=None):
    """
    Run a command inside the micromamba env at env_prefix.
    Returns (rc, stdout, stderr).
    """
    import os, subprocess
    from pathlib import Path
    mm = Path(env_prefix).parent.parent / "bin" / "micromamba"
    cmd = [str(mm), "run", "-p", str(env_prefix)] + list(argv)
    env = os.environ.copy()
    if extra_env:
        env.update(extra_env)
    p = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE if input_text is not None else None,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        env=env,
    )
    out, err = p.communicate(input=input_text)
    return p.returncode, out, err


def _ensure_equilibrator(env_prefix, state=None):
    """Best-effort, idempotent install of equilibrator stack with safe pins."""
    pins = [
        "numpy==1.26.4",
        "pandas==2.2.2",
        "pint==0.22",
        "equilibrator-api==0.6.0",
        "equilibrator-cache==0.6.0",
    ]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pins)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: pip install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


def _ensure_rdkit(env_prefix, state=None):
    """
    Install RDKit into the same micromamba env (used to convert SMILES→InChIKey).
    """
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import rdkit; print('rdkit_ok')"])
    if rc == 0 and "rdkit_ok" in (out or ""):
        return
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","rdkit-pypi"])
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("thermo: RDKit install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


# -------------------------------- thermo_node (patched) --------------------------------

# ---- helper to ensure RDKit in the *notebook* interpreter (not micromamba) ----
def _ensure_rdkit_outer():
    """Ensure rdkit is importable in the current Python (outside micromamba)."""
    try:
        import rdkit  # noqa: F401
        return
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rdkit-pypi"])
        import rdkit  # noqa: F401


# -------------------------------- thermo_node ---------------------------------

# === Minimal helpers so thermo_node + simulate_gem_node work ===
from pathlib import Path
import subprocess, os, math, time, pandas as pd

def _find_micromamba(workdir):
    for c in [
        Path(workdir) / "bin" / "micromamba",
        Path(workdir) / "micromamba" / "bin" / "micromamba",
        Path("/content/runs/run_001/bin/micromamba"),
        Path("/content/runs/run_001/micromamba/bin/micromamba"),
    ]:
        if c.exists():
            return c
    return "micromamba"


def _ensure_pkg(env_prefix, *specs):
    """Idempotent pip install for one or more package specs inside env_prefix."""
    return _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade",*specs])


def _ensure_equilibrator_and_rdkit(env_prefix, state=None):
    """
    Make sure the env has a compatible equilibrator stack + RDKit.
    This is safe to call on every run (fast if already satisfied).
    """
    pins_eq = [
        "pip", "setuptools", "wheel",
        "numpy==1.26.4", "pandas==2.2.2", "pint==0.22",
        "equilibrator-api==0.6.0", "equilibrator-cache==0.6.0",
    ]
    rc1, out1, err1 = _ensure_pkg(env_prefix, *pins_eq)
    if state is not None and rc1 != 0:
        state.setdefault("logs", []).append("thermo: warning installing equilibrator stack (continuing)")
        if err1:
            state["logs"].append(err1.strip().splitlines()[-1])

    # RDKit (wheel; no conda needed)
    rc2, out2, err2 = _ensure_pkg(env_prefix, "rdkit-pypi")
    if state is not None and rc2 != 0:
        state.setdefault("logs", []).append("thermo: warning installing RDKit (continuing)")
        if err2:
            state["logs"].append(err2.strip().splitlines()[-1])



def extract_node(state: dict) -> dict:
    _log(state, "▶ start extract")
    wd = Path(state["workdir"])
    retro = wd / "retro_out"
    finish = wd / "retro_finish_out"
    finish.mkdir(parents=True, exist_ok=True)

    # sanity: if retrosynthesis failed, abort early with context
    retro_flag = state.get("retro_status")  # set by your retrosynthesis node if you added it
    if retro_flag == "failed":
        errlog = retro / "_rbc2_runner.err.log"
        tail = _log_tail(errlog)
        raise RuntimeError(
            "Retrosynthesis produced a stub (empty outputs). "
            "See _rbc2_runner.err.log tail below:\n" + tail
        )

    # candidate inputs (newer writers first, then legacy)
    candidates = [
        retro / "pathways_solved_steps.csv",
        retro / "pathways_all_steps.csv",
        retro / "steps.csv",             # legacy
        retro / "pathways.csv",          # legacy
    ]

    df = None
    source = None
    for p in candidates:
        if _file_nonempty(p):
            try:
                df = pd.read_csv(p)
                source = p.name
                break
            except pd.errors.EmptyDataError:
                pass  # keep searching

    # Fallback: construct from JSON if CSVs are empty but we have route JSONs
    if df is None:
        routes_json = retro / "routes.json"
        if _file_nonempty(routes_json):
            with open(routes_json, "r", encoding="utf-8") as f:
                routes = json.load(f)
            rows = []
            for tag, route in routes.items():
                for i, step in enumerate(route.get("steps", []), start=1):
                    rows.append({
                        "pathway_tag": tag,
                        "step_idx": i,
                        "reaction_smiles": step.get("reaction_smiles", ""),
                        "substrates": " . ".join(step.get("substrates", [])),
                        "products": " . ".join(step.get("products", [])),
                        "rxn_type": step.get("rxn_type", ""),
                        "name": step.get("name", ""),
                        "rbc2_score": step.get("score", 0.0),
                    })
            if rows:
                df = pd.DataFrame(rows)
                source = "routes.json"
            else:
                df = None

    if df is None or df.empty:
        # Helpful diagnostics
        diag = []
        for p in candidates + [retro / "routes.json"]:
            diag.append(f"{p} : {'OK' if _file_nonempty(p) else 'missing/empty'}")
        err_tail = _log_tail(retro / "_rbc2_runner.err.log")
        raise RuntimeError(
            "extract_node could not find usable retrosynthesis outputs.\n"
            + "\n".join(diag)
            + ("\n\n_rbc2_runner.err.log (tail):\n" + err_tail if err_tail else "")
        )

    # Normalize/ensure required columns
    for col in ["pathway_tag","step_idx","reaction_smiles","substrates","products",
                "rbc2_score","precedent_best_similarity","selenzyme_url"]:
        if col not in df.columns:
            df[col] = 0.0 if col in ("rbc2_score","precedent_best_similarity") else ""

    # Persist standard downstream files
    steps_plan = finish / "steps_enzyme_plan.csv"
    ranked_no_thermo = finish / "pathways_ranked_no_thermo.csv"

    df.to_csv(steps_plan, index=False)

    # very light ranking (same as your previous)
    tmp = df.copy()
    tmp["rbc2_score"] = pd.to_numeric(tmp["rbc2_score"], errors="coerce").fillna(0.0)
    tmp["precedent_best_similarity"] = pd.to_numeric(tmp["precedent_best_similarity"], errors="coerce").fillna(0.0)
    agg = tmp.groupby("pathway_tag").agg(
        steps_count=("step_idx","max"),
        sum_rbc2=("rbc2_score","sum"),
        sum_prec_sim=("precedent_best_similarity","sum"),
    ).reset_index()
    agg["rank_score"] = agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] - 0.1*agg["steps_count"]
    ranked = agg.sort_values(
        ["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
        ascending=[False,False,False,True]
    ).reset_index(drop=True)
    ranked.to_csv(ranked_no_thermo, index=False)

    _log(state, f"extracted pathways from {source} "
                f"({len(df)} step rows, {ranked.shape[0]} pathways)")
    state["extract_steps_csv"] = str(steps_plan)
    state["extract_rank_csv"]  = str(ranked_no_thermo)
    _log(state, "✔ done extract")
    return state


def rank_node(state: dict) -> dict:
    """
    #Lightweight shim so your call `state = rank_node(state)` won’t error.
    #If thermo_node already wrote 'pathways_ranked_final.csv', we just log and return.
    #Otherwise, we promote 'pathways_ranked_no_thermo.csv' to 'pathways_ranked_final.csv'.
    """
    from pathlib import Path
    import shutil

    def log(msg): state.setdefault("logs", []).append(msg)

    run_id  = state.get("run_id", "run")
    workdir = Path(state.get("workdir", f"/content/runs/{run_id}"))
    finish_out = Path(state.get("retro_finish_out_dir", workdir / "retro_finish_out"))
    finish_out.mkdir(parents=True, exist_ok=True)

    ranked_final = finish_out / "pathways_ranked_final.csv"
    if ranked_final.exists():
        log("▶ start rank")
        log("ranked pathways (already computed with thermo)")
        log(f"ranked_final={ranked_final}")
        log("✔ done rank (0.0s)")
        return state

    ranked_no = finish_out / "pathways_ranked_no_thermo.csv"
    if ranked_no.exists():
        shutil.copy2(ranked_no, ranked_final)
        log("▶ start rank")
        log("ranked pathways (no thermo available; copied no_thermo as final)")
        log(f"ranked_final={ranked_final}")
        log("✔ done rank (0.0s)")
    else:
        log("▶ start rank")
        log("rank_node: no ranking inputs found")
        log("✔ done rank (0.0s)")

    return state

# ---------- helpers used by simulate_gem_node (safe to paste multiple times) ----------


def _ensure_cobra(env_prefix, state=None):
    """Install a cobra version compatible with numpy>=1.26 / pandas>=2."""
    pkgs = ["cobra==0.29.0", "optlang>=1.8.0", "swiglpk>=5.0.10"]
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","--upgrade"] + pkgs)
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("simulate_gem: cobra install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])

def _run_and_log(cmd, log_path, cwd=None, env=None):
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=cwd, env=env, text=True)
    with open(log_path, "w", encoding="utf-8") as f:
        for line in p.stdout:
            f.write(line)
    rc = p.wait()
    return rc

def _write_rbc2_driver(py_path: Path, target_smi: str, retro_out: Path):
    code = f"""
import json, pandas as pd
from pathlib import Path
from rbc2 import MCTS, get_expanders

retro_out = Path({json.dumps(str(retro_out))})
retro_out.mkdir(parents=True, exist_ok=True)

target_smi = {json.dumps(target_smi)}
expanders  = get_expanders(["retrobiocat"])  # or add others: ["retrobiocat","enzymemap","bkmst","retrorules"]

mcts = MCTS(target_smi, expanders)
mcts.config.max_search_time = int(20)  # seconds; tune as you like
mcts.run()

all_paths    = mcts.get_all_pathways()
solved_paths = mcts.get_solved_pathways()

def pathway_rows(pwy, tag):
    rows = []
    for i, rxn in enumerate(pwy.reactions, start=1):
        rows.append({{
            "pathway_tag": tag,
            "step_idx": i,
            "reaction_smiles": rxn.reaction_smiles(),
            "substrates": " . ".join(rxn.substrates),
            "products":  rxn.product,
            "rxn_type": rxn.rxn_type,
            "name": rxn.name,
            "rbc2_score": rxn.score,
            "precedent_best_similarity": 0.0,  # placeholder; your ranker can fill if available later
            "selenzyme_url": ""
        }})
    return rows

def write_bundle(paths, prefix):
    all_rows = []
    for k, pwy in enumerate(paths, start=1):
        tag = f"P{k:03d}"
        (retro_out / f"{{tag}}.json").write_text(json.dumps(pwy.save(), indent=2))
        all_rows.extend(pathway_rows(pwy, tag))
    if all_rows:
        df = pd.DataFrame(all_rows)
        df.to_csv(retro_out / f"pathways_{{prefix}}_steps.csv", index=False)
        # legacy name for downstream compatibility
        if prefix == "all":
            df.to_csv(retro_out / "steps.csv", index=False)
    return len(all_rows)

n_all = write_bundle(all_paths, "all")
n_sol = write_bundle(solved_paths, "solved")

print(f"[rbc2] Total pathways: {{len(all_paths)}} | Solved: {{len(solved_paths)}} | Rows(all): {{n_all}}")
"""
    py_path.write_text(textwrap.dedent(code))

# =========================
# FIXED RETROSYNTHESIS NODE
# =========================
from pathlib import Path
import json, traceback, pandas as pd, os, time

# --- helpers: paths + shell ---------------------------------------------------
from pathlib import Path
import subprocess, shlex, json, os, textwrap, sys
from datetime import datetime

def _p(*parts):
    return Path(*map(str, parts)).resolve()

def _now():
    return datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")

def _mkdir(p):
    p = Path(p); p.mkdir(parents=True, exist_ok=True); return p

def _write_text(p, txt):
    p = Path(p); _mkdir(p.parent); p.write_text(txt); return p

def _file_nonempty(p):
    p = Path(p)
    try:
        return p.exists() and p.is_file() and p.stat().st_size > 0
    except FileNotFoundError:
        return False

def _log_tail(p, n=60):
    p = Path(p)
    if not p.exists(): return "(no log)"
    try:
        with p.open("r", errors="ignore") as fh:
            lines = fh.readlines()
        return "".join(lines[-n:])
    except Exception as e:
        return f"(could not read log: {e})"

def _run(cmd, cwd=None, env=None, log_path=None):
    """Run a shell command and stream stdout/stderr to a log file."""
    proc_env = os.environ.copy()
    if env: proc_env.update(env)
    if isinstance(cmd, str):
        args = cmd if os.name == "nt" else cmd
        shell = True
    else:
        args = cmd
        shell = False
    log_fh = None
    try:
        if log_path:
            _mkdir(Path(log_path).parent)
            log_fh = open(log_path, "a", buffering=1, encoding="utf-8", errors="ignore")
            log_fh.write(f"\n[{_now()}] CMD: {cmd if isinstance(cmd,str) else ' '.join(map(shlex.quote,cmd))}\n")
        proc = subprocess.Popen(args, cwd=cwd, env=proc_env, shell=shell,
                                stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                                text=True, bufsize=1, universal_newlines=True)
        for line in proc.stdout:
            if log_fh: log_fh.write(line)
        proc.wait()
        rc = proc.returncode
        if log_fh: log_fh.write(f"[{_now()}] RC={rc}\n")
        return rc
    finally:
        if log_fh: log_fh.close()

# --- node: retrosynthesis (runs RBC2 *inside micromamba*) ---------------------

def retrosynthesis_node(state: dict) -> dict:
    """
    Runs RBC2 inside the micromamba env (retrobiocat) and writes outputs into
    <workdir>/retro_out so the rest of your pipeline (extract/thermo/...) works.
    """
    import os, json, textwrap, subprocess, shlex
    from pathlib import Path
    from datetime import datetime, timezone

    def _ts():
        return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

    workdir = Path(state["workdir"]).resolve()
    retro   = workdir / "retro_out"
    logsdir = workdir / "logs"
    bindir  = workdir / "bin"
    cfgdir  = workdir / "config"
    env_prefix = workdir / "micromamba" / "envs" / "retrobiocat"

    retro.mkdir(parents=True, exist_ok=True)
    logsdir.mkdir(parents=True, exist_ok=True)
    bindir.mkdir(parents=True, exist_ok=True)
    cfgdir.mkdir(parents=True, exist_ok=True)

    out_log = logsdir / "_rbc2_runner.out.log"
    err_log = logsdir / "_rbc2_runner.err.log"

    # Write a small config file the runner will read
    cfg_json = cfgdir / "retro_cfg.json"
    cfg_json.write_text(json.dumps({
        "target_smiles": state["target_smiles"],
        "max_steps": int(state.get("constraints", {}).get("max_steps", 5)),
        "outdir": retro.as_posix()
    }, indent=2))

    # Write the runner script WITHOUT outer f-strings (to avoid brace interpolation)
    runner_py = bindir / "rbc2_runner.py"
    runner_source = r'''
import json, sys
from pathlib import Path
from datetime import datetime, timezone

def ts():
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# Paths provided by the parent process
CFG_JSON_PATH = r"{CFG_JSON_PATH}"

cfg = json.loads(Path(CFG_JSON_PATH).read_text())
target = cfg["target_smiles"]
max_steps = int(cfg["max_steps"])
outdir = Path(cfg["outdir"])
outdir.mkdir(parents=True, exist_ok=True)

print("[{}] RBC2: starting for target {}".format(ts(), target))

try:
    from rbc2 import MCTS, get_expanders
except Exception as e:
    raise RuntimeError("RBC2 import failed: {}: {}".format(e.__class__.__name__, e))

# Conservative, broadly-compatible call pattern
expanders = get_expanders(["retrobiocat"])
mcts = MCTS(target, expanders)
mcts.config.max_search_time = int(20)
mcts.config.max_depth = max_steps

mcts.run()

all_paths = mcts.get_all_pathways()
solved_paths = mcts.get_solved_pathways()

# Serialize a minimal steps.csv and pathways.csv the extractor expects
import pandas as pd

def rows_for(paths):
    rows = []
    for p_i, pwy in enumerate(paths):
        tag = "P%03d" % (p_i + 1)
        for s_i, rxn in enumerate(pwy.reactions, start=1):
            rows.append({
                "pathway_tag": tag,
                "step_idx": s_i,
                "reaction_smiles": rxn.reaction_smiles(),
                "substrates": " . ".join(rxn.substrates),
                "products": rxn.product,
                "rxn_type": getattr(rxn, "rxn_type", ""),
                "name": getattr(rxn, "name", ""),
                "rbc2_score": float(getattr(rxn, "score", 0.0)),
                "precedent_best_similarity": float(getattr(rxn, "precedent_best_similarity", 0.0)),
                "selenzyme_url": getattr(rxn, "selenzyme_url", ""),
            })
    return rows

# Write main outputs
all_rows = rows_for(all_paths)
if all_rows:
    pd.DataFrame(all_rows).to_csv(outdir / "steps.csv", index=False)
    pd.DataFrame({"pathway_tag": sorted({r["pathway_tag"] for r in all_rows})}).to_csv(outdir / "pathways.csv", index=False)

# Optional "all" and "solved" variants if you like
solved_rows = rows_for(solved_paths)
if solved_rows:
    pd.DataFrame(solved_rows).to_csv(outdir / "pathways_solved_steps.csv", index=False)
if all_rows:
    pd.DataFrame(all_rows).to_csv(outdir / "pathways_all_steps.csv", index=False)

print("[{}] RBC2 complete → {}".format(ts(), outdir))
'''
    runner_py.write_text(
        runner_source.replace("{CFG_JSON_PATH}", cfg_json.as_posix())
    )

    # Locate micromamba
    micromamba = workdir / "bin" / "micromamba"
    if not micromamba.exists():
        micromamba = Path("micromamba")  # fallback

    cmd = [
        str(micromamba),
        "run",
        "-p", str(env_prefix),
        "python",
        str(runner_py),
    ]

    # Run and capture logs
    import subprocess, shlex
    with out_log.open("wb") as out, err_log.open("wb") as err:
        proc = subprocess.run(cmd, stdout=out, stderr=err, cwd=str(workdir))

    state.setdefault("logs", []).append({
        "node": "retrosynthesis",
        "ts": _ts(),
        "cmd": " ".join(shlex.quote(c) for c in cmd),
        "rc": proc.returncode,
        "out_log": out_log.as_posix(),
        "err_log": err_log.as_posix(),
    })

    # Validate outputs so extractor won’t crash on empties
    def _nonempty(p: Path) -> bool:
        try:
            return p.exists() and p.stat().st_size > 0
        except Exception:
            return False

    must_have = [retro / "steps.csv", retro / "pathways.csv"]
    if proc.returncode != 0 or not all(_nonempty(p) for p in must_have):
        tail = ""
        try:
            if err_log.exists():
                lines = err_log.read_text().splitlines()
                tail = "\n" + "\n".join(lines[-50:])
        except Exception:
            pass
        raise RuntimeError(
            "Retrosynthesis produced no usable outputs; see logs.\n"
            + ("--- _rbc2_runner.err.log (tail) ---\n" + tail if tail else "")
        )

    return state

import subprocess, shutil

def ensure_equilibrator(env_prefix):
    mm = "/content/runs/run_001/bin/micromamba"
    pkgs = ["equilibrator-api==0.6.0", "equilibrator-cache==0.6.0"]
    subprocess.run(
        [mm, "run", "-p", env_prefix, "python", "-m", "pip", "install", *pkgs],
        check=False
    )

# --------------------------- compound_map_node -----------------------------

def _ensure_rdkit_outer():
    """Ensure rdkit is importable in the *notebook* interpreter (not micromamba)."""
    try:
        import rdkit  # noqa: F401
        return
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rdkit-pypi"])
        import rdkit  # noqa: F401

def _tokenize_rxn_smiles(rxn):
    if not isinstance(rxn, str) or ">>" not in rxn:
        return [], []
    L, R = rxn.split(">>", 1)
    subs  = [s.strip() for s in L.split(".") if s.strip()]
    prods = [p.strip() for p in R.split(".") if p.strip()]
    return subs, prods

def compound_map_node(state):
    """
    Build a minimal compound map from the reaction SMILES in steps_enzyme_plan.csv.
    Output: retro_finish_out/metabolite_map.csv with columns:
      token, smiles, inchikey
    Notes:
      - Runs in the outer Python (no micromamba required).
      - If RDKit is available, uses MolToInchiKey; otherwise writes empty inchikey.
    """
    import os, csv
    from pathlib import Path
    import pandas as pd

    def _log(msg):
        print(msg)
        state.setdefault("logs", []).append(msg)

    workdir = Path(state["workdir"])
    finish  = workdir / "retro_finish_out"
    retro   = workdir / "retro_out"
    finish.mkdir(parents=True, exist_ok=True)

    steps_csv = finish / "steps_enzyme_plan.csv"
    if not steps_csv.exists():
        # fall back to extractor output
        alt = retro / "steps.csv"
        if alt.exists():
            steps_csv = alt
        else:
            _log("compound_map: no steps table found")
            return state

    try:
        df = pd.read_csv(steps_csv)
    except Exception as e:
        _log("compound_map: failed to read steps CSV: %s: %s" % (type(e).__name__, e))
        return state

    # Minimal tokenizer for "L.SMILES...>>R.SMILES..."
    def _split_rxn(r):
        if not isinstance(r, str) or ">>" not in r:
            return [], []
        L, R = r.split(">>", 1)
        subs  = [s.strip() for s in L.split(".") if s.strip()]
        prods = [p.strip() for p in R.split(".") if p.strip()]
        return subs, prods

    tokens = set()
    for rxn in df.get("reaction_smiles", pd.Series([""] * len(df))).astype(str).fillna(""):
        L, R = _split_rxn(rxn)
        tokens.update(L); tokens.update(R)
    tokens = sorted(tokens)

    # Try RDKit
    have_rdkit = False
    try:
        from rdkit import Chem
        have_rdkit = True
    except Exception:
        pass

    rows = []
    for t in tokens:
        ik = ""
        if have_rdkit:
            try:
                m = Chem.MolFromSmiles(t)
                if m is not None:
                    ik = Chem.MolToInchiKey(m) or ""
            except Exception:
                ik = ""
        rows.append({"token": t, "smiles": t, "inchikey": ik})

    out_csv = finish / "metabolite_map.csv"
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["token", "smiles", "inchikey"])
        w.writeheader()
        for r in rows:
            w.writerow(r)

    filled = sum(1 for r in rows if r.get("inchikey"))
    _log("compound_map: wrote %s (inchikey filled %d / %d)" % (str(out_csv), filled, len(rows)))

    state.setdefault("artifacts", []).append(str(out_csv))
    return state


from urllib.parse import quote_plus
import pandas as pd
from pathlib import Path

def add_selenzyme_links_node(state):
    """
    Ensure a 'selenzyme_url' column exists and is string-typed, and fill any
    missing/invalid entries with a deterministic Selenzyme link built from
    reaction_smiles. Writes back to retro_finish_out/steps_enzyme_plan.csv.
    """
    import urllib.parse as _url
    import pandas as _pd
    from pathlib import Path as _Path
    import math as _math
    import numpy as _np

    def _log(msg):
        print(msg)
        state.setdefault("logs", []).append(msg)

    _log("▶ start add_selenzyme_links")

    workdir   = _Path(state["workdir"])
    finish    = workdir / "retro_finish_out"
    retro_out = workdir / "retro_out"
    finish.mkdir(parents=True, exist_ok=True)

    # Prefer the enriched table if it already exists
    steps_csv = finish / "steps_enzyme_plan.csv"
    if not steps_csv.exists():
        # Fallback to extractor output
        cand = retro_out / "steps.csv"
        if not cand.exists():
            _log("add_selenzyme_links: no steps file found; skipping")
            _log("✔ done add_selenzyme_links")
            return state
        steps_csv = cand

    df = _pd.read_csv(steps_csv)

    # Guarantee required columns
    for col in ["pathway_tag", "step_idx", "reaction_smiles"]:
        if col not in df.columns:
            df[col] = "" if col != "step_idx" else 0

    # Make sure the url column exists and is object/str dtype (not float)
    if "selenzyme_url" not in df.columns:
        df["selenzyme_url"] = ""
    # Normalize any non-string junk to empty string
    def _norm_to_str(x):
        # Treat NaN, None, empty list, and the literal string "[]"/"nan" as empty
        if x is None:
            return ""
        if isinstance(x, float) and _math.isnan(x):
            return ""
        if isinstance(x, list):
            return ""
        sx = str(x)
        if sx.strip() in ("", "[]", "nan", "NaN", "None"):
            return ""
        return sx

    df["selenzyme_url"] = df["selenzyme_url"].map(_norm_to_str).astype("string")

    # Build URLs only where empty AND reaction_smiles looks valid
    def mk_url(rxn: str) -> str:
        rxn = "" if rxn is None else str(rxn)
        if ">>" not in rxn:
            return ""
        return "https://selenzyme.org/selenzy?reaction=" + _url.quote(rxn, safe="")

    before_nonempty = df["selenzyme_url"].ne("") & df["selenzyme_url"].notna()
    needs_fill = ~before_nonempty & df["reaction_smiles"].astype(str).str.contains(">>", na=False)

    # Fill
    if needs_fill.any():
        df.loc[needs_fill, "selenzyme_url"] = df.loc[needs_fill, "reaction_smiles"].astype(str).map(mk_url).astype("string")

    # Count after fill: treat empty string as missing
    filled_count = (df["selenzyme_url"].notna() & df["selenzyme_url"].ne("")).sum()

    # Always write the enriched table to finish/
    out_csv = finish / "steps_enzyme_plan.csv"
    df.to_csv(out_csv, index=False)

    _log(f"selenzyme_links: filled {int(filled_count)} URLs out of {len(df)} rows → {out_csv}")
    _log("✔ done add_selenzyme_links")
    return state


# --- Micromamba bootstrap + robust _run_in_env (Python cell) ---
from pathlib import Path
import os, subprocess, sys, textwrap

MAMBA_ROOT = Path("/content/runs/run_001/micromamba")
MAMBA_BIN  = Path("/content/runs/run_001/bin/micromamba")
ENV_PREFIX = MAMBA_ROOT / "envs" / "retrobiocat"


def _ensure_cobra(env_prefix, state=None):
    """Install a cobra stack compatible with numpy>=1.26 / pandas>=2 inside the env (idempotent)."""
    rc, out, err = _run_in_env(env_prefix, ["python","-c","import cobra; print('cobra_ok')"])
    if rc == 0 and (out or "").strip().endswith("cobra_ok"):
        return
    rc, out, err = _run_in_env(env_prefix, ["python","-m","pip","install","-q","cobra==0.29.0","optlang>=1.8.0","swiglpk>=5.0.10"])
    if state is not None and rc != 0:
        state.setdefault("logs", []).append("simulate_gem: cobra install warning (continuing)")
        if err:
            state["logs"].append(err.strip().splitlines()[-1])


# =========================
# Wire the graph
# =========================
from functools import partial

# --- build the graph ---
graph = StateGraph(DBTLState)

# Design/analysis stack
graph.add_node("retrosynthesis", retrosynthesis_node)
graph.add_node("extract", extract_node)
graph.add_node("add_selenzyme_links", add_selenzyme_links_node)
graph.add_node("thermo", thermo_node)
graph.add_node("rank", rank_node)
graph.add_node("simulate_gem", simulate_gem_node)
graph.add_node("strain_optimize", strain_optimize_node)

# Enzyme/sequence stack
graph.add_node("selenzyme_rank", selenzyme_rank_node)
graph.add_node("sequence_fetch", lambda s: sequence_fetch_node(s, per_step=10, reviewed_first=False, timeout=12.0))
graph.add_node("sequence_rank", sequence_rank_node)

# Build + new Test/Learn
graph.add_node("build", build_node)
graph.add_node("test", test_node)
graph.add_node("learn", learn_node)

# Optional export (if you already have one)
graph.add_node("export", export_node)

# --- entry point ---
graph.set_entry_point("retrosynthesis")

# --- forward edges through Design ---
graph.add_edge("retrosynthesis", "extract")
graph.add_edge("extract", "add_selenzyme_links")
graph.add_edge("add_selenzyme_links", "thermo")
graph.add_edge("thermo", "rank")
graph.add_edge("rank", "simulate_gem")

# --- enzyme/sequence route (after simulate_gem) ---
graph.add_edge("simulate_gem", "selenzyme_rank")
# --- enzyme/sequence route (after simulate_gem) ---
graph.add_edge("simulate_gem", "selenzyme_rank")
graph.add_edge("selenzyme_rank", "sequence_fetch")
graph.add_edge("sequence_fetch", "sequence_rank")

# Gate strain_optimize on basic coverage; compute a quick report if missing
def _coverage_router(state):
    import pandas as pd, pathlib
    finish = pathlib.Path(state.get("workdir") or state.get("run_dir") or ".") / "retro_finish_out"
    rep_csv = finish / "sequence_coverage_report.csv"

    try:
        if rep_csv.exists():
            rep = pd.read_csv(rep_csv)
        else:
            # Minimal on-the-fly report
            steps = pd.read_csv(finish / "steps_enzyme_plan.csv")[["pathway_tag","step_idx"]].drop_duplicates()
            cands = pd.read_csv(finish / "steps_sequence_candidates.csv")[["pathway_tag","step_idx"]].drop_duplicates()
            picks = pd.read_csv(finish / "steps_sequence_plan.csv")[["pathway_tag","step_idx"]].drop_duplicates()

            n_cands = (cands.groupby(["pathway_tag","step_idx"])
                            .size().reset_index(name="n_candidates"))
            rep = (steps.merge(n_cands, on=["pathway_tag","step_idx"], how="left")
                         .fillna({"n_candidates": 0}))
            picked_pairs = set(zip(picks.pathway_tag, picks.step_idx))
            rep["picked"] = [(pt, si) in picked_pairs for pt, si in zip(rep.pathway_tag, rep.step_idx)]

            def _reason(row):
                if row["n_candidates"] == 0: return "no_candidates_found"
                if not row["picked"]:       return "not_selected_in_rank"
                return "ok"
            rep["reason"] = rep.apply(_reason, axis=1)
            rep.sort_values(["pathway_tag","step_idx"]).to_csv(rep_csv, index=False)

        ok = (rep["n_candidates"] > 0).mean() >= 0.80
    except Exception:
        ok = False

    return "ok" if ok else "retry"

# Route: if coverage OK → strain_optimize → build; else retry enzyme search
graph.add_conditional_edges(
    "sequence_rank",
    _coverage_router,
    {"ok": "strain_optimize", "retry": "selenzyme_rank"}
)

# After strain optimization, proceed to build → test → learn
graph.add_edge("strain_optimize", "build")
graph.add_edge("build", "test")
graph.add_edge("test", "learn")

# Learn routes: loop back or finish/export
def _learn_router(state):
    return "loop" if state.get("signals", {}).get("loop_back") else "finish"

graph.add_conditional_edges(
    "learn",
    _learn_router,
    {"loop": "selenzyme_rank", "finish": "export"}
)

graph.add_edge("export", END)


# =========================
# Example run
# =========================
# init_state = DBTLState(
#     run_id="run_001",
#    workdir="/content/runs/run_001",
#     target_smiles="COC1=C(C=CC(=C1)C=O)O",
#     host="Escherichia coli",
#     constraints={"max_steps":"5"},
#     logs=[]
# )
# for state in app.stream(init_state, stream_mode="values"):
#     node   = state.get("last_node", "?")
#     status = state.get("status", "?")
#     print(f"[{node}] {status}")
# print("Final logs:")
# print("\n".join(state.get("logs", [])))


In [ ]:
# --- hotfix: place micromamba at /content/bin/micromamba and set root prefix ---
import io, os, stat, tarfile, urllib.request, pathlib, subprocess

BIN_DST = pathlib.Path("/content/bin/micromamba")
ROOT    = pathlib.Path("/content/runs/run_001/micromamba")
ENV     = ROOT / "envs" / "retrobiocat"

# 1) put micromamba exactly where older helpers expect it
if not BIN_DST.exists():
    BIN_DST.parent.mkdir(parents=True, exist_ok=True)
    url = "https://micro.mamba.pm/api/micromamba/linux-64/latest"
    print("↓ downloading micromamba…", url)
    with urllib.request.urlopen(url) as r:
        data = r.read()
    print("→ extracting micromamba (auto-detect compression)…")
    with tarfile.open(fileobj=io.BytesIO(data), mode="r:*") as tf:
        member = None
        for m in tf.getmembers():
            name = m.name.replace("\\", "/")
            if name.endswith("/micromamba") and "/bin/" in name:
                member = m
                break
        if member is None:
            for m in tf.getmembers():
                if m.name.split("/")[-1] == "micromamba":
                    member = m
                    break
        if member is None:
            raise RuntimeError("micromamba binary not found in archive")
        with tf.extractfile(member) as src, open(BIN_DST, "wb") as dst:
            dst.write(src.read())
    BIN_DST.chmod(BIN_DST.stat().st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)
print("✅ micromamba:", BIN_DST, "exists?", BIN_DST.exists())

# 2) use the same root prefix your pipeline expects
os.environ["MAMBA_ROOT_PREFIX"] = str(ROOT)

# 3) ensure the env exists (fast if already there)
if not ENV.exists():
    subprocess.check_call([str(BIN_DST), "create", "-y", "-p", str(ENV), "-c", "conda-forge", "python=3.10", "pip"])

# 4) very light sanity ping
subprocess.check_call([str(BIN_DST), "run", "-p", str(ENV), "python", "-c", "import sys; print('python in env:', sys.executable)"])
print("✅ ready. Re-run your execution cell.")



↓ downloading micromamba… https://micro.mamba.pm/api/micromamba/linux-64/latest
→ extracting micromamba (auto-detect compression)…
✅ micromamba: /content/bin/micromamba exists? True
✅ ready. Re-run your execution cell.


In [ ]:
# --- micromamba path shim (idempotent) ---
import io, os, stat, tarfile, urllib.request, shutil
from pathlib import Path

expected = Path("/content/runs/run_001/micromamba/bin/micromamba")
candidates = [
    Path("/content/runs/run_001/bin/micromamba"),
    Path("/content/bin/micromamba"),
]

def ensure_binary(p: Path):
    if p.exists():
        return p
    # download micromamba and place it here
    p.parent.mkdir(parents=True, exist_ok=True)
    url = "https://micro.mamba.pm/api/micromamba/linux-64/latest"
    print("↓ downloading micromamba…", url)
    with urllib.request.urlopen(url) as r:
        data = r.read()
    print("→ extracting micromamba (auto-detect compression)…")
    with tarfile.open(fileobj=io.BytesIO(data), mode="r:*") as tf:
        member = None
        for m in tf.getmembers():
            name = m.name.replace("\\", "/")
            if name.endswith("/micromamba") and "/bin/" in name:
                member = m
                break
        if member is None:
            for m in tf.getmembers():
                if m.name.split("/")[-1] == "micromamba":
                    member = m
                    break
        if member is None:
            raise RuntimeError("micromamba binary not found in archive")
        with tf.extractfile(member) as src, open(p, "wb") as dst:
            dst.write(src.read())
    p.chmod(p.stat().st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)
    return p

# 1) find or fetch a working micromamba
src = None
for c in candidates:
    if c.exists():
        src = c
        break
if src is None:
    # put a copy in /content/runs/run_001/bin for convenience
    src = ensure_binary(Path("/content/runs/run_001/bin/micromamba"))

# 2) mirror it into the "expected" legacy location
if not expected.exists():
    expected.parent.mkdir(parents=True, exist_ok=True)
    try:
        expected.symlink_to(src)
        print(f"→ linked {expected} -> {src}")
    except Exception:
        shutil.copy2(src, expected)
        expected.chmod(expected.stat().st_mode | 0o111)
        print(f"→ copied {src} -> {expected}")

# 3) make sure root prefix is set so micromamba uses the same tree
os.environ["MAMBA_ROOT_PREFIX"] = "/content/runs/run_001/micromamba"

print("✅ micromamba shim ready at:", expected)
print("   also available at:", src, "(exists:", src.exists(), ")")


→ linked /content/runs/run_001/micromamba/bin/micromamba -> /content/runs/run_001/bin/micromamba
✅ micromamba shim ready at: /content/runs/run_001/micromamba/bin/micromamba
   also available at: /content/runs/run_001/bin/micromamba (exists: True )


In [ ]:
def strain_optimize_node(state):
    import subprocess, json, pathlib, os, textwrap

    def _log(msg):
        print(msg)
        state.setdefault("logs", []).append(msg)

    _log("▶ start strain_optimize")

    env_path = "/content/runs/run_001/micromamba/envs/retrobiocat"
    workdir  = pathlib.Path(state.get("workdir") or ".")
    finish   = workdir / "retro_finish_out"
    finish.mkdir(parents=True, exist_ok=True)

    cfg       = (state.get("strain") or {}).copy()
    model_path = state.get("sbml_model_path") or ""

    # sane defaults
    cfg.setdefault("solver", "glpk")
    cfg.setdefault("time_limit_s", 300)
    cfg.setdefault("fva", False)
    cfg.setdefault("search", "auto")
    cfg.setdefault("max_kos", 2)
    cfg.setdefault("min_growth", 0.0)
    cfg.setdefault("objective_id", None)  # biomass rxn id
    cfg.setdefault("product_rxn", None)   # desired product rxn id

    # heuristic candidate selection tweaks
    cfg.setdefault("ko_filters", {
        "skip_exchange": True,
        "skip_transport": True,
        "protect_essentials": True,   # drop KOs that kill growth below min_growth
        "prefer_subsystems": [        # boosts selection priority
            "Glycolysis/Gluconeogenesis","TCA cycle","Pentose Phosphate Pathway",
            "Pyruvate Metabolism","Anaplerotic reactions"
        ],
        "max_candidates": 100
    })
    # optional triples sweep
    cfg.setdefault("triples", {
        "enable": False,
        "top_k": 30,       # restrict to top-k candidates before triples
        "patience": 50     # stop if no improvement after N combos
    })

    # Build the inner script WITHOUT any .format/f-string interpolation.
    py = textwrap.dedent("""
    import os, json, itertools, copy
    import pandas as pd
    from pathlib import Path

    OUT_DIR    = Path(os.environ["FINISH"])
    MODEL_PATH = os.environ["MODEL_PATH"]
    CFG        = json.loads(os.environ["CFG_JSON"])
    SOLVER     = os.environ.get("SOLVER") or "glpk"
    TIME_LIMIT = float(os.environ.get("TIME_LIMIT") or 0)

    summary_json = OUT_DIR / "strain_opt_summary.json"
    flux_csv     = OUT_DIR / "strain_opt_fluxes.csv"
    fva_csv      = OUT_DIR / "strain_opt_fva.csv"
    ko_csv       = OUT_DIR / "strain_opt_ko_results.csv"
    ok_csv       = OUT_DIR / "strain_opt_optknock.csv"

    def set_solver(model, solver_name, time_limit):
        try:
            model.solver = solver_name
        except Exception:
            pass
        try:
            if time_limit:
                # try common optlang handles
                try:
                    model.solver.configuration.timeout = time_limit
                except Exception:
                    try:
                        model.solver.problem.parameters.timelimit = time_limit
                    except Exception:
                        pass
        except Exception:
            pass

    def apply_medium(model, medium_map):
        for rxn_id, lb in (medium_map or {}).items():
            if rxn_id in model.reactions:
                try:
                    model.reactions.get_by_id(rxn_id).lower_bound = float(lb)
                except Exception:
                    pass

    def apply_knockouts(model, ko_list):
        for ko in (ko_list or []):
            if ko in model.reactions:
                model.reactions.get_by_id(ko).knock_out()
            elif ko in model.genes:
                model.genes.get_by_id(ko).knock_out()

    def run_fba(model, objective_id=None):
        if objective_id and objective_id in model.reactions:
            model.objective = objective_id
        sol = model.optimize()
        status = getattr(sol, "status", "unknown")
        obj    = float(getattr(sol, "objective_value", 0.0) or 0.0)
        fluxes = None
        try:
            fs = sol.fluxes
            fluxes = pd.DataFrame({"reaction_id": fs.index, "flux": fs.values})
        except Exception:
            pass
        return status, obj, fluxes

    def run_fva(model, reaction_ids=None, fraction_of_optimum=0.9):
        from cobra.flux_analysis import flux_variability_analysis
        try:
            df = flux_variability_analysis(
                model,
                reaction_list=[model.reactions.get_by_id(i) for i in reaction_ids] if reaction_ids else None,
                fraction_of_optimum=fraction_of_optimum
            )
            return df.reset_index().rename(columns={"index":"reaction_id"})
        except Exception:
            return None

    def product_flux(flux_df, product_rxn):
        if flux_df is None or not product_rxn:
            return None
        try:
            row = flux_df.loc[flux_df["reaction_id"] == product_rxn]
            return float(row["flux"].iloc[0]) if len(row) else None
        except Exception:
            return None

    def constrain_growth(model, biomass_id, min_growth):
        if biomass_id and biomass_id in model.reactions and min_growth is not None:
            try:
                model.reactions.get_by_id(biomass_id).lower_bound = float(min_growth)
            except Exception:
                pass

    def annotate_reaction(r):
        rid = r.id
        name = getattr(r, "name", "") or ""
        subsystem = getattr(r, "subsystem", "") or ""
        is_exchange = bool(r.boundary) or rid.startswith("EX_") or "exchange" in name.lower()
        is_transport = ("transport" in name.lower()) or ("transport" in subsystem.lower()) or rid.startswith("TCE") or rid.startswith("TRANS")
        return rid, subsystem, is_exchange, is_transport

    def choose_ko_candidates(model, flux_df, filters):
        # start from reactions with highest |flux|
        if flux_df is None or not len(flux_df):
            rxns = [r.id for r in model.reactions]
            df = pd.DataFrame({"reaction_id": rxns, "abs_flux":[0.0]*len(rxns)})
        else:
            df = flux_df.copy()
            df["abs_flux"] = df["flux"].abs()

        # annotate and filter
        ann = [annotate_reaction(model.reactions.get_by_id(rid)) for rid in df["reaction_id"] if rid in model.reactions]
        ann_df = pd.DataFrame(ann, columns=["reaction_id","subsystem","is_exchange","is_transport"])
        df = df.merge(ann_df, on="reaction_id", how="left")

        if filters.get("skip_exchange", True):
            df = df[df["is_exchange"] != True]
        if filters.get("skip_transport", True):
            df = df[df["is_transport"] != True]

        # boost preferred subsystems
        prefs = set(filters.get("prefer_subsystems", []) or [])
        df["score"] = df["abs_flux"] + df["subsystem"].apply(lambda s: 10.0 if s in prefs else 0.0)

        # keep top N
        maxN = int(filters.get("max_candidates", 100) or 100)
        df = df.sort_values("score", ascending=False).head(maxN).reset_index(drop=True)
        return df["reaction_id"].tolist()

    def drop_essential_kos(model, biomass_id, min_growth, cand_ids):
        safe = []
        for rid in cand_ids:
            m = copy.deepcopy(model)
            apply_knockouts(m, [rid])
            status, obj, fl = run_fba(m, objective_id=biomass_id)
            # keep if growth >= min_growth
            if obj is not None and obj >= float(min_growth or 0.0):
                safe.append(rid)
        return safe

    def heuristic_ko_search(model, product_rxn, biomass_id, min_growth, max_kos, ko_ids, triples_cfg):
        results = []

        # baseline
        m0 = copy.deepcopy(model)
        constrain_growth(m0, biomass_id, min_growth)
        st, obj, fl = run_fba(m0, objective_id=product_rxn or biomass_id)
        results.append({"kos": [], "status": st, "objective": obj, "product_flux": product_flux(fl, product_rxn)})

        # singles
        for a in ko_ids:
            m = copy.deepcopy(model)
            apply_knockouts(m, [a])
            constrain_growth(m, biomass_id, min_growth)
            st, obj, fl = run_fba(m, objective_id=product_rxn or biomass_id)
            results.append({"kos": [a], "status": st, "objective": obj, "product_flux": product_flux(fl, product_rxn)})

        # doubles
        if int(max_kos or 0) >= 2:
            for a, b in itertools.combinations(ko_ids, 2):
                m = copy.deepcopy(model)
                apply_knockouts(m, [a, b])
                constrain_growth(m, biomass_id, min_growth)
                st, obj, fl = run_fba(m, objective_id=product_rxn or biomass_id)
                results.append({"kos": [a, b], "status": st, "objective": obj, "product_flux": product_flux(fl, product_rxn)})

        # triples (optional)
        if triples_cfg.get("enable") and int(max_kos or 0) >= 3:
            top_k = int(triples_cfg.get("top_k", 30) or 30)
            patience = int(triples_cfg.get("patience", 50) or 50)
            # reduce search space to top_k by best single/double result
            df_tmp = pd.DataFrame(results).dropna(subset=["product_flux"])
            ranked = df_tmp.explode("kos")
            ranked = ranked[ranked["kos"].notna()]
            top = ranked.groupby("kos")["product_flux"].max().sort_values(ascending=False).head(top_k).index.tolist()
            best_pf = df_tmp["product_flux"].max() if len(df_tmp) else -1e9
            no_improve = 0
            for a, b, c in itertools.combinations(top, 3):
                if no_improve >= patience:
                    break
                m = copy.deepcopy(model)
                apply_knockouts(m, [a, b, c])
                constrain_growth(m, biomass_id, min_growth)
                st, obj, fl = run_fba(m, objective_id=product_rxn or biomass_id)
                pf = product_flux(fl, product_rxn)
                results.append({"kos": [a, b, c], "status": st, "objective": obj, "product_flux": pf})
                if pf is not None and pf > best_pf:
                    best_pf = pf
                    no_improve = 0
                else:
                    no_improve += 1

        return pd.DataFrame(results)

    def try_optknock(model, product_rxn, biomass_id, max_kos):
        try:
            import cameo
            from cameo.strain_design.deterministic.linear_programming import OptKnock
            from cameo.strain_design.heuristic.evolutionary.optimization import OptGen
        except Exception:
            return None

        designs = []

        # OptKnock
        try:
            ok = OptKnock(model, fraction_of_optimum=0.1, max_knockouts=int(max_kos or 2))
            res = ok.run(max_knockouts=int(max_kos or 2))
            if hasattr(res, "data_frame"):
                for row in res.data_frame.itertuples(index=False):
                    ks = list(getattr(row, "knockouts", [])) or list(getattr(row, "design", [])) or []
                    fit = getattr(row, "objective_value", None) or getattr(row, "fitness", None)
                    designs.append({"method":"optknock", "design": ks, "fitness": fit})
        except Exception:
            pass

        # OptGen
        try:
            if product_rxn and product_rxn in model.reactions:
                model.objective = product_rxn
            og = OptGen(model=model, size=int(max_kos or 2), max_evaluations=200)
            res = og.run(max_evaluations=200)
            if hasattr(res, "data_frame"):
                for row in res.data_frame.itertuples(index=False):
                    ks = list(getattr(row, "knockouts", [])) or list(getattr(row, "design", [])) or []
                    fit = getattr(row, "objective_value", None) or getattr(row, "fitness", None)
                    designs.append({"method":"optgen", "design": ks, "fitness": fit})
        except Exception:
            pass

        return pd.DataFrame(designs) if designs else None

    # -------- main --------
    try:
        from cobra.io import read_sbml_model
        import cobra
    except Exception as e:
        with open(summary_json, "w") as f:
            json.dump({"error": f"cobra import failed: {type(e).__name__}: {e}"}, f, indent=2)
        print("strain_optimize: cobra import failed")
        raise

    model = read_sbml_model(MODEL_PATH)
    set_solver(model, SOLVER, TIME_LIMIT)

    apply_medium(model, CFG.get("medium", {}))

    biomass_id = CFG.get("objective_id")
    product_rxn = CFG.get("product_rxn")
    min_growth = float(CFG.get("min_growth", 0.0) or 0.0)
    max_kos    = int(CFG.get("max_kos", 2) or 2)

    # baseline FBA (biomass objective)
    base_status, base_obj, base_flux = run_fba(model.copy(), objective_id=biomass_id)
    if base_flux is not None:
        base_flux.to_csv(flux_csv, index=False)
    base_prod = product_flux(base_flux, product_rxn)

    # FVA (optional)
    fva_df = None
    if bool(CFG.get("fva", False)):
        rxn_ids = [r.id for r in model.reactions][:500]
        fva_df = run_fva(model.copy(), reaction_ids=rxn_ids, fraction_of_optimum=0.9)
        if fva_df is not None:
            fva_df.to_csv(fva_csv, index=False)

    # search strategy
    search = (CFG.get("search") or "auto").lower()
    if search not in ("auto","heuristic","optknock"):
        search = "auto"

    # seed candidate list (by flux)
    _, _, seed_flux = run_fba(model.copy(), objective_id=biomass_id)
    ko_ids = choose_ko_candidates(model, seed_flux, CFG.get("ko_filters", {}))

    # protect essentials if requested
    if (CFG.get("ko_filters", {}).get("protect_essentials", True)) and len(ko_ids):
        ko_ids = drop_essential_kos(model, biomass_id, min_growth, ko_ids)

    ko_df = None
    ok_df = None

    if search in ("auto","optknock"):
        ok_df = try_optknock(model.copy(), product_rxn, biomass_id, max_kos)
        if ok_df is not None and len(ok_df):
            ok_df.to_csv(ok_csv, index=False)

    if (ok_df is None or not len(ok_df)) and search in ("auto","heuristic"):
        ko_df = heuristic_ko_search(
            model.copy(), product_rxn, biomass_id, min_growth, max_kos, ko_ids,
            CFG.get("triples", {})
        )
        if ko_df is not None and len(ko_df):
            ko_df.to_csv(ko_csv, index=False)

    # pick best by product flux
    best = {"method":"none","kos":[], "product_flux": base_prod, "status": base_status}
    def consider(df, method):
        nonlocal best
        if df is None or not len(df):
            return
        d = df.dropna(subset=["product_flux"])
        if not len(d):
            return
        i = d["product_flux"].astype(float).idxmax()
        row = d.loc[i]
        cand = {"method": method, "kos": list(row["kos"]) if "kos" in row else list(row.get("design", [])),
                "product_flux": float(row["product_flux"]), "status": str(row.get("status",""))}
        if best["product_flux"] is None or cand["product_flux"] > (best["product_flux"] or -1e9):
            best = cand

    consider(ko_df, "heuristic")
    # map cameo frame if needed
    if ok_df is not None and len(ok_df):
        tmp = ok_df.copy()
        if "design" not in tmp.columns and "knockouts" in tmp.columns:
            tmp["design"] = tmp["knockouts"]
        if "fitness" in tmp.columns and "product_flux" not in tmp.columns:
            tmp["product_flux"] = tmp["fitness"]
        if "design" in tmp.columns and "product_flux" in tmp.columns:
            tmp = tmp.rename(columns={"design":"kos"})
            consider(tmp, "optknock/optgen")

    summary = {
        "solver": SOLVER,
        "time_limit_s": TIME_LIMIT,
        "biomass_id": biomass_id,
        "product_rxn": product_rxn,
        "min_growth": min_growth,
        "base": {"status": base_status, "objective": base_obj, "product_flux": base_prod},
        "selected_design": best,
        "files": {
            "fluxes_csv": str(flux_csv),
            "fva_csv": str(fva_csv) if fva_df is not None else None,
            "ko_results_csv": str(ko_csv) if (ko_df is not None and len(ko_df)) else None,
            "optknock_csv": str(ok_csv) if (ok_df is not None and len(ok_df)) else None
        }
    }
    with open(summary_json, "w") as f:
        json.dump(summary, f, indent=2)

    print("strain_optimize:", json.dumps({"selected": summary["selected_design"], "base_prod": base_prod}))
    """)

    # run inside micromamba env with env vars carrying parameters
    env = os.environ.copy()
    env.update({
        "FINISH": str(finish),
        "MODEL_PATH": str(model_path),
        "CFG_JSON": json.dumps(cfg),
        "SOLVER": str(cfg.get("solver")),
        "TIME_LIMIT": str(cfg.get("time_limit_s"))
    })

    cmd = ["/content/runs/run_001/bin/micromamba", "run", "-p", env_path, "python", "-c", py]
    r = subprocess.run(cmd, capture_output=True, text=True, env=env)
    if r.stdout.strip():
        _log(r.stdout.strip())
    if r.returncode != 0:
        _log(f"strain_optimize: ERROR → {r.stderr.strip()}")
    else:
        state["strain_optimize_summary"]   = str(finish / "strain_opt_summary.json")
        state["strain_optimize_fluxes"]    = str(finish / "strain_opt_fluxes.csv")
        state["strain_optimize_fva"]       = str(finish / "strain_opt_fva.csv")
        state["strain_optimize_kos"]       = str(finish / "strain_opt_ko_results.csv")
        state["strain_optimize_optknock"]  = str(finish / "strain_opt_optknock.csv")
        _log("✔ done strain_optimize")
    return state



In [ ]:
! /content/runs/run_001/bin/micromamba run -p /content/runs/run_001/micromamba/envs/retrobiocat \
  python -m equilibrator_cache.cli download --to /content/runs/run_001/equilibrator_cache


/content/runs/run_001/micromamba/envs/retrobiocat/bin/python: No module named equilibrator_cache.cli


In [ ]:
# === Execution cell ===

# sanity checks (unchanged)
import os, urllib.request, pathlib

# 1) Prepare model if needed
model_path = pathlib.Path("/content/models/iML1515.xml")
model_path.parent.mkdir(parents=True, exist_ok=True)
if not model_path.exists():
    urllib.request.urlretrieve(
        "https://github.com/opencobra/COBRA.tutorials/raw/main/src/data/iML1515.xml",
        str(model_path)
    )
print("✅ SBML model ready:", model_path)

# 2) Initialize state
state = {
    "run_id": "run_001",
    "workdir": "/content/runs/run_001",
    "target_smiles": "COC1=C(C=CC(=C1)C=O)O",
    "host": "Escherichia coli",
    "signals": {"human_approved": True},
    "approved": True,  # human gate default
    "constraints": {
        "max_steps": 5,
        "net_access": True,
        "uniprot": {"timeout_s": 10, "max_hits_per_query": 3},
        "expanders": ["RetroBioCat", "EnzymeMap", "BKMS", "RetroRules"],
        "max_search_time_s": 15,
    },
    "thermo_params": {
        "pH": 7.5,
        "ionic_strength": 0.25,
        "pMg": 3.0,
        "temperature_K": 298.15,
        "pass_margin_kJ": 0.0,
    },
    "sbml_model_path": str(model_path),
    "metabolite_map": {},
    "simulate": {"mu_min": 0.15, "v_prod_min": 1e-4, "product_id": None},
    "logs": [],
}

# 3) Run sequentially (with simple guards around sequence stage)
state["logs"].clear()

# upstream stages
state = retrosynthesis_node(state)
state = extract_node(state)
state = add_selenzyme_links_node(state)
state = selenzyme_rank_node(state)
state = compound_map_node(state)
state = thermo_node(state)
state = rank_node(state)
state = simulate_gem_node(state)
state = selenzyme_rank_node(state) or state                           # (optional if you already run it)
state = sequence_fetch_node(state, per_step=10, reviewed_first=False, timeout=12.0) or state
state = sequence_rank_node(state) or state                             # picks 1 per step
state = build_node(state) or state                                  # FASTA + cloning plan
state = test_node(state) or state
state = learn_node(state) or state

# --- sequence coverage summary logger ---
# --- sequence coverage summary logger (robust) ---
import pandas as pd
from pathlib import Path

finish = Path("/content/runs/run_001/retro_finish_out")

def _safe_read(path, cols=None):
    try:
        df = pd.read_csv(path)
        if cols:  # keep only needed cols if asked
            for c in cols:
                if c not in df.columns:
                    df[c] = []
            df = df[cols]
        return df
    except Exception:
        return pd.DataFrame(columns=cols or [])

steps = _safe_read(finish / "steps_enzyme_plan.csv", cols=["pathway_tag","step_idx"])
cands = _safe_read(finish / "steps_sequence_candidates.csv", cols=["pathway_tag","step_idx"])
picks = _safe_read(finish / "steps_sequence_plan.csv", cols=["pathway_tag","step_idx"])

# dtype normalize
for df in (steps, cands, picks):
    df["pathway_tag"] = df["pathway_tag"].astype(str)
    df["step_idx"] = pd.to_numeric(df["step_idx"], errors="coerce").fillna(0).astype(int)

steps = steps.drop_duplicates().assign(source="steps")
cands = cands.drop_duplicates().assign(source="candidates")
picks = picks.drop_duplicates().assign(source="picks")

# aggregate counts
n_cands = (
    cands.groupby(["pathway_tag","step_idx"])
    .size()
    .reset_index(name="n_candidates")
) if len(cands) else pd.DataFrame(columns=["pathway_tag","step_idx","n_candidates"])

# merge with master step list
rep = steps.merge(n_cands, on=["pathway_tag","step_idx"], how="left").fillna({"n_candidates": 0})

pick_set = set(zip(picks.pathway_tag, picks.step_idx)) if len(picks) else set()
rep["picked"] = [(pt, si) in pick_set for pt, si in zip(rep.pathway_tag, rep.step_idx)]

def _reason(row):
    if int(row["n_candidates"]) == 0:
        return "no_candidates_found"
    if not bool(row["picked"]):
        return "not_selected_in_rank"
    return "ok"

rep["reason"] = rep.apply(_reason, axis=1)
rep = rep.sort_values(["pathway_tag", "step_idx"]).reset_index(drop=True)

out_csv = finish / "sequence_coverage_report.csv"
rep.to_csv(out_csv, index=False)
print(f"✅ wrote coverage summary → {out_csv}")
print(rep.groupby("reason").size() if len(rep) else "no rows")
print(rep.head(15))


# (optional) human approval gate before export
state = human_gate_node(state)
state = export_node(state)

# 4) Stream logs
for l in state["logs"]:
    print(l)

✅ SBML model ready: /content/models/iML1515.xml
▶ start extract
extracted pathways from pathways_solved_steps.csv (82 step rows, 23 pathways)
✔ done extract
▶ start add_selenzyme_links
selenzyme_links: filled 82 URLs out of 82 rows → /content/runs/run_001/retro_finish_out/steps_enzyme_plan.csv
✔ done add_selenzyme_links
compound_map: wrote /content/runs/run_001/retro_finish_out/metabolite_map.csv (inchikey filled 0 / 25)
▶ start thermo
thermo: token IDs available for 0 / 23 tokens
thermo: id routes: {'via_explicit_id': 0, 'via_inchikey_literal': 0, 'via_rdkit_inchikey': 0, 'via_rdkit_inchikey_only': 23, 'via_pubchem_kegg': 0, 'via_pubchem_smiles': 0, 'via_pubchem_cas': 0, 'via_match_smiles': 0, 'via_name': 0, 'unmapped': 23}
thermo: NOTE – RDKit produced InChIKeys but none resolved to KEGG via PubChem.
thermo: example tokens with IK but no KEGG (first 5): ['CNC(=O)C(=O)c1ccc(O)c(OC)c1', 'CNC(=O)C(O)c1ccc(O)c(OC)c1', 'COC1=C(C=CC(=C1)C=O)O', 'COc1cc(C(=O)C#N)ccc1O', 'COc1cc(C(=O)C(=O)C(

In [ ]:
!/content/runs/run_001/bin/micromamba install -y -p /content/runs/run_001/micromamba/envs/retrobiocat -c conda-forge cobra


[+] 0.0s
[+] 0.1s
[+] 0.2s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.3s
conda-forge/linux-64   1%
conda-forge/noarch     1%[+] 0.4s
conda-forge/linux-64   2%
conda-forge/noarch     3%[+] 0.5s
conda-forge/linux-64   4%
conda-forge/noarch     5%[+] 0.6s
conda-forge/linux-64   4%
conda-forge/noarch     7%[+] 0.7s
conda-forge/linux-64   6%
conda-forge/noarch    10%[+] 0.8s
conda-forge/linux-64   7%
conda-forge/noarch    13%[+] 0.9s
conda-forge/linux-64  10%
conda-forge/noarch    18%[+] 1.0s
conda-forge/linux-64  10%
conda-forge/noarch    20%[+] 1.1s
conda-forge/linux-64  12%
conda-forge/noarch    24%[+] 1.2s
conda-forge/linux-64  14%
conda-forge/noarch    27%[+] 1.3s
conda-forge/linux-64  16%
conda-forge/noarch    30%[+] 1.4s
conda-forge/linux-64  16%
conda-forge/noarch    32%[+] 1.5s
conda-forge/linux-64  17%
conda-forge/noarch    34%[+] 1.6s
conda-forge/linux-64  19%
conda-forge/noarch    38%[+] 1.7s
conda-forge/linux-64  20%
conda-forge/noarch    39%[+] 1.8s
conda-forge/l

In [ ]:
! /content/runs/run_001/bin/micromamba run -p /content/runs/run_001/micromamba/envs/retrobiocat \
python -c "import sys; print('python:', sys.executable); import importlib.util as u; print('cobra found?', u.find_spec('cobra') is not None)"


python: /content/runs/run_001/micromamba/envs/retrobiocat/bin/python
cobra found? True


In [ ]:
# === Execution cell ===
import os, urllib.request, pathlib, pandas as pd

# 1) Prepare model if needed
model_path = pathlib.Path("/content/models/iML1515.xml")
model_path.parent.mkdir(parents=True, exist_ok=True)
if not model_path.exists():
    urllib.request.urlretrieve(
        "https://github.com/opencobra/COBRA.tutorials/raw/main/src/data/iML1515.xml",
        str(model_path)
    )
print("✅ SBML model ready:", model_path)

# 2) Initialize state
workdir = pathlib.Path("/content/runs/run_001")
(workdir / "retro_finish_out").mkdir(parents=True, exist_ok=True)

state = {
    "run_id": "run_001",
    "workdir": str(workdir),
    "target_smiles": "COC1=C(C=CC(=C1)C=O)O",
    "host": "Escherichia coli",
    "signals": {"human_approved": True},
    "approved": True,  # human gate default
    "constraints": {
        "max_steps": 5,
        "net_access": True,
        "uniprot": {"timeout_s": 10, "max_hits_per_query": 3},
        "expanders": ["RetroBioCat", "EnzymeMap", "BKMS", "RetroRules"],
        "max_search_time_s": 15,
    },
    "thermo_params": {
        "pH": 7.5,
        "ionic_strength": 0.25,
        "pMg": 3.0,
        "temperature_K": 298.15,
        "pass_margin_kJ": 0.0,
    },
    "sbml_model_path": str(model_path),
    "metabolite_map": {},
    "simulate": {"mu_min": 0.15, "v_prod_min": 1e-4, "product_id": None},
    "logs": [],
}

# 3) Run sequentially
state["logs"].clear()

# Upstream design / analysis
state = retrosynthesis_node(state)
state = extract_node(state)
state = add_selenzyme_links_node(state)
state = selenzyme_rank_node(state) or state        # writes steps_enzyme_choice.csv + seed_uniprots
state = compound_map_node(state)
state = thermo_node(state)
state = rank_node(state)
state = simulate_gem_node(state)

# Enzyme/sequence stack
# (A second selenzyme_rank pass is OK but not required; keep one to be fast)
# state = selenzyme_rank_node(state) or state
state = sequence_fetch_node(state, per_step=10, reviewed_first=False, timeout=12.0) or state
state = sequence_rank_node(state) or state  # writes steps_sequence_plan.csv

# --- sequence coverage summary logger (robust) ---
from pathlib import Path
finish = Path(state.get("workdir") or ".") / "retro_finish_out"

def _safe_read(path, cols=None):
    try:
        df = pd.read_csv(path)
        if cols:
            for c in cols:
                if c not in df.columns:
                    df[c] = []
            df = df[cols]
        return df
    except Exception:
        return pd.DataFrame(columns=cols or [])

steps = _safe_read(finish / "steps_enzyme_plan.csv", cols=["pathway_tag","step_idx"])
cands = _safe_read(finish / "steps_sequence_candidates.csv", cols=["pathway_tag","step_idx"])
picks = _safe_read(finish / "steps_sequence_plan.csv", cols=["pathway_tag","step_idx"])

for df in (steps, cands, picks):
    if len(df):
        df["pathway_tag"] = df["pathway_tag"].astype(str)
        df["step_idx"] = pd.to_numeric(df["step_idx"], errors="coerce").fillna(0).astype(int)

steps = steps.drop_duplicates().assign(source="steps")
cands = cands.drop_duplicates().assign(source="candidates")
picks = picks.drop_duplicates().assign(source="picks")

n_cands = (
    cands.groupby(["pathway_tag","step_idx"]).size().reset_index(name="n_candidates")
) if len(cands) else pd.DataFrame(columns=["pathway_tag","step_idx","n_candidates"])

rep = steps.merge(n_cands, on=["pathway_tag","step_idx"], how="left").fillna({"n_candidates": 0})
pick_set = set(zip(picks.pathway_tag, picks.step_idx)) if len(picks) else set()
rep["picked"] = [(pt, si) in pick_set for pt, si in zip(rep.pathway_tag, rep.step_idx)]

def _reason(row):
    if int(row["n_candidates"]) == 0:
        return "no_candidates_found"
    if not bool(row["picked"]):
        return "not_selected_in_rank"
    return "ok"

rep["reason"] = rep.apply(_reason, axis=1)
rep = rep.sort_values(["pathway_tag", "step_idx"]).reset_index(drop=True)
out_csv = finish / "sequence_coverage_report.csv"
rep.to_csv(out_csv, index=False)
print(f"✅ wrote coverage summary → {out_csv}")
print(rep.groupby("reason").size() if len(rep) else "no rows")
print(rep.head(15))

# (Optional) simple coverage gate for sequential run
coverage_ok = (rep["n_candidates"] > 0).mean() >= 0.80 if len(rep) else False
if not coverage_ok:
    print("⚠️ coverage < 80% → quick retry with relaxed fetch")
    state = sequence_fetch_node(state, per_step=15, reviewed_first=False, timeout=15.0) or state
    state = sequence_rank_node(state) or state


state["strain"] = {
    "objective_id": "BIOMASS_Ec_iML1515_core_75p37M",
    "product_rxn":  state.get("simulate",{}).get("product_id") or None,  # or set explicit rxn id
    "min_growth":   0.05,
    "max_kos":      2,
    "solver":       "glpk",
    "fva":          True,
    "time_limit_s": 300,
    "search":       "auto",   # "heuristic" if you want to skip cameo
    "medium":       {"EX_glc__D_e": -10.0},

    # Candidate chooser heuristics
    "ko_filters": {
        "skip_exchange": True,
        "skip_transport": True,
        "protect_essentials": True,
        "prefer_subsystems": ["Glycolysis/Gluconeogenesis","TCA cycle","Pentose Phosphate Pathway"],
        "max_candidates": 100
    },

    # Optional triples sweep
    "triples": {"enable": False, "top_k": 30, "patience": 50}
}


# Strain optimization → Build → Test → Learn → Human gate → Export
state = strain_optimize_node(state) or state
state = build_node(state) or state
state = test_node(state) or state
state = learn_node(state) or state
state = human_gate_node(state)
state = export_node(state)

# 4) Stream logs
for l in state.get("logs", []):
    print(l)


✅ SBML model ready: /content/models/iML1515.xml
▶ start extract
extracted pathways from pathways_solved_steps.csv (82 step rows, 23 pathways)
✔ done extract
▶ start add_selenzyme_links
selenzyme_links: filled 82 URLs out of 82 rows → /content/runs/run_001/retro_finish_out/steps_enzyme_plan.csv
✔ done add_selenzyme_links
compound_map: wrote /content/runs/run_001/retro_finish_out/metabolite_map.csv (inchikey filled 0 / 25)
▶ start thermo
thermo: token IDs available for 0 / 23 tokens
thermo: id routes: {'via_explicit_id': 0, 'via_inchikey_literal': 0, 'via_rdkit_inchikey': 0, 'via_rdkit_inchikey_only': 23, 'via_pubchem_kegg': 0, 'via_pubchem_smiles': 0, 'via_pubchem_cas': 0, 'via_match_smiles': 0, 'via_name': 0, 'unmapped': 23}
thermo: NOTE – RDKit produced InChIKeys but none resolved to KEGG via PubChem.
thermo: example tokens with IK but no KEGG (first 5): ['CNC(=O)C(=O)c1ccc(O)c(OC)c1', 'CNC(=O)C(O)c1ccc(O)c(OC)c1', 'COC1=C(C=CC(=C1)C=O)O', 'COc1cc(C(=O)C#N)ccc1O', 'COc1cc(C(=O)C(=O)C(